In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os
import numpy as np
import pandas as pd
import cv2
import re
from scipy import ndimage
from skimage import io, measure
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import umap
from tqdm import tqdm

# Function to extract sample ID from new filename structure
def extract_sample_id(filename):
    """
    Extract the sample ID from a filename based on the new naming pattern for Static-x20.

    Examples:
    - '0Pa_A1_20dec21_20xA_L2RA_FlatA_seq012_cell_mask_merged_conservative.tif'
      → '0Pa_A1_20dec21_20xA_L2RA_FlatA_seq012'
    - 'denoised_0Pa_A1_20dec21_20xA_L2RA_FlatA_seq012_Nuclei_filtered_mask.tif'
      → '0Pa_A1_20dec21_20xA_L2RA_FlatA_seq012'
    """
    # Remove file extension
    base_name = os.path.splitext(filename)[0]

    # Handle special prefixes like "denoised_"
    if base_name.startswith('denoised_'):
        base_name = base_name[len('denoised_'):]

    # Regex to capture the part up to _seqXXX including all preceding identifiers
    # This pattern looks for a structure ending in _seq followed by digits.
    # It aims to capture everything from the start up to and including the sequence number.
    # Example: 0Pa_A1_20dec21_20xA_L2RA_FlatA_seq012
    pattern = re.compile(r'(.+?_seq\d+)')
    match = pattern.search(base_name)

    if match:
        # The first group is the desired sample ID
        extracted_id = match.group(1)
        # Further clean-up: remove common suffixes if they were accidentally captured before _seq
        # This step might need adjustment if the pattern before _seqXXX varies significantly
        # For the provided examples, this should be fine.
        # If specific suffixes like '_cell_mask_merged_conservative' or '_Nuclei_filtered_mask'
        # are part of the match *before* _seqXXX (which is unlikely with the current regex),
        # they would need to be stripped here. However, the regex `(.+?_seq\d+)`
        # should correctly stop at `_seq\d+`.

        return extracted_id

    # Fallback: if the regex didn't work, try a simpler split,
    # assuming 'seq' is a reliable delimiter.
    parts = base_name.split('_')
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0: # Ensure 'seq' is not the first part
            # Join all parts up to and including the 'seqXXX' part
            return '_'.join(parts[:i+1])

    # Last resort: return a significant portion of the filename if specific patterns fail
    # This is a very basic fallback and might not be robust.
    print(f"Warning: Could not robustly extract sample ID from '{filename}'. Using fallback.")
    return '_'.join(base_name.split('_')[:7]) # Adjust number of parts as a heuristic

def find_mask_files(cell_dir, nuclei_dir):
    """Finds and pairs cell and nuclei mask files based on extracted sample ID."""
    print("\n--- Finding and Pairing Mask Files ---")

    # Get all relevant mask files
    cell_files = [f for f in os.listdir(cell_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    nuclei_files = [f for f in os.listdir(nuclei_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]

    print(f"Found {len(cell_files)} cell mask files and {len(nuclei_files)} nuclei mask files")

    # Create lookup dictionary for nuclei files with extracted sample IDs
    nuclei_lookup = {}
    for nuclei_file in nuclei_files:
        sample_id = extract_sample_id(nuclei_file)
        if sample_id:
            nuclei_lookup[sample_id] = nuclei_file
            # print(f"Nuclei file: '{nuclei_file}' → Sample ID: '{sample_id}'") # Verbose
        else:
            print(f"Warning: Could not extract sample ID for nuclei file: {nuclei_file}")


    # Match cell files to nuclei files
    file_pairs = []
    pairs_found = 0

    for cell_file in cell_files:
        sample_id = extract_sample_id(cell_file)
        # print(f"Cell file: '{cell_file}' → Sample ID: '{sample_id}'") # Verbose

        if sample_id and sample_id in nuclei_lookup:
            # print(f"Match found for ID: {sample_id}") # Verbose
            nuclei_file = nuclei_lookup[sample_id]
            file_pair = {
                'cell_file': os.path.join(cell_dir, cell_file),
                'nuclei_file': os.path.join(nuclei_dir, nuclei_file),
                'sample_id': sample_id
            }
            file_pairs.append(file_pair)
            pairs_found += 1
        elif sample_id:
            print(f"Warning: No matching nuclei file found for cell sample ID: {sample_id} (from file {cell_file})")
        else:
            print(f"Warning: Could not extract sample ID for cell file: {cell_file}")


    print(f"Total matching cell-nuclei file pairs found: {pairs_found}")
    if pairs_found == 0 and (len(cell_files) > 0 or len(nuclei_files) > 0) :
        print("CRITICAL WARNING: No file pairs were matched. Please check the `extract_sample_id` function and your filenames.")
        print("Example cell filenames:")
        for cf in cell_files[:3]: print(f"  - {cf} -> extracted ID: {extract_sample_id(cf)}")
        print("Example nuclei filenames:")
        for nf in nuclei_files[:3]: print(f"  - {nf} -> extracted ID: {extract_sample_id(nf)}")


    return file_pairs

def load_mask_image(filepath):
    """Loads a mask image, ensuring it's binary (0 or 1) or labeled."""
    try:
        img = io.imread(filepath)
        # Convert to a common integer type for labels, e.g., uint16
        # If the image is already labeled (e.g., integer types with max > 1)
        if img.dtype.kind in 'iu' and np.max(img) > 1: # integer unsigned/signed
            img = img.astype(np.uint16)
        # If it's boolean, convert to uint8 (0 or 1) then label
        elif img.dtype == bool:
            img = img.astype(np.uint8)
            img, _ = ndimage.label(img)
            img = img.astype(np.uint16)
        # If it's float, threshold, convert to uint8, then label
        elif img.dtype.kind == 'f':
            img = (img > 0.5).astype(np.uint8) # Assuming >0.5 is foreground
            img, _ = ndimage.label(img)
            img = img.astype(np.uint16)
        # If it's uint8 and likely binary (max is 0 or 1)
        elif img.dtype == np.uint8 and np.max(img) <= 1:
            img, _ = ndimage.label(img)
            img = img.astype(np.uint16)
        # If it's already uint8 and max > 1, assume it's labeled
        elif img.dtype == np.uint8 and np.max(img) > 1:
            img = img.astype(np.uint16)
        else: # Fallback for other types, attempt to label if it looks binary
            print(f"Warning: Unhandled image type {img.dtype} for {filepath}. Attempting binary conversion and labeling.")
            try:
                img_binary = (img > np.median(img)).astype(np.uint8) # Basic thresholding
                img, _ = ndimage.label(img_binary)
                img = img.astype(np.uint16)
            except Exception as e_conv:
                print(f"Error converting and labeling image {filepath}: {str(e_conv)}")
                return None


        # Handle multi-channel images (e.g. from saving labeled images as RGB)
        # This usually means the labels are replicated across channels or in one channel.
        if img.ndim > 2:
            print(f"Warning: Image {filepath} has multiple channels ({img.shape}). Taking first channel.")
            img = img[..., 0] # Take the first channel
            # Re-check if labeling is needed after taking one channel
            if np.max(img) <= 1:
                 img, _ = ndimage.label(img)
                 img = img.astype(np.uint16)


        return img
    except Exception as e:
        print(f"Error loading image {filepath}: {str(e)}")
        return None

def accurately_track_nuclei_in_cells(cell_mask, nuclei_mask):
    """
    Accurately identifies which nuclei are inside which cells.
    Assumes cell_mask and nuclei_mask are already labeled integer masks.
    """
    # Ensure masks are properly labeled (they should be from load_mask_image)
    labeled_cells = cell_mask
    num_cells = np.max(labeled_cells)

    labeled_nuclei = nuclei_mask
    num_nuclei = np.max(labeled_nuclei)

    print(f"Found {num_cells} potential cell objects and {num_nuclei} potential nuclei objects.")
    if num_cells == 0 or num_nuclei == 0:
        print("Warning: No cells or no nuclei found in the masks.")
        return {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}


    # Extract properties for cells and nuclei
    # Note: intensity_image can be the mask itself for regionprops if no other image is used for intensity measures
    cell_props = measure.regionprops(labeled_cells, intensity_image=labeled_cells)
    nuclei_props = measure.regionprops(labeled_nuclei, intensity_image=labeled_nuclei)

    results = {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    # Process each cell
    for cell_region in cell_props:
        cell_id = cell_region.label # This is the integer label from the mask
        cell_data = {
            'cell_id': cell_id, # Store the original label
            'area': cell_region.area,
            'perimeter': cell_region.perimeter,
            'eccentricity': cell_region.eccentricity if hasattr(cell_region, 'eccentricity') else 0,
            'orientation': np.degrees(cell_region.orientation) if hasattr(cell_region, 'orientation') else None,
            'major_axis_length': cell_region.major_axis_length if hasattr(cell_region, 'major_axis_length') else None,
            'minor_axis_length': cell_region.minor_axis_length if hasattr(cell_region, 'minor_axis_length') else None,
            'centroid_y': cell_region.centroid[0],
            'centroid_x': cell_region.centroid[1],
            'nuclei_count': 0,
            'nuclei_ids_in_cell': [] # Store IDs of nuclei within this cell
        }
        results['cell_data'].append(cell_data)
        results['cell_nuclei_mapping'][cell_id] = []


    # Find which nuclei belong to which cells
    for nucleus_region in nuclei_props:
        nucleus_id = nucleus_region.label # Original label from nuclei mask
        nucleus_mask_binary_single = (labeled_nuclei == nucleus_id) # Mask for this specific nucleus
        nucleus_area = nucleus_region.area

        contained_in_cell_id = None
        max_overlap_ratio = 0

        # Find which cell label this nucleus overlaps with the most
        # We check the label of the cell_mask at the nucleus's centroid
        # or, more robustly, by overlap percentage.

        # Method 1: Centroid-based (simpler, but can fail for complex shapes)
        # nuc_centroid_y, nuc_centroid_x = int(nucleus_region.centroid[0]), int(nucleus_region.centroid[1])
        # if 0 <= nuc_centroid_y < labeled_cells.shape[0] and 0 <= nuc_centroid_x < labeled_cells.shape[1]:
        #    cell_label_at_nuc_centroid = labeled_cells[nuc_centroid_y, nuc_centroid_x]
        #    if cell_label_at_nuc_centroid > 0: # Belongs to a cell
        #        # Check overlap for confirmation
        #        cell_mask_binary_single = (labeled_cells == cell_label_at_nuc_centroid)
        #        overlap = np.logical_and(cell_mask_binary_single, nucleus_mask_binary_single)
        #        overlap_area = np.sum(overlap)
        #        if nucleus_area > 0: # Avoid division by zero
        #             overlap_ratio = overlap_area / nucleus_area
        #             if overlap_ratio > 0.5: # More than 50% of nucleus in this cell
        #                 contained_in_cell_id = cell_label_at_nuc_centroid
        #                 max_overlap_ratio = overlap_ratio

        # Method 2: Overlap-based (more robust) - Iterate through cells (can be slow if many cells)
        # A faster overlap method:
        # Get the cell labels that overlap with the current nucleus's bounding box first
        min_row, min_col, max_row, max_col = nucleus_region.bbox
        candidate_cell_labels = np.unique(labeled_cells[min_row:max_row, min_col:max_col])

        for cell_label_candidate in candidate_cell_labels:
            if cell_label_candidate == 0: continue # Skip background

            cell_mask_binary_single_candidate = (labeled_cells == cell_label_candidate)
            overlap = np.logical_and(cell_mask_binary_single_candidate, nucleus_mask_binary_single)
            overlap_area = np.sum(overlap)

            if nucleus_area > 0:
                overlap_ratio = overlap_area / nucleus_area
                if overlap_ratio > max_overlap_ratio:
                    max_overlap_ratio = overlap_ratio
                    contained_in_cell_id = cell_label_candidate

        # Assign nucleus if overlap > 50%
        if contained_in_cell_id is not None and max_overlap_ratio > 0.5:
            nucleus_data = {
                'nucleus_id': nucleus_id, # Original label
                'cell_id': contained_in_cell_id, # Original label of the cell it's in
                'area': nucleus_area,
                'eccentricity': nucleus_region.eccentricity if hasattr(nucleus_region, 'eccentricity') else 0,
                'centroid_y': nucleus_region.centroid[0],
                'centroid_x': nucleus_region.centroid[1],
                'overlap_ratio': max_overlap_ratio
            }
            results['nuclei_data'].append(nucleus_data)

            # Update cell's nuclei count and list of nuclei
            for cd in results['cell_data']:
                if cd['cell_id'] == contained_in_cell_id:
                    cd['nuclei_count'] += 1
                    cd['nuclei_ids_in_cell'].append(nucleus_id)
                    break
            results['cell_nuclei_mapping'][contained_in_cell_id].append(nucleus_id)

    # Summary printouts
    cells_with_nuclei_count = sum(1 for cd in results['cell_data'] if cd['nuclei_count'] > 0)
    if num_cells > 0:
        print(f"Cells with at least one nucleus: {cells_with_nuclei_count}/{num_cells} ({100*cells_with_nuclei_count/num_cells:.1f}% of cells)")
        nuclei_counts_dist = [cd['nuclei_count'] for cd in results['cell_data']]
        for count_val in sorted(set(nuclei_counts_dist)):
            num_cells_with_this_count = sum(1 for n in nuclei_counts_dist if n == count_val)
            print(f"  Cells with {count_val} nuclei: {num_cells_with_this_count} ({100*num_cells_with_this_count/num_cells:.1f}%)")
    else:
        print("No cells found to analyze nuclei counts.")

    return results


def extract_features_for_cell(cell_data_item, cell_nuclei_map, nuclei_data_list):
    """
    Extract comprehensive morphometric features for a cell and its associated nuclei.
    'cell_data_item' is a single dictionary for one cell from results['cell_data'].
    'nuclei_data_list' is the full list of results['nuclei_data'].
    """
    features = {}

    # Basic cell features
    features['cell_area'] = cell_data_item['area']
    features['cell_perimeter'] = cell_data_item['perimeter']
    features['cell_eccentricity'] = cell_data_item.get('eccentricity', 0) # Use .get for safety

    if cell_data_item['perimeter'] > 0:
        features['cell_circularity'] = 4 * np.pi * cell_data_item['area'] / (cell_data_item['perimeter'] ** 2)
    else:
        features['cell_circularity'] = 0

    major_axis = cell_data_item.get('major_axis_length')
    minor_axis = cell_data_item.get('minor_axis_length')
    if major_axis is not None and minor_axis is not None and minor_axis > 0:
        features['cell_aspect_ratio'] = major_axis / minor_axis
    else:
        features['cell_aspect_ratio'] = 1.0


    # Nuclear features
    # cell_id here is the original integer label from the mask
    original_cell_id_label = cell_data_item['cell_id']
    # Get the list of nucleus original labels associated with this cell's original label
    nucleus_original_labels_in_cell = cell_nuclei_map.get(original_cell_id_label, [])
    features['nuclei_count'] = len(nucleus_original_labels_in_cell)


    # Initialize nuclear features with defaults
    features['avg_nucleus_area'] = 0
    features['total_nuclear_area'] = 0
    features['max_nucleus_area'] = 0
    features['avg_nucleus_eccentricity'] = 0
    features['nucleus_area_std'] = 0
    features['nucleus_displacement_avg'] = 0
    features['nucleus_to_cell_area_ratio'] = 0


    if features['nuclei_count'] > 0:
        # Filter nuclei_data_list to get only those nuclei whose original_label is in nucleus_original_labels_in_cell
        # AND whose 'cell_id' (which is the cell's original_label they were assigned to) matches current cell's original_cell_id_label

        # This is a bit redundant if cell_nuclei_map is correctly built by accurately_track_nuclei_in_cells
        # We can directly use nucleus_original_labels_in_cell to filter nuclei_data_list

        cell_nuclei_details = [n_data for n_data in nuclei_data_list if n_data['nucleus_id'] in nucleus_original_labels_in_cell and n_data['cell_id'] == original_cell_id_label]

        if cell_nuclei_details: # Ensure we found matching nuclei details
            nuclear_areas = [n['area'] for n in cell_nuclei_details]
            nuclear_eccentricities = [n.get('eccentricity', 0) for n in cell_nuclei_details]

            features['avg_nucleus_area'] = np.mean(nuclear_areas) if nuclear_areas else 0
            features['total_nuclear_area'] = sum(nuclear_areas)
            features['max_nucleus_area'] = max(nuclear_areas) if nuclear_areas else 0
            features['avg_nucleus_eccentricity'] = np.mean(nuclear_eccentricities) if nuclear_eccentricities else 0
            features['nucleus_area_std'] = np.std(nuclear_areas) if len(nuclear_areas) > 1 else 0

            if features['cell_area'] > 0:
                features['nucleus_to_cell_area_ratio'] = features['total_nuclear_area'] / features['cell_area']

            # Calculate average displacement of nuclei from cell center
            displacements = []
            for nucleus_detail in cell_nuclei_details:
                dx = nucleus_detail['centroid_x'] - cell_data_item['centroid_x']
                dy = nucleus_detail['centroid_y'] - cell_data_item['centroid_y']
                displacements.append(np.sqrt(dx**2 + dy**2))
            features['nucleus_displacement_avg'] = np.mean(displacements) if displacements else 0
        else:
             print(f"Warning: Cell {original_cell_id_label} reported {features['nuclei_count']} nuclei, but no matching details found in nuclei_data_list.")


    # Derived features for senescence (can be refined)
    # These are placeholders and should be calibrated based on your specific dataset and biological knowledge.
    avg_normal_nucleus_area = 500  # Placeholder - determine from control data if possible
    avg_normal_cell_area = 2000    # Placeholder - determine from control data

    features['is_polynucleated'] = 1 if features['nuclei_count'] > 1 else 0
    features['nuclear_enlargement_factor'] = features['avg_nucleus_area'] / avg_normal_nucleus_area if avg_normal_nucleus_area > 0 and features['avg_nucleus_area'] > 0 else 1.0
    features['cell_enlargement_factor'] = features['cell_area'] / avg_normal_cell_area if avg_normal_cell_area > 0 and features['cell_area'] > 0 else 1.0

    return features


def perform_clustering(all_features_df, umap_n_neighbors=30, umap_min_dist=0.1, umap_random_state=42, kmeans_random_state=42):
    """
    Performs UMAP dimensionality reduction and k-Means clustering.
    (This function is largely kept from your original, ensure features used are appropriate)
    """
    print("Starting clustering process...")
    if all_features_df.empty:
        print("Input DataFrame is empty. Cannot perform clustering.")
        return pd.DataFrame()

    # Define feature columns for clustering (exclude IDs and labels)
    potential_id_cols = ['cell_id_unique', 'sample_id', 'original_cell_label'] # Add any other ID columns
    feature_columns = [col for col in all_features_df.columns if col not in potential_id_cols and col not in ['umap_x', 'umap_y', 'cluster', 'cell_type']]

    numeric_feature_columns = []
    for col in feature_columns:
        if pd.api.types.is_numeric_dtype(all_features_df[col]):
            numeric_feature_columns.append(col)
        else:
            try:
                all_features_df[col] = pd.to_numeric(all_features_df[col])
                numeric_feature_columns.append(col)
            except ValueError:
                print(f"Warning: Column {col} is not numeric and could not be converted. Excluded from clustering.")

    feature_columns = numeric_feature_columns
    if not feature_columns:
        print("No valid numeric feature columns found for clustering.")
        return all_features_df # Or an empty DataFrame

    features_for_clustering_df = all_features_df[feature_columns].copy()
    # Handle NaNs by filling with mean (or a more sophisticated method if needed)
    features_for_clustering_df = features_for_clustering_df.fillna(features_for_clustering_df.mean())


    # Log Transform Area-based Features (before scaling)
    area_features_to_log = ['cell_area', 'avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'cell_perimeter']
    print("\nApplying log transformation to selected area/perimeter features for clustering...")
    for col in area_features_to_log:
        if col in features_for_clustering_df.columns:
            features_for_clustering_df[col] = np.log1p(features_for_clustering_df[col]) # log1p handles zeros

    # Standardize Features
    print("\nStandardizing features for clustering...")
    scaler = StandardScaler()
    features_standardized = scaler.fit_transform(features_for_clustering_df)
    # features_standardized_df = pd.DataFrame(features_standardized, columns=feature_columns, index=features_for_clustering_df.index) # Not strictly needed if only using array

    # UMAP Dimensionality Reduction
    print("\nPerforming UMAP reduction...")
    # Ensure n_neighbors is less than the number of samples
    actual_umap_n_neighbors = min(umap_n_neighbors, features_standardized.shape[0] - 1)
    if actual_umap_n_neighbors < 2: # UMAP requires at least 2 neighbors
        print(f"Warning: Not enough samples ({features_standardized.shape[0]}) for UMAP with n_neighbors={umap_n_neighbors}. Skipping UMAP and KMeans.")
        clustered_df = all_features_df.copy()
        clustered_df['umap_x'] = np.nan
        clustered_df['umap_y'] = np.nan
        clustered_df['cluster'] = np.nan
        clustered_df['cell_type'] = 'Unknown' # Default if clustering fails
        return clustered_df

    reducer = umap.UMAP(n_neighbors=actual_umap_n_neighbors, min_dist=umap_min_dist, random_state=umap_random_state, n_components=2)
    embedding = reducer.fit_transform(features_standardized)

    clustered_df = all_features_df.copy()
    clustered_df['umap_x'] = embedding[:, 0]
    clustered_df['umap_y'] = embedding[:, 1]

    # k-Means Clustering (k=2 for Senescent/Non-senescent)
    print("\nPerforming k-Means clustering (k=2)...")
    if embedding.shape[0] >= 2: # KMeans needs at least n_clusters samples
        kmeans = KMeans(n_clusters=2, random_state=kmeans_random_state, n_init='auto')
        clustered_df['cluster'] = kmeans.fit_predict(embedding) # Cluster on UMAP embedding
    else:
        print("Warning: Not enough samples for KMeans clustering. Assigning NaN to clusters.")
        clustered_df['cluster'] = np.nan


    # Identify Senescent Cluster (heuristic based on your previous logic)
    # This part is highly dependent on your data and how features separate senescent cells.
    # The original notebook used a weighted score on cluster means.
    print("\nIdentifying senescent cluster (heuristic based on UMAP and feature means)...")
    if 'cluster' in clustered_df.columns and clustered_df['cluster'].notna().any():
        # Simplified heuristic: Assume cluster with larger mean cell_area is senescent
        # This needs to be validated and potentially replaced with your more robust scoring.
        # For now, using a placeholder.
        try:
            cluster_means = clustered_df.groupby('cluster')[['cell_area', 'avg_nucleus_area']].mean()
            # A simple way: senescent cluster has higher mean cell_area
            if not cluster_means.empty and 'cell_area' in cluster_means.columns:
                 senescent_cluster_label = cluster_means['cell_area'].idxmax()
                 print(f"Cluster {senescent_cluster_label} tentatively identified as 'Senescent' based on max mean cell_area.")
            else: # Fallback if groupby fails or cell_area not available
                senescent_cluster_label = 0 # Default to 0
                print("Warning: Could not determine senescent cluster based on cell_area. Defaulting.")

            clustered_df['cell_type'] = clustered_df['cluster'].apply(
                lambda x: 'Senescent' if x == senescent_cluster_label else 'Non-senescent'
            )
            # Apply multinucleation rule (cells with > 1 nucleus are senescent)
            # This threshold might need adjustment (e.g. >2)
            if 'nuclei_count' in clustered_df.columns:
                print("Applying multinucleation rule (nuclei_count > 1)...")
                clustered_df.loc[clustered_df['nuclei_count'] > 1, 'cell_type'] = 'Senescent'
            else:
                print("Warning: 'nuclei_count' not in DataFrame. Multinucleation rule not applied.")
        except Exception as e_clust_id:
            print(f"Error identifying senescent cluster: {e_clust_id}. Defaulting cell_type.")
            clustered_df['cell_type'] = 'Unknown'
    else:
        print("Clustering did not produce valid cluster labels. Setting cell_type to 'Unknown'.")
        clustered_df['cell_type'] = 'Unknown'


    print("\nClustering process complete.")
    return clustered_df

def visualize_clustering_results(clustered_df, output_dir_main):
    """Create visualizations of the clustering results."""
    print("\n--- Creating Visualizations ---")
    if clustered_df.empty or 'umap_x' not in clustered_df.columns or clustered_df['umap_x'].isnull().all():
        print("Skipping visualizations as UMAP data is missing or empty.")
        return

    # UMAP plot colored by final cell_type
    plt.figure(figsize=(10, 8))
    if 'cell_type' in clustered_df.columns and clustered_df['cell_type'].notna().any():
        sns.scatterplot(
            x='umap_x', y='umap_y',
            hue='cell_type',
            palette={'Senescent': 'red', 'Non-senescent': 'blue', 'Unknown': 'grey'},
            data=clustered_df,
            s=30, alpha=0.7, edgecolor='k', linewidth=0.5
        )
        plt.title('UMAP Projection by Final Cell Type', fontsize=14)
    else: # Fallback if cell_type is not available, color by cluster if possible
        if 'cluster' in clustered_df.columns and clustered_df['cluster'].notna().any():
             sns.scatterplot(x='umap_x', y='umap_y', hue='cluster', data=clustered_df, palette='viridis', s=30, alpha=0.7)
             plt.title('UMAP Projection by KMeans Cluster', fontsize=14)
        else: # Just plot UMAP points
             plt.scatter(clustered_df['umap_x'], clustered_df['umap_y'], s=30, alpha=0.5)
             plt.title('UMAP Projection (No Color)', fontsize=14)


    plt.xlabel('UMAP Dimension 1', fontsize=12)
    plt.ylabel('UMAP Dimension 2', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(title='Cell Type' if 'cell_type' in clustered_df.columns else 'Cluster')
    plt.savefig(os.path.join(output_dir_main, 'umap_final_cell_type.png'), dpi=300, bbox_inches='tight')
    plt.close()
    print("UMAP visualization saved.")

    # Key feature distributions
    key_features_for_plot = ['cell_area', 'avg_nucleus_area', 'nuclei_count', 'cell_circularity', 'nucleus_to_cell_area_ratio']
    key_features_present = [f for f in key_features_for_plot if f in clustered_df.columns and clustered_df[f].notna().any()]

    if key_features_present and 'cell_type' in clustered_df.columns and clustered_df['cell_type'].notna().any():
        num_feats = len(key_features_present)
        fig, axes = plt.subplots( (num_feats + 1)//2 , 2, figsize=(12, 3 * ((num_feats + 1)//2) ), squeeze=False)
        axes = axes.flatten()
        for i, feature in enumerate(key_features_present):
            sns.boxplot(x='cell_type', y=feature, data=clustered_df, ax=axes[i], palette={'Senescent': 'red', 'Non-senescent': 'blue', 'Unknown': 'grey'})
            axes[i].set_title(feature.replace('_', ' ').title(), fontsize=10)
            axes[i].set_xlabel('')
            axes[i].set_ylabel('')

        # Hide any unused subplots
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir_main, 'feature_distributions_by_type.png'), dpi=300, bbox_inches='tight')
        plt.close()
        print("Feature distribution boxplots saved.")


def save_results(clustered_df, output_directory):
    """Save the clustering results to CSV files."""
    if clustered_df.empty:
        print("No data to save.")
        return

    # Create output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Save full results
    clustered_df.to_csv(os.path.join(output_directory, 'cell_classification_full_results.csv'), index=False)
    print(f"Full cell classification results saved to: {os.path.join(output_directory, 'cell_classification_full_results.csv')}")


    # Create summary by sample
    if 'sample_id' in clustered_df.columns and 'cell_type' in clustered_df.columns:
        sample_summary = clustered_df.groupby('sample_id')['cell_type'].value_counts(normalize=False).unstack(fill_value=0)
        if 'Senescent' not in sample_summary.columns: sample_summary['Senescent'] = 0
        if 'Non-senescent' not in sample_summary.columns: sample_summary['Non-senescent'] = 0
        if 'Unknown' not in sample_summary.columns: sample_summary['Unknown'] = 0

        sample_summary['total_cells'] = sample_summary.sum(axis=1)
        if 'total_cells' in sample_summary.columns and (sample_summary['total_cells'] > 0).any() :
             sample_summary['percent_senescent'] = sample_summary['Senescent'] / sample_summary['total_cells'] * 100
             sample_summary['percent_non_senescent'] = sample_summary['Non-senescent'] / sample_summary['total_cells'] * 100
             sample_summary['percent_unknown'] = sample_summary['Unknown'] / sample_summary['total_cells'] * 100
        else:
            sample_summary['percent_senescent'] = 0
            sample_summary['percent_non_senescent'] = 0
            sample_summary['percent_unknown'] = 0


        sample_summary.to_csv(os.path.join(output_directory, 'sample_summary_by_type.csv'))
        print(f"Sample summary saved to: {os.path.join(output_directory, 'sample_summary_by_type.csv')}")

        # Plot senescent percentage by sample
        if 'percent_senescent' in sample_summary.columns:
            plt.figure(figsize=(12, 6))
            sns.barplot(x=sample_summary.index, y='percent_senescent', data=sample_summary.reset_index(), color='red')
            plt.title('Percentage of Senescent Cells by Sample')
            plt.xlabel('Sample ID')
            plt.ylabel('Senescent Cells (%)')
            plt.xticks(rotation=90, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_directory, 'senescent_percentage_by_sample.png'), dpi=300)
            plt.close()
            print("Senescent percentage bar plot saved.")


def main(base_dir, cell_subdir, nuclei_subdir, output_parent_dir):
    """
    Main function to perform the analysis for the Static-x20 dataset.
    """
    print(f"=== Senescent Cell Classification Analysis for {base_dir} ===")

    cell_mask_dir = os.path.join(base_dir, cell_subdir)
    nuclei_dir = os.path.join(base_dir, nuclei_subdir)

    # Construct a unique output directory for this run
    # e.g. /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/Static-x20/Senescence_Results
    dataset_name = os.path.basename(base_dir) # Should be "Static-x20"
    output_dir_main = os.path.join(output_parent_dir, dataset_name, "Senescence_Results")

    if not os.path.exists(output_dir_main):
        os.makedirs(output_dir_main)
        print(f"Created output directory: {output_dir_main}")

    # Find and pair mask files
    file_pairs = find_mask_files(cell_mask_dir, nuclei_dir)
    if not file_pairs:
        print("No matching file pairs found. Exiting analysis.")
        return

    all_cell_features_list = []

    for file_pair in tqdm(file_pairs, desc="Processing image pairs"):
        print(f"\nProcessing sample: {file_pair['sample_id']}")
        try:
            cell_mask = load_mask_image(file_pair['cell_file'])
            nuclei_mask = load_mask_image(file_pair['nuclei_file'])

            if cell_mask is None or nuclei_mask is None:
                print(f"Skipping {file_pair['sample_id']} due to mask loading error.")
                continue

            # Track nuclei in cells
            # This function returns a dict: {'cell_data': list_of_cell_dicts, 'nuclei_data': list_of_nuc_dicts, 'cell_nuclei_mapping': dict}
            tracking_results = accurately_track_nuclei_in_cells(cell_mask, nuclei_mask)

            # Extract features for each cell in the current image pair
            for single_cell_data in tracking_results['cell_data']:
                # 'single_cell_data' is a dict for one cell, includes its original_label as 'cell_id'
                # 'tracking_results['cell_nuclei_mapping']' maps cell original_label to list of nuclei original_labels
                # 'tracking_results['nuclei_data']' is a list of dicts for all nuclei in the image

                # Ensure the cell_id from single_cell_data is the original label used in mapping
                cell_original_label = single_cell_data['cell_id']

                features = extract_features_for_cell(
                    single_cell_data,
                    tracking_results['cell_nuclei_mapping'], # Pass the whole map
                    tracking_results['nuclei_data']          # Pass the list of all nuclei data for this image
                )

                # Create a globally unique cell ID and store original label
                features['cell_id_unique'] = f"{file_pair['sample_id']}_{cell_original_label}"
                features['sample_id'] = file_pair['sample_id']
                features['original_cell_label'] = cell_original_label # Keep track of original mask label

                all_cell_features_list.append(features)

        except Exception as e:
            print(f"Error processing {file_pair['sample_id']}: {str(e)}")
            import traceback
            traceback.print_exc()


    if not all_cell_features_list:
        print("No cell features were extracted. Please check input files and processing steps.")
        return

    all_features_df = pd.DataFrame(all_cell_features_list)
    print(f"\nTotal cells extracted across all samples: {len(all_features_df)}")

    # Save raw features before clustering (optional, for debugging)
    # all_features_df.to_csv(os.path.join(output_dir_main, "raw_features_extracted.csv"), index=False)


    # Perform clustering and classification
    clustered_df = perform_clustering(all_features_df)

    # Save results (CSV and summary plots)
    save_results(clustered_df, output_dir_main)

    # Visualize clustering results (UMAP plots, feature distributions)
    visualize_clustering_results(clustered_df, output_dir_main)


    print("\nAnalysis complete!")


if __name__ == "__main__":
    # Define base directory for the new dataset
    # Assuming "Static-x20" is at the same level as "flow3-x20"
    # /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20

    # Path to the parent directory containing different datasets like "flow3-x20", "Static-x20"
    segmented_data_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented"
    # Name of the current dataset's folder
    current_dataset_folder_name = "1.4Pa-x20"

    # Construct the base_dir for the current dataset
    base_input_dir = os.path.join(segmented_data_parent_dir, current_dataset_folder_name)

    # Subdirectory names (assumed to be the same)
    cell_masks_subdir = "Cell_merged_conservative"
    nuclei_masks_subdir = "Nuclei"

    # Parent directory for all analysis output
    analysis_output_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis"

    # Check if the base input directory exists
    if not os.path.isdir(base_input_dir):
        print(f"Error: Base input directory not found: {base_input_dir}")
        print("Please ensure the 'Static-x20' folder exists at the specified location.")
    else:
        main(base_input_dir, cell_masks_subdir, nuclei_masks_subdir, analysis_output_parent_dir)



=== Senescent Cell Classification Analysis for /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20 ===

--- Finding and Pairing Mask Files ---
Found 23 cell mask files and 23 nuclei mask files
Total matching cell-nuclei file pairs found: 23


Processing image pairs:   0%|          | 0/23 [00:00<?, ?it/s]


Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001
Found 326 potential cell objects and 347 potential nuclei objects.


Processing image pairs:   4%|▍         | 1/23 [00:01<00:25,  1.18s/it]

Cells with at least one nucleus: 326/326 (100.0% of cells)
  Cells with 1 nuclei: 309 (94.8%)
  Cells with 2 nuclei: 16 (4.9%)
  Cells with 3 nuclei: 1 (0.3%)

Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002
Found 273 potential cell objects and 290 potential nuclei objects.


...

Cells with at least one nucleus: 301/301 (100.0% of cells)
  Cells with 1 nuclei: 286 (95.0%)
  Cells with 2 nuclei: 15 (5.0%)

Total cells extracted across all samples: 6663
Starting clustering process...

Applying log transformation to selected area/perimeter features for clustering...

Standardizing features for clustering...

Performing UMAP reduction...



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(



Performing k-Means clustering (k=2)...

Identifying senescent cluster (heuristic based on UMAP and feature means)...
Cluster 1 tentatively identified as 'Senescent' based on max mean cell_area.
Applying multinucleation rule (nuclei_count > 1)...

Clustering process complete.
Full cell classification results saved to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/cell_classification_full_results.csv
Sample summary saved to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/sample_summary_by_type.csv
Senescent percentage bar plot saved.

--- Creating Visualizations ---
UMAP visualization saved.


<ipython-input-11-473482782ca3>:551: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='cell_type', y=feature, data=clustered_df, ax=axes[i], palette={'Senescent': 'red', 'Non-senescent': 'blue', 'Unknown': 'grey'})
<ipython-input-11-473482782ca3>:551: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='cell_type', y=feature, data=clustered_df, ax=axes[i], palette={'Senescent': 'red', 'Non-senescent': 'blue', 'Unknown': 'grey'})
<ipython-input-11-473482782ca3>:551: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='cell_type', y=feature, data=clust

Feature distribution boxplots saved.

Analysis complete!


In [3]:
import os
import re
from collections import defaultdict

# Function copied from your code
def extract_sample_id(filename):
    # Remove file extension
    base_name = os.path.splitext(filename)[0]

    # Handle special prefixes like "denoised_"
    if base_name.startswith('denoised_'):
        base_name = base_name[len('denoised_'):]

    # Regex to capture the part up to _seqXXX including all preceding identifiers
    pattern = re.compile(r'(.+?_seq\d+)')
    match = pattern.search(base_name)

    if match:
        return match.group(1)

    # Fallback: try a simpler split
    parts = base_name.split('_')
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0:
            return '_'.join(parts[:i+1])

    # Last resort
    print(f"Warning: Could not extract sample ID from '{filename}'")
    return '_'.join(base_name.split('_')[:7])

# Paths based on your code
base_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20"
cell_dir = os.path.join(base_dir, "Cell_merged_conservative")
nuclei_dir = os.path.join(base_dir, "Nuclei")

# Get all mask files
cell_files = [f for f in os.listdir(cell_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
nuclei_files = [f for f in os.listdir(nuclei_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]

print(f"Found {len(cell_files)} cell mask files and {len(nuclei_files)} nuclei mask files")

# Extract sample IDs for all files
cell_sample_ids = {f: extract_sample_id(f) for f in cell_files}
nuclei_sample_ids = {f: extract_sample_id(f) for f in nuclei_files}

# Create a lookup for nuclei files by sample ID
nuclei_lookup = {}
for nuclei_file, sample_id in nuclei_sample_ids.items():
    if sample_id:
        nuclei_lookup[sample_id] = nuclei_file

# Count how many cell files map to each nuclei file
nuclei_usage = defaultdict(list)
for cell_file, sample_id in cell_sample_ids.items():
    if sample_id and sample_id in nuclei_lookup:
        nuclei_file = nuclei_lookup[sample_id]
        nuclei_usage[nuclei_file].append(cell_file)

# Display mapping statistics
print("\n=== Analysis of File Pairings ===")
print(f"Number of unique sample IDs from cell files: {len(set(cell_sample_ids.values()))}")
print(f"Number of unique sample IDs from nuclei files: {len(set(nuclei_sample_ids.values()))}")
print(f"Number of sample IDs that appear in both cell and nuclei files: {len(set(cell_sample_ids.values()) & set(nuclei_sample_ids.values()))}")

# Show which nuclei files are being used multiple times
print("\n=== Nuclei Files Used Multiple Times ===")
for nuclei_file, matching_cell_files in nuclei_usage.items():
    if len(matching_cell_files) > 1:
        print(f"\nNuclei file: {nuclei_file}")
        print(f"Sample ID: {nuclei_sample_ids[nuclei_file]}")
        print(f"Used with {len(matching_cell_files)} cell files:")
        for i, cell_file in enumerate(matching_cell_files, 1):
            print(f"  {i}. {cell_file}")

# Verify total pairings matches expected count
total_pairs = sum(len(cell_files) for cell_files in nuclei_usage.values())
print(f"\nTotal file pairs found: {total_pairs}")

# Show a few example pairs
print("\n=== Example File Pairs ===")
pair_count = 0
for nuclei_file, matching_cell_files in nuclei_usage.items():
    for cell_file in matching_cell_files:
        if pair_count < 5:  # Show first 5 pairs
            print(f"Pair {pair_count+1}:")
            print(f"  Cell: {cell_file}")
            print(f"  Nuclei: {nuclei_file}")
            print(f"  Sample ID: {cell_sample_ids[cell_file]}")
            print()
        pair_count += 1

Found 23 cell mask files and 23 nuclei mask files

=== Analysis of File Pairings ===
Number of unique sample IDs from cell files: 23
Number of unique sample IDs from nuclei files: 23
Number of sample IDs that appear in both cell and nuclei files: 23

=== Nuclei Files Used Multiple Times ===

Total file pairs found: 23

=== Example File Pairs ===
Pair 1:
  Cell: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask_merged_conservative.tif
  Nuclei: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Nuclei_regional_tophat_filtered_mask.tif
  Sample ID: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001

Pair 2:
  Cell: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask_merged_conservative.tif
  Nuclei: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Nuclei_regional_tophat_filtered_mask.tif
  Sample ID: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002

Pair 3:
  Cell: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_cell_mask_merged_conservative.tif
  Nuclei: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_Nuclei_regional

In [4]:
import os
import shutil
from collections import defaultdict

# Paths
base_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20"
src_cell_dir = os.path.join(base_dir, "Cell_merged_conservative")
backup_cell_dir = os.path.join(base_dir, "Cell_merged_conservative_old")
new_cell_dir = os.path.join(base_dir, "Cell_merged_conservative")  # Will recreate this

# Create backup directory and copy all files there
if not os.path.exists(backup_cell_dir):
    print(f"Creating backup directory: {backup_cell_dir}")
    shutil.copytree(src_cell_dir, backup_cell_dir)
    print("Files backed up successfully")
else:
    print(f"Backup directory already exists: {backup_cell_dir}")

# Remove original directory and create empty new one
if os.path.exists(src_cell_dir):
    print(f"Removing original directory: {src_cell_dir}")
    shutil.rmtree(src_cell_dir)
print(f"Creating new clean directory: {new_cell_dir}")
os.makedirs(new_cell_dir)

# Group files by sample ID
cell_files = [f for f in os.listdir(backup_cell_dir) if f.endswith(('.tif', '.tiff'))]
files_by_sample_id = defaultdict(list)

print("\nGrouping files by sample ID...")
for file in cell_files:
    # Extract sample ID using your extract_sample_id function or a simpler approach
    # For simplicity, let's use a pattern that matches your files
    parts = file.split('_')
    sample_id = None
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0:
            sample_id = '_'.join(parts[:i+1])
            break

    if sample_id:
        files_by_sample_id[sample_id].append(file)

# For each sample ID, choose one file (preferring non-denoised version)
files_to_copy = []
for sample_id, file_list in files_by_sample_id.items():
    # Get non-denoised files if any exist
    non_denoised_files = [f for f in file_list if not f.startswith('denoised_')]

    if non_denoised_files:
        # Prefer non-denoised files
        chosen_file = non_denoised_files[0]
        print(f"For sample {sample_id}: Selected non-denoised file: {chosen_file}")
    else:
        # Use denoised file if that's all we have
        chosen_file = file_list[0]
        print(f"For sample {sample_id}: Selected file: {chosen_file}")

    files_to_copy.append(chosen_file)

# Copy the selected files to the new directory
print(f"\nCopying {len(files_to_copy)} files to new directory...")
for file in files_to_copy:
    src_path = os.path.join(backup_cell_dir, file)
    dst_path = os.path.join(new_cell_dir, file)
    shutil.copy2(src_path, dst_path)
    print(f"Copied: {file}")

print("\nProcess complete!")
print(f"Original files backed up to: {backup_cell_dir}")
print(f"New directory with {len(files_to_copy)} unique files: {new_cell_dir}")

Backup directory already exists: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative_old
Removing original directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative
Creating new clean directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative

Grouping files by sample ID...
For sample denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001: Selected file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask_merged_conservative.tif
For sample denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002: Selected file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask_merged_conservative.tif
For sample denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003: Selected file: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_cell_mask_merged_conservative.tif
For sample denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004: Selected file: denoised_1

In [5]:
import os
import glob

# Path to the cell mask directory
cell_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative"

# Find all files that start with "denoised_"
denoised_files = glob.glob(os.path.join(cell_dir, "denoised_*.tif*"))

# Count files before removal
total_files = len(glob.glob(os.path.join(cell_dir, "*.tif*")))
num_denoised = len(denoised_files)

print(f"Found {total_files} total files in {cell_dir}")
print(f"Found {num_denoised} files with 'denoised_' prefix")

# Ask for confirmation before deleting
print("\nFiles to be removed:")
for file in denoised_files[:5]:  # Show first 5 examples
    print(f"  - {os.path.basename(file)}")

if len(denoised_files) > 5:
    print(f"  ... and {len(denoised_files) - 5} more")

proceed = input("\nProceed with removal? (yes/no): ")

if proceed.lower() == "yes":
    # Remove each denoised file
    for file in denoised_files:
        try:
            os.remove(file)
            print(f"Removed: {os.path.basename(file)}")
        except Exception as e:
            print(f"Error removing {os.path.basename(file)}: {e}")

    # Count remaining files
    remaining = len(glob.glob(os.path.join(cell_dir, "*.tif*")))
    print(f"\nRemoval complete. {remaining} files remain in the directory.")
else:
    print("Operation canceled. No files were removed.")

Found 46 total files in /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20/Cell_merged_conservative
Found 23 files with 'denoised_' prefix

Files to be removed:
  - denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask_merged_conservative.tif
  - denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask_merged_conservative.tif
  - denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_cell_mask_merged_conservative.tif
  - denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq004_cell_mask_merged_conservative.tif
  - denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq005_cell_mask_merged_conservative.tif
  ... and 18 more

Proceed with removal? (yes/no): yes
Removed: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_cell_mask_merged_conservative.tif
Removed: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_cell_mask_merged_conservative.tif
Removed: denoised_1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_cell_mask_merged_conservative.tif
Removed: denoised_1.4Pa_A1_19dec21_20xA_L2RA_F

In [12]:
import os
import numpy as np
import pandas as pd
import cv2
import re
from scipy import ndimage
from skimage import io, measure, segmentation # Added segmentation
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# KMeans is no longer used for final classification but can be kept for exploratory UMAP clustering
from sklearn.cluster import KMeans
import umap
from tqdm import tqdm

# --- Configuration & Parameters ---
# Features to be used for UMAP generation
FEATURES_FOR_UMAP = [
    'cell_area', 'cell_perimeter', 'cell_eccentricity', 'cell_circularity',
    'cell_aspect_ratio', 'avg_nucleus_area', 'max_nucleus_area',
    'avg_nucleus_eccentricity', 'nucleus_area_std', 'nucleus_displacement_avg', # Renamed from nucleus_displacement
    'nucleus_to_cell_area_ratio',
    'nuclear_enlargement_factor', 'cell_enlargement_factor' # Renamed from nuclear_enlargement, cell_enlargement
]

# Features to log-transform before UMAP and Senescence Score calculation
AREA_FEATURES_TO_LOG = ['cell_area', 'avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'cell_perimeter']

# Weights for calculating the senescence score
SENESCENCE_SCORE_WEIGHTS = {
    'cell_area': 1.5,
    'cell_perimeter': 0.5,
    'cell_eccentricity': 0.5,
    'cell_circularity': -1.0,
    'cell_aspect_ratio': 0.5,
    'avg_nucleus_area': 1.0,
    'avg_nucleus_eccentricity': 0.3,
    'nucleus_to_cell_area_ratio': -1.0, # Note: original script used -1.5, using -1.0 from user's second script example
    'nuclear_enlargement_factor': 1.0, # Matched to feature name from extract_features_for_cell
    'cell_enlargement_factor': 1.5,    # Matched to feature name from extract_features_for_cell
    'nucleus_displacement_avg': 0.2,   # Matched to feature name from extract_features_for_cell
}

# Rule-based gating definition from user
RULE_BASED_GATES = [
    {   'name': 'Polynucleated',
        'conditions': [('nuclei_count', '>', 1)],
        'output_label': 'Rule_Sen_Poly' },
    {   'name': 'Very_Large_Cell',
        'conditions': [('cell_area', '>', 4000)],
        'output_label': 'Rule_Sen_VeryLarge' },
    {   'name': 'Low_Circularity',
        'conditions': [('cell_circularity', '<', 0.2)],
        'output_label': 'Rule_Sen_LowCirc' },
    {   'name': 'Low_NucToCellRatio',
        'conditions': [('nucleus_to_cell_area_ratio', '<', 0.1)],
        'output_label': 'Rule_Sen_LowNucRatio' },
    {   'name': 'High_Score_Not_Otherwise_Caught', # This rule needs 'senescence_score_normalized'
        'conditions': [('senescence_score_normalized', '>', 0.85)],
        'output_label': 'Rule_Sen_HighScore' }
]
RULE_BASED_DEFAULT_LABEL = 'Rule_NonSenescent'


# Function to extract sample ID from new filename structure
def extract_sample_id(filename):
    """
    Extract the sample ID from a filename based on the new naming pattern for Static-x20.
    """
    base_name = os.path.splitext(filename)[0]
    if base_name.startswith('denoised_'):
        base_name = base_name[len('denoised_'):]
    pattern = re.compile(r'(.+?_seq\d+)')
    match = pattern.search(base_name)
    if match:
        return match.group(1)
    parts = base_name.split('_')
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0:
            return '_'.join(parts[:i+1])
    print(f"Warning: Could not robustly extract sample ID from '{filename}'. Using fallback.")
    return '_'.join(base_name.split('_')[:7])

def find_mask_files(cell_dir, nuclei_dir):
    """Finds and pairs cell and nuclei mask files based on extracted sample ID."""
    print("\n--- Finding and Pairing Mask Files ---")
    cell_files = [f for f in os.listdir(cell_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    nuclei_files = [f for f in os.listdir(nuclei_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    print(f"Found {len(cell_files)} cell mask files and {len(nuclei_files)} nuclei mask files")

    nuclei_lookup = {}
    for nuclei_file in nuclei_files:
        sample_id = extract_sample_id(nuclei_file)
        if sample_id:
            nuclei_lookup[sample_id] = nuclei_file
        else:
            print(f"Warning: Could not extract sample ID for nuclei file: {nuclei_file}")

    file_pairs = []
    pairs_found = 0
    for cell_file in cell_files:
        sample_id = extract_sample_id(cell_file)
        if sample_id and sample_id in nuclei_lookup:
            nuclei_file = nuclei_lookup[sample_id]
            file_pairs.append({
                'cell_file': os.path.join(cell_dir, cell_file),
                'nuclei_file': os.path.join(nuclei_dir, nuclei_file),
                'sample_id': sample_id
            })
            pairs_found += 1
        elif sample_id:
            print(f"Warning: No matching nuclei file for cell sample ID: {sample_id} (from {cell_file})")
        else:
            print(f"Warning: Could not extract sample ID for cell file: {cell_file}")

    print(f"Total matching cell-nuclei file pairs found: {pairs_found}")
    if pairs_found == 0 and (len(cell_files) > 0 or len(nuclei_files) > 0) :
        print("CRITICAL WARNING: No file pairs matched. Check `extract_sample_id` and filenames.")
    return file_pairs

def load_mask_image(filepath):
    """Loads a mask image, ensuring it's labeled."""
    try:
        img = io.imread(filepath)
        if img.dtype.kind in 'iu' and np.max(img) > 1: img = img.astype(np.uint16)
        elif img.dtype == bool: img = ndimage.label(img.astype(np.uint8))[0].astype(np.uint16)
        elif img.dtype.kind == 'f': img = ndimage.label((img > 0.5).astype(np.uint8))[0].astype(np.uint16)
        elif img.dtype == np.uint8 and np.max(img) <= 1: img = ndimage.label(img)[0].astype(np.uint16)
        elif img.dtype == np.uint8 and np.max(img) > 1: img = img.astype(np.uint16)
        else:
            print(f"Warning: Unhandled img type {img.dtype} for {filepath}. Attempting binary labeling.")
            try: img = ndimage.label((img > np.median(img)).astype(np.uint8))[0].astype(np.uint16)
            except Exception as e_conv: print(f"Error converting/labeling {filepath}: {e_conv}"); return None
        if img.ndim > 2:
            print(f"Warning: Multi-channel image {filepath} ({img.shape}). Taking first channel.")
            img = img[..., 0]
            if np.max(img) <= 1: img = ndimage.label(img)[0].astype(np.uint16)
        return img
    except Exception as e: print(f"Error loading image {filepath}: {e}"); return None

def accurately_track_nuclei_in_cells(cell_mask, nuclei_mask):
    """Identifies which nuclei are inside which cells."""
    labeled_cells, num_cells = cell_mask, np.max(cell_mask)
    labeled_nuclei, num_nuclei = nuclei_mask, np.max(nuclei_mask)
    print(f"Found {num_cells} cell objects, {num_nuclei} nuclei objects.")
    if num_cells == 0 or num_nuclei == 0: return {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    cell_props = measure.regionprops(labeled_cells)
    nuclei_props = measure.regionprops(labeled_nuclei)
    results = {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    for cell_region in cell_props:
        cell_id = cell_region.label
        results['cell_data'].append({
            'cell_id': cell_id, 'area': cell_region.area, 'perimeter': cell_region.perimeter,
            'eccentricity': getattr(cell_region, 'eccentricity', 0),
            'orientation': np.degrees(getattr(cell_region, 'orientation', 0)),
            'major_axis_length': getattr(cell_region, 'major_axis_length', 0),
            'minor_axis_length': getattr(cell_region, 'minor_axis_length', 0),
            'centroid_y': cell_region.centroid[0], 'centroid_x': cell_region.centroid[1],
            'nuclei_count': 0, 'nuclei_ids_in_cell': []
        })
        results['cell_nuclei_mapping'][cell_id] = []

    for nucleus_region in nuclei_props:
        nucleus_id, nucleus_area = nucleus_region.label, nucleus_region.area
        nucleus_mask_single = (labeled_nuclei == nucleus_id)
        contained_in_cell_id, max_overlap_ratio = None, 0
        min_r, min_c, max_r, max_c = nucleus_region.bbox
        candidate_cell_labels = np.unique(labeled_cells[min_r:max_r, min_c:max_c])
        for cell_label_cand in candidate_cell_labels:
            if cell_label_cand == 0: continue
            overlap_area = np.sum(np.logical_and(labeled_cells == cell_label_cand, nucleus_mask_single))
            if nucleus_area > 0:
                overlap_ratio = overlap_area / nucleus_area
                if overlap_ratio > max_overlap_ratio:
                    max_overlap_ratio, contained_in_cell_id = overlap_ratio, cell_label_cand
        if contained_in_cell_id is not None and max_overlap_ratio > 0.5:
            results['nuclei_data'].append({
                'nucleus_id': nucleus_id, 'cell_id': contained_in_cell_id, 'area': nucleus_area,
                'eccentricity': getattr(nucleus_region, 'eccentricity', 0),
                'centroid_y': nucleus_region.centroid[0], 'centroid_x': nucleus_region.centroid[1],
                'overlap_ratio': max_overlap_ratio
            })
            for cd in results['cell_data']:
                if cd['cell_id'] == contained_in_cell_id:
                    cd['nuclei_count'] += 1; cd['nuclei_ids_in_cell'].append(nucleus_id); break
            results['cell_nuclei_mapping'][contained_in_cell_id].append(nucleus_id)
    return results

def extract_features_for_cell(cell_data_item, cell_nuclei_map, nuclei_data_list):
    """Extracts morphometric features for a cell and its nuclei."""
    features = {}
    features['cell_area'] = cell_data_item['area']
    features['cell_perimeter'] = cell_data_item['perimeter']
    features['cell_eccentricity'] = cell_data_item.get('eccentricity', 0)
    features['cell_circularity'] = (4 * np.pi * features['cell_area'] / (features['cell_perimeter']**2)) if features['cell_perimeter'] > 0 else 0
    major_axis, minor_axis = cell_data_item.get('major_axis_length'), cell_data_item.get('minor_axis_length')
    features['cell_aspect_ratio'] = (major_axis / minor_axis) if major_axis and minor_axis and minor_axis > 0 else 1.0

    original_cell_label = cell_data_item['cell_id']
    nucleus_original_labels = cell_nuclei_map.get(original_cell_label, [])
    features['nuclei_count'] = len(nucleus_original_labels)

    for k in ['avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area',
              'avg_nucleus_eccentricity', 'nucleus_area_std',
              'nucleus_displacement_avg', 'nucleus_to_cell_area_ratio']: features[k] = 0

    if features['nuclei_count'] > 0:
        cell_nuclei_details = [nd for nd in nuclei_data_list if nd['nucleus_id'] in nucleus_original_labels and nd['cell_id'] == original_cell_label]
        if cell_nuclei_details:
            areas = [n['area'] for n in cell_nuclei_details]
            eccs = [n.get('eccentricity', 0) for n in cell_nuclei_details]
            features['avg_nucleus_area'] = np.mean(areas) if areas else 0
            features['total_nuclear_area'] = sum(areas)
            features['max_nucleus_area'] = max(areas) if areas else 0
            features['avg_nucleus_eccentricity'] = np.mean(eccs) if eccs else 0
            features['nucleus_area_std'] = np.std(areas) if len(areas) > 1 else 0
            if features['cell_area'] > 0:
                features['nucleus_to_cell_area_ratio'] = features['total_nuclear_area'] / features['cell_area']
            displacements = [np.sqrt((n['centroid_x'] - cell_data_item['centroid_x'])**2 + (n['centroid_y'] - cell_data_item['centroid_y'])**2) for n in cell_nuclei_details]
            features['nucleus_displacement_avg'] = np.mean(displacements) if displacements else 0

    avg_normal_nucleus_area, avg_normal_cell_area = 500, 2000
    features['nuclear_enlargement_factor'] = features.get('avg_nucleus_area',0) / avg_normal_nucleus_area if avg_normal_nucleus_area > 0 and features.get('avg_nucleus_area',0) > 0 else 1.0
    features['cell_enlargement_factor'] = features['cell_area'] / avg_normal_cell_area if avg_normal_cell_area > 0 and features['cell_area'] > 0 else 1.0

    return features

def calculate_senescence_score(df, score_weights, area_log_features):
    """Calculates a per-cell senescence score and its normalized version."""
    print("\nCalculating per-cell senescence score...")
    features_present_for_scoring = [f for f in score_weights.keys() if f in df.columns]

    if not features_present_for_scoring:
        print("Error: No features for senescence score calculation are present. Score will be NaN.")
        df['senescence_score'] = np.nan
        df['senescence_score_normalized'] = np.nan
        return df

    score_df_subset = df[features_present_for_scoring].copy()

    for col in area_log_features:
        if col in score_df_subset.columns:
            score_df_subset[col] = np.log1p(score_df_subset[col])
            print(f"  Log-transformed for scoring: {col}")

    scaler = StandardScaler()
    numeric_cols_in_subset = score_df_subset.select_dtypes(include=np.number).columns
    if not numeric_cols_in_subset.empty:
        score_features_standardized_values = scaler.fit_transform(score_df_subset[numeric_cols_in_subset])
        score_features_standardized_df = pd.DataFrame(score_features_standardized_values,
                                                      columns=numeric_cols_in_subset,
                                                      index=score_df_subset.index)
    else:
        print("  Warning: No numeric columns found in scoring subset for standardization.")
        score_features_standardized_df = pd.DataFrame(index=score_df_subset.index)

    df['senescence_score'] = 0.0
    for feature, weight in score_weights.items():
        if feature in score_features_standardized_df.columns:
            df['senescence_score'] += score_features_standardized_df[feature] * weight
        elif feature in df.columns:
             print(f"  Warning: Scoring feature '{feature}' not in standardized set, using original. Ensure this is intended.")
        else:
            print(f"  Warning: Feature '{feature}' for scoring not found in DataFrame. Skipping.")

    min_score, max_score = df['senescence_score'].min(), df['senescence_score'].max()
    if pd.isna(min_score) or pd.isna(max_score) or min_score == max_score:
        df['senescence_score_normalized'] = 0.5
        print("  Warning: Senescence score is NaN or uniform. Normalized score set to 0.5.")
    else:
        df['senescence_score_normalized'] = (df['senescence_score'] - min_score) / (max_score - min_score)

    print("Senescence score calculation complete.")
    return df

def apply_rule_based_classification(df, rules, default_label):
    """Applies rule-based gates for classification."""
    print("\n--- Applying Rule-Based Classification ---")
    df['rule_based_classification_granular'] = default_label

    all_rule_features = set()
    for rule in rules:
        for condition_feature, _, _ in rule['conditions']:
            all_rule_features.add(condition_feature)

    missing_features_for_rules = [feat for feat in all_rule_features if feat not in df.columns]
    if missing_features_for_rules:
        print(f"  ERROR: Critical features for rule-based gating are missing: {missing_features_for_rules}. Aborting rule application.")
        df['cell_type'] = 'Unknown_Rules_Missing_Features'
        return df

    for rule in rules:
        print(f"  Applying rule: {rule['name']}")
        eligible_mask = (df['rule_based_classification_granular'] == default_label)
        if not eligible_mask.any():
            continue

        current_rule_mask = pd.Series([True] * len(df), index=df.index)
        for feature, operator, value in rule['conditions']:
            feature_series = pd.to_numeric(df[feature], errors='coerce')
            nan_mask = feature_series.isnull()

            if operator == '>': current_rule_mask &= (feature_series > value)
            elif operator == '<': current_rule_mask &= (feature_series < value)
            elif operator == '>=': current_rule_mask &= (feature_series >= value)
            elif operator == '<=': current_rule_mask &= (feature_series <= value)
            elif operator == '==': current_rule_mask &= (feature_series == value)
            elif operator == '!=': current_rule_mask &= (feature_series != value)
            else: print(f"    Unknown operator '{operator}' in rule '{rule['name']}'."); current_rule_mask[:] = False; break
            current_rule_mask[nan_mask] = False

        if not current_rule_mask.any():
            continue

        cells_to_label_now = eligible_mask & current_rule_mask
        df.loc[cells_to_label_now, 'rule_based_classification_granular'] = rule['output_label']
        print(f"    {cells_to_label_now.sum()} cells labeled as '{rule['output_label']}'.")

    df['cell_type'] = np.where(
        df['rule_based_classification_granular'] == default_label,
        'Non-senescent',
        'Senescent'
    )
    print(f"\nFinal rule-based classification counts (granular):\n{df['rule_based_classification_granular'].value_counts(dropna=False)}")
    print(f"\nFinal rule-based classification counts (binary 'cell_type'):\n{df['cell_type'].value_counts(dropna=False)}")
    return df

def generate_umap_coordinates(df, features_for_umap, area_log_features, umap_n_neighbors=15, umap_min_dist=0.1, umap_random_state=42):
    """Generates UMAP coordinates from specified features."""
    print("\n--- Generating UMAP Coordinates ---")

    actual_features_for_umap = [f for f in features_for_umap if f in df.columns]
    if not actual_features_for_umap:
        print("Error: No features specified for UMAP are present in the DataFrame.")
        df['umap_x'] = np.nan
        df['umap_y'] = np.nan
        return df

    umap_features_df = df[actual_features_for_umap].copy()

    for col in area_log_features:
        if col in umap_features_df.columns:
            umap_features_df[col] = np.log1p(umap_features_df[col])
            print(f"  Log-transformed for UMAP: {col}")

    if umap_features_df.isnull().values.any():
        print(f"  Found {umap_features_df.isnull().values.sum()} NaN values in UMAP features. Filling with column means.")
        umap_features_df = umap_features_df.fillna(umap_features_df.mean())

    if umap_features_df.isnull().values.any():
        print("  Error: NaNs still present after attempting to fill. UMAP may fail.")
        df['umap_x'] = np.nan
        df['umap_y'] = np.nan
        return df

    scaler = StandardScaler()
    scaled_umap_features = scaler.fit_transform(umap_features_df)

    actual_n_neighbors = min(umap_n_neighbors, scaled_umap_features.shape[0] - 1)
    if actual_n_neighbors < 2:
        print(f"Warning: Not enough samples ({scaled_umap_features.shape[0]}) for UMAP. Skipping UMAP.")
        df['umap_x'], df['umap_y'] = np.nan, np.nan
        return df

    reducer = umap.UMAP(n_neighbors=actual_n_neighbors, min_dist=umap_min_dist, random_state=umap_random_state, n_components=2)
    embedding = reducer.fit_transform(scaled_umap_features)
    df['umap_x'] = embedding[:, 0]
    df['umap_y'] = embedding[:, 1]
    print("UMAP coordinates generated.")
    return df

def visualize_results(df_to_visualize, output_dir_main):
    """Creates UMAP visualization based on rule-based classification."""
    print("\n--- Creating Visualizations (Rule-Based) ---")
    if df_to_visualize.empty or 'umap_x' not in df_to_visualize.columns or df_to_visualize['umap_x'].isnull().all():
        print("Skipping UMAP visualizations as UMAP data is missing or empty.")
        return

    plt.figure(figsize=(10, 8))
    if 'cell_type' in df_to_visualize.columns and df_to_visualize['cell_type'].notna().any():
        sns.scatterplot(
            x='umap_x', y='umap_y',
            hue='cell_type',
            palette={'Senescent': 'red', 'Non-senescent': 'blue', 'Unknown_Rules_Missing_Features': 'grey'},
            data=df_to_visualize,
            s=30, alpha=0.7, edgecolor='k', linewidth=0.5
        )
        plt.title('UMAP by Rule-Based Classification (Senescent/Non-senescent)', fontsize=14)
    else:
        plt.scatter(df_to_visualize['umap_x'], df_to_visualize['umap_y'], s=30, alpha=0.5)
        plt.title('UMAP Projection (Coloring unavailable)', fontsize=14)

    plt.xlabel('UMAP Dimension 1', fontsize=12)
    plt.ylabel('UMAP Dimension 2', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    if 'cell_type' in df_to_visualize.columns: plt.legend(title='Cell Type (Rule-Based)')
    plt.savefig(os.path.join(output_dir_main, 'umap_rule_based_binary.png'), dpi=300, bbox_inches='tight')
    plt.close()
    print("UMAP visualization (binary rule-based) saved.")

    if 'rule_based_classification_granular' in df_to_visualize.columns and df_to_visualize['rule_based_classification_granular'].notna().any():
        plt.figure(figsize=(12, 10))
        unique_granular_labels = sorted(df_to_visualize['rule_based_classification_granular'].unique())
        palette_granular = {}
        sen_colors = sns.color_palette("Reds_r", n_colors=len([lbl for lbl in unique_granular_labels if lbl.startswith("Rule_Sen_")]))
        sen_idx = 0
        for label in unique_granular_labels:
            if label == RULE_BASED_DEFAULT_LABEL:
                palette_granular[label] = 'blue'
            elif label.startswith("Rule_Sen_"):
                palette_granular[label] = sen_colors[sen_idx % len(sen_colors)]
                sen_idx +=1
            else:
                palette_granular[label] = 'grey'

        sns.scatterplot(
            x='umap_x', y='umap_y',
            hue='rule_based_classification_granular',
            palette=palette_granular,
            hue_order=unique_granular_labels,
            data=df_to_visualize,
            s=30, alpha=0.7, edgecolor='k', linewidth=0.5
        )
        plt.title('UMAP by Granular Rule-Based Classification', fontsize=14)
        plt.xlabel('UMAP Dimension 1', fontsize=12)
        plt.ylabel('UMAP Dimension 2', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Rule Applied', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        plt.savefig(os.path.join(output_dir_main, 'umap_rule_based_granular.png'), dpi=300)
        plt.close()
        print("UMAP visualization (granular rule-based) saved.")

def save_results(processed_df, output_directory):
    """Saves the final DataFrame and summary stats."""
    if processed_df.empty: print("No data to save."); return
    if not os.path.exists(output_directory): os.makedirs(output_directory)

    processed_df.to_csv(os.path.join(output_directory, 'cell_classification_rule_based_full.csv'), index=False)
    print(f"Full rule-based results saved to: {os.path.join(output_directory, 'cell_classification_rule_based_full.csv')}")

    if 'sample_id' in processed_df.columns and 'cell_type' in processed_df.columns:
        summary = processed_df.groupby('sample_id')['cell_type'].value_counts(normalize=False).unstack(fill_value=0)
        for col in ['Senescent', 'Non-senescent', 'Unknown_Rules_Missing_Features']:
            if col not in summary.columns: summary[col] = 0
        summary['total_cells'] = summary.sum(axis=1)
        if (summary['total_cells'] > 0).any():
            summary['percent_senescent'] = summary['Senescent'] / summary['total_cells'] * 100
        else: summary['percent_senescent'] = 0
        summary.to_csv(os.path.join(output_directory, 'sample_summary_rule_based.csv'))
        print(f"Rule-based sample summary saved to: {os.path.join(output_directory, 'sample_summary_rule_based.csv')}")

        if 'percent_senescent' in summary.columns:
            plt.figure(figsize=(12,6)); sns.barplot(x=summary.index, y='percent_senescent', data=summary.reset_index(), color='red')
            plt.title('Senescent Cells by Sample (Rule-Based)'); plt.xlabel('Sample ID'); plt.ylabel('Senescent (%)')
            plt.xticks(rotation=90, ha='right'); plt.tight_layout()
            plt.savefig(os.path.join(output_directory, 'senescent_percentage_rule_based.png'), dpi=300); plt.close()
            print("Rule-based senescent percentage plot saved.")

def main(base_dir, cell_subdir, nuclei_subdir, output_parent_dir):
    """Main analysis workflow."""
    print(f"=== Senescence Analysis (Rule-Based) for {base_dir} ===")
    cell_mask_dir = os.path.join(base_dir, cell_subdir)
    nuclei_dir = os.path.join(base_dir, nuclei_subdir)
    dataset_name = os.path.basename(base_dir)
    # MODIFIED LINE: Changed "Senescence_RuleBased_Results" to "Senescence_Results"
    output_dir_main = os.path.join(output_parent_dir, dataset_name, "Senescence_Results")
    if not os.path.exists(output_dir_main): os.makedirs(output_dir_main); print(f"Created: {output_dir_main}")

    file_pairs = find_mask_files(cell_mask_dir, nuclei_dir)
    if not file_pairs: print("No matching file pairs. Exiting."); return

    all_cell_features_list = []
    for file_pair in tqdm(file_pairs, desc="Processing image pairs"):
        print(f"\nProcessing sample: {file_pair['sample_id']}")
        try:
            cell_mask = load_mask_image(file_pair['cell_file'])
            nuclei_mask = load_mask_image(file_pair['nuclei_file'])
            if cell_mask is None or nuclei_mask is None: print(f"Skipping {file_pair['sample_id']}."); continue

            tracking_results = accurately_track_nuclei_in_cells(cell_mask, nuclei_mask)
            for sc_data in tracking_results['cell_data']:
                features = extract_features_for_cell(sc_data, tracking_results['cell_nuclei_mapping'], tracking_results['nuclei_data'])
                features['cell_id_unique'] = f"{file_pair['sample_id']}_{sc_data['cell_id']}"
                features['sample_id'] = file_pair['sample_id']
                features['original_cell_label'] = sc_data['cell_id']
                all_cell_features_list.append(features)
        except Exception as e: print(f"Error processing {file_pair['sample_id']}: {e}"); import traceback; traceback.print_exc()

    if not all_cell_features_list: print("No cell features extracted."); return
    all_features_df = pd.DataFrame(all_cell_features_list)
    print(f"\nTotal cells extracted: {len(all_features_df)}")

    all_features_df = calculate_senescence_score(all_features_df, SENESCENCE_SCORE_WEIGHTS, AREA_FEATURES_TO_LOG)
    all_features_df = generate_umap_coordinates(all_features_df, FEATURES_FOR_UMAP, AREA_FEATURES_TO_LOG)
    classified_df = apply_rule_based_classification(all_features_df, RULE_BASED_GATES, RULE_BASED_DEFAULT_LABEL)

    save_results(classified_df, output_dir_main)
    visualize_results(classified_df, output_dir_main)

    print("\nRule-Based Analysis complete!")

if __name__ == "__main__":
    segmented_data_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented"
    current_dataset_folder_name = "1.4Pa-x20"
    base_input_dir = os.path.join(segmented_data_parent_dir, current_dataset_folder_name)
    cell_masks_subdir = "Cell_merged_conservative"
    nuclei_masks_subdir = "Nuclei"
    analysis_output_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis"

    if not os.path.isdir(base_input_dir):
        print(f"Error: Base input directory not found: {base_input_dir}")
    else:
        main(base_input_dir, cell_masks_subdir, nuclei_masks_subdir, analysis_output_parent_dir)


=== Senescence Analysis (Rule-Based) for /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20 ===

--- Finding and Pairing Mask Files ---
Found 23 cell mask files and 23 nuclei mask files
Total matching cell-nuclei file pairs found: 23


Processing image pairs:   0%|          | 0/23 [00:00<?, ?it/s]


Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001
Found 326 cell objects, 347 nuclei objects.


Processing image pairs:   4%|▍         | 1/23 [00:01<00:26,  1.18s/it]


Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002
Found 273 cell objects, 290 nuclei objects.


...


Processing sample: 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017
Found 261 cell objects, 277 nuclei objects.


Processing image pairs:  96%|█████████▌| 22/23 [00:26<00:01,  1.21s/it]


Processing sample: 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018
Found 301 cell objects, 320 nuclei objects.


Processing image pairs: 100%|██████████| 23/23 [00:27<00:00,  1.18s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(



Total cells extracted: 6663

Calculating per-cell senescence score...
  Log-transformed for scoring: cell_area
  Log-transformed for scoring: avg_nucleus_area
  Log-transformed for scoring: cell_perimeter
Senescence score calculation complete.

--- Generating UMAP Coordinates ---
  Log-transformed for UMAP: cell_area
  Log-transformed for UMAP: avg_nucleus_area
  Log-transformed for UMAP: max_nucleus_area
  Log-transformed for UMAP: cell_perimeter
UMAP coordinates generated.

--- Applying Rule-Based Classification ---
  Applying rule: Polynucleated
    417 cells labeled as 'Rule_Sen_Poly'.
  Applying rule: Very_Large_Cell
    1651 cells labeled as 'Rule_Sen_VeryLarge'.
  Applying rule: Low_Circularity
    66 cells labeled as 'Rule_Sen_LowCirc'.
  Applying rule: Low_NucToCellRatio
    67 cells labeled as 'Rule_Sen_LowNucRatio'.
  Applying rule: High_Score_Not_Otherwise_Caught
    0 cells labeled as 'Rule_Sen_HighScore'.

Final rule-based classification counts (granular):
rule_based_cla

In [13]:
import os
import numpy as np
import pandas as pd
import cv2
import re
from scipy import ndimage
from skimage import io, measure, segmentation # Added segmentation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # Added for legends in mask visualization
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# KMeans is no longer used for final classification but can be kept for exploratory UMAP clustering
from sklearn.cluster import KMeans
import umap
from tqdm import tqdm

# --- Configuration & Parameters ---
# Features to be used for UMAP generation
FEATURES_FOR_UMAP = [
    'cell_area', 'cell_perimeter', 'cell_eccentricity', 'cell_circularity',
    'cell_aspect_ratio', 'avg_nucleus_area', 'max_nucleus_area',
    'avg_nucleus_eccentricity', 'nucleus_area_std', 'nucleus_displacement_avg',
    'nucleus_to_cell_area_ratio',
    'nuclear_enlargement_factor', 'cell_enlargement_factor'
]

# Features to log-transform before UMAP and Senescence Score calculation
AREA_FEATURES_TO_LOG = ['cell_area', 'avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'cell_perimeter']

# Weights for calculating the senescence score
SENESCENCE_SCORE_WEIGHTS = {
    'cell_area': 1.5,
    'cell_perimeter': 0.5,
    'cell_eccentricity': 0.5,
    'cell_circularity': -1.0,
    'cell_aspect_ratio': 0.5,
    'avg_nucleus_area': 1.0,
    'avg_nucleus_eccentricity': 0.3,
    'nucleus_to_cell_area_ratio': -1.0,
    'nuclear_enlargement_factor': 1.0,
    'cell_enlargement_factor': 1.5,
    'nucleus_displacement_avg': 0.2,
}

# Rule-based gating definition from user
RULE_BASED_GATES = [
    {   'name': 'Polynucleated',
        'conditions': [('nuclei_count', '>', 1)],
        'output_label': 'Rule_Sen_Poly' },
    {   'name': 'Very_Large_Cell',
        'conditions': [('cell_area', '>', 4000)],
        'output_label': 'Rule_Sen_VeryLarge' },
    {   'name': 'Low_Circularity',
        'conditions': [('cell_circularity', '<', 0.2)],
        'output_label': 'Rule_Sen_LowCirc' },
    {   'name': 'Low_NucToCellRatio',
        'conditions': [('nucleus_to_cell_area_ratio', '<', 0.1)],
        'output_label': 'Rule_Sen_LowNucRatio' },
    {   'name': 'High_Score_Not_Otherwise_Caught',
        'conditions': [('senescence_score_normalized', '>', 0.85)],
        'output_label': 'Rule_Sen_HighScore' }
]
RULE_BASED_DEFAULT_LABEL = 'Rule_NonSenescent'

# Specific label for polynucleated senescent cells (must match an output_label from RULE_BASED_GATES)
POLYNUCLEATED_SENESCENT_RULE_LABEL = 'Rule_Sen_Poly'

# --- Color Definitions for Mask Visualization ---
COLOR_NON_SENESCENT = [0, 0, 255]  # Blue
COLOR_SENESCENT_POLYNUCLEATED = [255, 165, 0]  # Orange
COLOR_SENESCENT_OTHER_RULES = [255, 0, 0]  # Red
COLOR_CELL_BOUNDARY = [255, 255, 255]  # White
COLOR_NUCLEI_OUTLINE = [255, 255, 0]  # Yellow
COLOR_UNKNOWN = [128, 128, 128] # Grey

# Subdirectory for mask visualizations
MASK_VISUALIZATION_SUBDIR = "mask_overlays_enhanced"


# Function to extract sample ID
def extract_sample_id(filename):
    base_name = os.path.splitext(filename)[0]
    if base_name.startswith('denoised_'):
        base_name = base_name[len('denoised_'):]
    pattern = re.compile(r'(.+?_seq\d+)')
    match = pattern.search(base_name)
    if match: return match.group(1)
    parts = base_name.split('_')
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0: return '_'.join(parts[:i+1])
    print(f"Warning: Fallback ID extraction for '{filename}'.")
    return '_'.join(base_name.split('_')[:7])

def find_mask_files(cell_dir, nuclei_dir):
    print("\n--- Finding and Pairing Mask Files ---")
    cell_files = [f for f in os.listdir(cell_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    nuclei_files = [f for f in os.listdir(nuclei_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    print(f"Found {len(cell_files)} cell files, {len(nuclei_files)} nuclei files")
    nuclei_lookup = {extract_sample_id(nf): nf for nf in nuclei_files if extract_sample_id(nf)}
    file_pairs = []
    for cf in cell_files:
        sid = extract_sample_id(cf)
        if sid and sid in nuclei_lookup:
            file_pairs.append({'cell_file': os.path.join(cell_dir, cf),
                               'nuclei_file': os.path.join(nuclei_dir, nuclei_lookup[sid]),
                               'sample_id': sid})
        elif sid: print(f"Warning: No matching nuclei file for cell ID: {sid} (from {cf})")
        else: print(f"Warning: Could not extract ID for cell file: {cf}")
    print(f"Total matching pairs: {len(file_pairs)}")
    if not file_pairs and (cell_files or nuclei_files): print("CRITICAL: No pairs matched.")
    return file_pairs

def load_mask_image(filepath):
    try:
        img = io.imread(filepath)
        if img.ndim > 2:
            img = img[..., 0]
        if img.dtype.kind in 'iu' and np.max(img) > 1:
            return img.astype(np.uint16)
        if img.dtype == bool or (img.dtype == np.uint8 and np.max(img) <= 1) or img.dtype.kind == 'f':
            binary_img = (img > (0.5 if img.dtype.kind == 'f' else 0)).astype(np.uint8)
            labeled_img, _ = ndimage.label(binary_img)
            return labeled_img.astype(np.uint16)
        if img.dtype == np.uint8 and np.max(img) > 1:
            return img.astype(np.uint16)
        print(f"Warning: Unhandled image type {img.dtype} for {filepath}. Attempting basic labeling.")
        labeled_img, _ = ndimage.label((img > np.median(img)).astype(np.uint8))
        return labeled_img.astype(np.uint16)
    except Exception as e: print(f"Error loading image {filepath}: {e}"); return None

def accurately_track_nuclei_in_cells(cell_mask, nuclei_mask):
    labeled_cells, num_cells = cell_mask, np.max(cell_mask)
    labeled_nuclei, num_nuclei = nuclei_mask, np.max(nuclei_mask)
    print(f"Found {num_cells} cells, {num_nuclei} nuclei.")
    if num_cells == 0 or num_nuclei == 0: return {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    cell_props = measure.regionprops(labeled_cells)
    nuclei_props = measure.regionprops(labeled_nuclei)
    results = {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    for cp in cell_props:
        results['cell_data'].append({
            'cell_id': cp.label, 'area': cp.area, 'perimeter': cp.perimeter,
            'eccentricity': getattr(cp, 'eccentricity', 0), 'orientation': np.degrees(getattr(cp, 'orientation', 0)),
            'major_axis_length': getattr(cp, 'major_axis_length', 0), 'minor_axis_length': getattr(cp, 'minor_axis_length', 0),
            'centroid_y': cp.centroid[0], 'centroid_x': cp.centroid[1], 'nuclei_count': 0, 'nuclei_ids_in_cell': []
        })
        results['cell_nuclei_mapping'][cp.label] = []

    for np_region in nuclei_props:
        nuc_id, nuc_area = np_region.label, np_region.area
        nuc_mask_single = (labeled_nuclei == nuc_id)
        best_cell_id, max_overlap = None, 0
        min_r, min_c, max_r, max_c = np_region.bbox
        for cell_cand_lbl in np.unique(labeled_cells[min_r:max_r, min_c:max_c]):
            if cell_cand_lbl == 0: continue
            overlap = np.sum(np.logical_and(labeled_cells == cell_cand_lbl, nuc_mask_single))
            if nuc_area > 0 and (overlap / nuc_area) > max_overlap:
                max_overlap, best_cell_id = (overlap / nuc_area), cell_cand_lbl
        if best_cell_id and max_overlap > 0.5:
            results['nuclei_data'].append({
                'nucleus_id': nuc_id, 'cell_id': best_cell_id, 'area': nuc_area,
                'eccentricity': getattr(np_region, 'eccentricity', 0),
                'centroid_y': np_region.centroid[0], 'centroid_x': np_region.centroid[1], 'overlap_ratio': max_overlap
            })
            for cd_item in results['cell_data']:
                if cd_item['cell_id'] == best_cell_id: cd_item['nuclei_count']+=1; cd_item['nuclei_ids_in_cell'].append(nuc_id); break
            results['cell_nuclei_mapping'][best_cell_id].append(nuc_id)
    return results

def extract_features_for_cell(cd_item, cn_map, nuc_data_list):
    feats = {'cell_area': cd_item['area'], 'cell_perimeter': cd_item['perimeter'],
             'cell_eccentricity': cd_item.get('eccentricity',0)}
    feats['cell_circularity'] = (4*np.pi*feats['cell_area']/(feats['cell_perimeter']**2)) if feats['cell_perimeter']>0 else 0
    maj, minor = cd_item.get('major_axis_length'), cd_item.get('minor_axis_length')
    feats['cell_aspect_ratio'] = (maj/minor) if maj and minor and minor>0 else 1.0

    orig_cell_lbl = cd_item['cell_id']
    nuc_orig_lbls = cn_map.get(orig_cell_lbl, [])
    feats['nuclei_count'] = len(nuc_orig_lbls)

    for k_nuc in ['avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'avg_nucleus_eccentricity',
                  'nucleus_area_std', 'nucleus_displacement_avg', 'nucleus_to_cell_area_ratio']: feats[k_nuc]=0
    if feats['nuclei_count'] > 0:
        nuc_details_incell = [nd for nd in nuc_data_list if nd['nucleus_id'] in nuc_orig_lbls and nd['cell_id']==orig_cell_lbl]
        if nuc_details_incell:
            areas = [n['area'] for n in nuc_details_incell]; eccs = [n.get('eccentricity',0) for n in nuc_details_incell]
            feats.update({
                'avg_nucleus_area': np.mean(areas) if areas else 0, 'total_nuclear_area': sum(areas),
                'max_nucleus_area': max(areas) if areas else 0, 'avg_nucleus_eccentricity': np.mean(eccs) if eccs else 0,
                'nucleus_area_std': np.std(areas) if len(areas)>1 else 0,
                'nucleus_to_cell_area_ratio': (sum(areas)/feats['cell_area']) if feats['cell_area']>0 else 0
            })
            disps = [np.sqrt((n['centroid_x']-cd_item['centroid_x'])**2 + (n['centroid_y']-cd_item['centroid_y'])**2) for n in nuc_details_incell]
            feats['nucleus_displacement_avg'] = np.mean(disps) if disps else 0

    norm_nuc_A, norm_cell_A = 500, 2000
    feats['nuclear_enlargement_factor'] = feats.get('avg_nucleus_area',0)/norm_nuc_A if norm_nuc_A>0 and feats.get('avg_nucleus_area',0)>0 else 1.0
    feats['cell_enlargement_factor'] = feats['cell_area']/norm_cell_A if norm_cell_A>0 and feats['cell_area']>0 else 1.0
    return feats

def calculate_senescence_score(df, weights, log_features):
    print("\nCalculating senescence score...")
    present_feats = [f for f in weights.keys() if f in df.columns]
    if not present_feats: df['senescence_score'], df['senescence_score_normalized'] = np.nan, np.nan; return df

    score_subset = df[present_feats].copy()
    for col in log_features:
        if col in score_subset.columns: score_subset[col] = np.log1p(score_subset[col])

    scaler = StandardScaler()
    num_cols = score_subset.select_dtypes(include=np.number).columns
    std_df = pd.DataFrame(index=score_subset.index)
    if not num_cols.empty:
        std_vals = scaler.fit_transform(score_subset[num_cols])
        std_df = pd.DataFrame(std_vals, columns=num_cols, index=score_subset.index)

    df['senescence_score'] = sum(std_df[f] * w for f, w in weights.items() if f in std_df.columns)
    min_s, max_s = df['senescence_score'].min(), df['senescence_score'].max()
    df['senescence_score_normalized'] = 0.5 if pd.isna(min_s) or min_s==max_s else (df['senescence_score']-min_s)/(max_s-min_s)
    print("Senescence score calculation complete.")
    return df

def apply_rule_based_classification(df, rules, default_label):
    print("\n--- Applying Rule-Based Classification ---")
    df['rule_based_classification_granular'] = default_label
    all_rule_feats = set(cond[0] for r in rules for cond in r['conditions'])
    if any(feat not in df.columns for feat in all_rule_feats):
        print(f"ERROR: Missing rule features: {[f for f in all_rule_feats if f not in df.columns]}. Aborting."); df['cell_type'] = 'Unknown_Rules_Missing'; return df

    for rule in rules:
        print(f"  Applying rule: {rule['name']}")
        eligible = (df['rule_based_classification_granular'] == default_label)
        if not eligible.any(): continue

        current_mask = pd.Series([True]*len(df), index=df.index)
        for feat, op, val in rule['conditions']:
            feat_series = pd.to_numeric(df[feat], errors='coerce')
            nan_m = feat_series.isnull()
            if op=='>': current_mask &= (feat_series > val)
            elif op=='<': current_mask &= (feat_series < val)
            elif op=='>=': current_mask &= (feat_series >=value) # Added >=
            elif op=='<=': current_mask &= (feat_series <=value) # Added <=
            elif op=='==': current_mask &= (feat_series ==value) # Added ==
            elif op=='!=': current_mask &= (feat_series !=value) # Added !=
            else: print(f"Unknown op: {op}"); current_mask[:]=False; break
            current_mask[nan_m] = False

        if current_mask.any():
            to_label = eligible & current_mask
            df.loc[to_label, 'rule_based_classification_granular'] = rule['output_label']
            print(f"    {to_label.sum()} cells labeled as '{rule['output_label']}'.")

    df['cell_type'] = np.where(df['rule_based_classification_granular']==default_label, 'Non-senescent', 'Senescent')
    print(f"\nGranular counts:\n{df['rule_based_classification_granular'].value_counts(dropna=False)}")
    print(f"\nBinary 'cell_type' counts:\n{df['cell_type'].value_counts(dropna=False)}")
    return df

def generate_umap_coordinates(df, umap_feats, log_feats, n_neigh=15, min_d=0.1, rand_state=42):
    print("\n--- Generating UMAP Coordinates ---")
    actual_umap_feats = [f for f in umap_feats if f in df.columns]
    if not actual_umap_feats: df['umap_x'], df['umap_y'] = np.nan, np.nan; return df

    umap_df = df[actual_umap_feats].copy()
    for col in log_feats:
        if col in umap_df.columns: umap_df[col] = np.log1p(umap_df[col])
    if umap_df.isnull().values.any(): umap_df = umap_df.fillna(umap_df.mean())
    if umap_df.isnull().values.any(): df['umap_x'],df['umap_y']=np.nan,np.nan; return df

    scaled_feats = StandardScaler().fit_transform(umap_df)
    actual_n_neigh = min(n_neigh, scaled_feats.shape[0]-1)
    if actual_n_neigh < 2: df['umap_x'],df['umap_y']=np.nan,np.nan; return df

    embedding = umap.UMAP(n_neighbors=actual_n_neigh, min_dist=min_d, random_state=rand_state).fit_transform(scaled_feats)
    df['umap_x'], df['umap_y'] = embedding[:,0], embedding[:,1]
    print("UMAP coordinates generated.")
    return df

def visualize_results(df_viz, out_dir):
    print("\n--- Creating Visualizations (Rule-Based) ---")
    if df_viz.empty or 'umap_x' not in df_viz or df_viz['umap_x'].isnull().all(): print("Skipping UMAP plots."); return

    plt.figure(figsize=(10,8));
    if 'cell_type' in df_viz and df_viz['cell_type'].notna().any():
        sns.scatterplot(x='umap_x', y='umap_y', hue='cell_type',
                        palette={'Senescent':'red', 'Non-senescent':'blue', 'Unknown_Rules_Missing':'grey'},
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.legend(title='Cell Type (Rule-Based)')
    else: plt.scatter(df_viz['umap_x'], df_viz['umap_y'], s=30, alpha=0.5)
    plt.title('UMAP by Rule-Based Classification (Binary)'); plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(out_dir, 'umap_rule_based_binary.png'), dpi=300, bbox_inches='tight'); plt.close()
    print("Binary UMAP saved.")

    if 'rule_based_classification_granular' in df_viz and df_viz['rule_based_classification_granular'].notna().any():
        plt.figure(figsize=(12,10));
        labels = sorted(df_viz['rule_based_classification_granular'].unique())
        pal = {lbl:('blue' if lbl==RULE_BASED_DEFAULT_LABEL else ('grey' if not lbl.startswith("Rule_Sen_") else None)) for lbl in labels}
        sen_lbls = [lbl for lbl in labels if lbl.startswith("Rule_Sen_")]
        # Ensure enough colors for senescent categories, even if few are present
        num_sen_categories = len(RULE_BASED_GATES) -1 # Max possible senescent categories excluding default
        sen_cols = sns.color_palette("coolwarm_r", n_colors=max(1,num_sen_categories))

        # Map specific senescent rule labels to colors
        # This ensures consistent coloring if not all senescent rules are met in the data
        sen_rule_labels_ordered = [r['output_label'] for r in RULE_BASED_GATES if r['output_label'] != RULE_BASED_DEFAULT_LABEL]

        for i, sl_config in enumerate(sen_rule_labels_ordered):
            if sl_config in pal: # Check if this rule label is actually present in the data
                 pal[sl_config] = sen_cols[i % len(sen_cols)]


        sns.scatterplot(x='umap_x', y='umap_y', hue='rule_based_classification_granular', palette=pal, hue_order=labels,
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.title('UMAP by Granular Rule Classification'); plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.grid(True, alpha=0.3)
        plt.legend(title='Rule Applied', bbox_to_anchor=(1.05,1), loc='upper left'); plt.tight_layout(rect=[0,0,0.85,1])
        plt.savefig(os.path.join(out_dir, 'umap_rule_based_granular.png'), dpi=300); plt.close()
        print("Granular UMAP saved.")

def save_results(df_proc, out_dir):
    if df_proc.empty: print("No data to save."); return
    if not os.path.exists(out_dir): os.makedirs(out_dir)
    df_proc.to_csv(os.path.join(out_dir, 'cell_classification_rule_based_full.csv'), index=False)
    print(f"Full rule-based results saved.")
    if 'sample_id' in df_proc and 'cell_type' in df_proc:
        summary = df_proc.groupby('sample_id')['cell_type'].value_counts(normalize=False).unstack(fill_value=0)
        for col in ['Senescent','Non-senescent','Unknown_Rules_Missing']:
            if col not in summary: summary[col]=0
        summary['total_cells'] = summary.sum(axis=1)
        summary['percent_senescent'] = (summary['Senescent']/summary['total_cells']*100) if (summary['total_cells']>0).any() else 0
        summary.to_csv(os.path.join(out_dir, 'sample_summary_rule_based.csv'))
        print(f"Rule-based sample summary saved.")
        if 'percent_senescent' in summary:
            plt.figure(figsize=(12,6)); sns.barplot(x=summary.index, y='percent_senescent', data=summary.reset_index(), color='red')
            plt.title('Senescent Cells by Sample (Rules)'); plt.xticks(rotation=90, ha='right'); plt.tight_layout()
            plt.savefig(os.path.join(out_dir, 'senescent_percentage_rule_based.png'), dpi=300); plt.close()
            print("Senescent percentage plot saved.")

def visualize_enhanced_classification_on_masks(df_results, cell_mask_dir_path, nuclei_mask_dir_path, output_dir_masks_path):
    """Visualizes cell classifications on original mask images, highlighting specific rules."""
    print(f"\nGenerating enhanced classification overlays in: {output_dir_masks_path}")
    if not os.path.exists(output_dir_masks_path):
        os.makedirs(output_dir_masks_path)

    # Use 'sample_id' for iterating samples, 'cell_id_unique' for lookup
    required_cols = ['sample_id', 'cell_id_unique', 'rule_based_classification_granular']
    if not all(col in df_results.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_results.columns]
        print(f"Error: DataFrame missing required columns for mask viz: {missing}.")
        return

    classification_lookup = pd.Series(
        df_results.rule_based_classification_granular.values,
        index=df_results.cell_id_unique
    ).to_dict()

    unique_samples = df_results['sample_id'].unique()

    available_cell_masks = {extract_sample_id(f): f for f in os.listdir(cell_mask_dir_path) if f.endswith(('.tif', '.tiff'))}
    available_nuclei_masks = {extract_sample_id(f): f for f in os.listdir(nuclei_mask_dir_path) if f.endswith(('.tif', '.tiff'))}

    for sample_id_val in tqdm(unique_samples, desc="Generating Enhanced Mask Overlays"):
        cell_mask_fname = available_cell_masks.get(sample_id_val)
        nuclei_mask_fname = available_nuclei_masks.get(sample_id_val)

        if not cell_mask_fname:
            print(f"  Warning: Cell mask file not found for sample ID: {sample_id_val}")
            continue

        print(f"\n  Processing sample for enhanced overlay: {sample_id_val}")
        labeled_cell_mask_img = load_mask_image(os.path.join(cell_mask_dir_path, cell_mask_fname))
        if labeled_cell_mask_img is None: continue

        overlay_img = np.zeros((labeled_cell_mask_img.shape[0], labeled_cell_mask_img.shape[1], 3), dtype=np.uint8)

        for cell_region_props in measure.regionprops(labeled_cell_mask_img):
            original_label_from_mask = cell_region_props.label
            lookup_id = f"{sample_id_val}_{original_label_from_mask}"
            granular_status_label = classification_lookup.get(lookup_id, 'Unknown_In_CSV')

            fill_color = COLOR_UNKNOWN
            if granular_status_label == POLYNUCLEATED_SENESCENT_RULE_LABEL:
                fill_color = COLOR_SENESCENT_POLYNUCLEATED
            elif granular_status_label.startswith('Rule_Sen_'):
                fill_color = COLOR_SENESCENT_OTHER_RULES
            elif granular_status_label == RULE_BASED_DEFAULT_LABEL:
                fill_color = COLOR_NON_SENESCENT

            overlay_img[labeled_cell_mask_img == original_label_from_mask] = fill_color

        cell_boundaries_img = segmentation.find_boundaries(labeled_cell_mask_img, mode='outer', background=0)
        overlay_img[cell_boundaries_img] = COLOR_CELL_BOUNDARY

        labeled_nuc_mask_img = None
        if nuclei_mask_fname:
            labeled_nuc_mask_img = load_mask_image(os.path.join(nuclei_mask_dir_path, nuclei_mask_fname))
            if labeled_nuc_mask_img is not None:
                nuc_boundaries_img = segmentation.find_boundaries(labeled_nuc_mask_img, mode='inner', background=0)
                overlay_img[nuc_boundaries_img] = COLOR_NUCLEI_OUTLINE

        fig_plot, ax_plot = None, None
        try:
            h_img, w_img = overlay_img.shape[:2]
            fig_w_calc = max(10, w_img / 100 if w_img > 0 else 10)
            fig_h_calc = max(8, h_img / 100 if h_img > 0 else 8) * (fig_w_calc / (w_img/100 if w_img > 0 else 1))

            fig_plot, ax_plot = plt.subplots(figsize=(fig_w_calc, fig_h_calc), dpi=100)
            ax_plot.imshow(overlay_img)

            legend_handles = [
                mpatches.Patch(color=np.array(COLOR_NON_SENESCENT)/255., label=f"{RULE_BASED_DEFAULT_LABEL} (Non-Senescent)"),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_OTHER_RULES)/255., label='Senescent (Other Rules)'),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_POLYNUCLEATED)/255., label=f'Senescent ({POLYNUCLEATED_SENESCENT_RULE_LABEL})')
            ]
            # Check if any cells in the current image were actually 'Unknown_In_CSV'
            is_unknown_present = any(
                classification_lookup.get(f"{sample_id_val}_{p.label}", 'Unknown_In_CSV') == 'Unknown_In_CSV'
                for p in measure.regionprops(labeled_cell_mask_img)
            )
            if is_unknown_present:
                 legend_handles.append(mpatches.Patch(color=np.array(COLOR_UNKNOWN)/255., label='Unknown/Not in CSV'))

            if nuclei_mask_fname and labeled_nuc_mask_img is not None:
                legend_handles.append(mpatches.Patch(color=np.array(COLOR_NUCLEI_OUTLINE)/255., label='Nuclei Outline'))
            legend_handles.append(mpatches.Patch(edgecolor=np.array(COLOR_CELL_BOUNDARY)/255., facecolor='none', label='Cell Boundary', linewidth=1))

            ax_plot.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1.02, 0.5), fontsize='small', title="Legend")
            ax_plot.axis('off')
            plt.tight_layout(rect=[0, 0, 0.83, 1])

            output_img_filename = os.path.join(output_dir_masks_path, f"{sample_id_val}_enhanced_mask_overlay.png")
            plt.savefig(output_img_filename, dpi=200)
            print(f"    Saved enhanced overlay for {sample_id_val} to {output_img_filename}")
        except Exception as e_plt:
            print(f"    Error during plotting/saving mask overlay for {sample_id_val}: {e_plt}")
        finally:
            if fig_plot: plt.close(fig_plot)

    print("\nEnhanced mask overlay visualization complete.")


def main(base_dir, cell_subdir, nuclei_subdir, output_parent_dir):
    print(f"=== Senescence Analysis (Rule-Based) for {base_dir} ===")
    cell_mask_dir_path = os.path.join(base_dir, cell_subdir)
    nuclei_mask_dir_path = os.path.join(base_dir, nuclei_subdir)
    dataset_folder_name = os.path.basename(base_dir)
    main_output_dir = os.path.join(output_parent_dir, dataset_folder_name, "Senescence_Results")
    if not os.path.exists(main_output_dir): os.makedirs(main_output_dir); print(f"Created: {main_output_dir}")

    file_pairs_list = find_mask_files(cell_mask_dir_path, nuclei_mask_dir_path)
    if not file_pairs_list: print("No file pairs. Exiting."); return

    features_list = []
    for pair_info in tqdm(file_pairs_list, desc="Processing image pairs"):
        print(f"\nProcessing sample: {pair_info['sample_id']}")
        try:
            cm_img = load_mask_image(pair_info['cell_file'])
            nm_img = load_mask_image(pair_info['nuclei_file'])
            if cm_img is None or nm_img is None: print(f"Skipping {pair_info['sample_id']}."); continue

            track_res = accurately_track_nuclei_in_cells(cm_img, nm_img)
            for cell_d in track_res['cell_data']:
                cell_feats = extract_features_for_cell(cell_d, track_res['cell_nuclei_mapping'], track_res['nuclei_data'])
                cell_feats['cell_id_unique'] = f"{pair_info['sample_id']}_{cell_d['cell_id']}"
                cell_feats['sample_id'] = pair_info['sample_id']
                cell_feats['original_cell_label'] = cell_d['cell_id']
                features_list.append(cell_feats)
        except Exception as e_proc: print(f"Error processing {pair_info['sample_id']}: {e_proc}"); import traceback; traceback.print_exc()

    if not features_list: print("No features extracted."); return
    master_df = pd.DataFrame(features_list)
    print(f"\nTotal cells extracted: {len(master_df)}")

    master_df = calculate_senescence_score(master_df, SENESCENCE_SCORE_WEIGHTS, AREA_FEATURES_TO_LOG)
    master_df = generate_umap_coordinates(master_df, FEATURES_FOR_UMAP, AREA_FEATURES_TO_LOG)
    final_classified_df = apply_rule_based_classification(master_df, RULE_BASED_GATES, RULE_BASED_DEFAULT_LABEL)

    save_results(final_classified_df, main_output_dir)
    visualize_results(final_classified_df, main_output_dir)

    enhanced_masks_output_path = os.path.join(main_output_dir, MASK_VISUALIZATION_SUBDIR)
    visualize_enhanced_classification_on_masks(final_classified_df, cell_mask_dir_path, nuclei_mask_dir_path, enhanced_masks_output_path)

    print("\nRule-Based Analysis and Mask Visualization complete!")

if __name__ == "__main__":
    segmented_data_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented"
    current_dataset_folder_name = "1.4Pa-x20"
    base_input_dir = os.path.join(segmented_data_parent_dir, current_dataset_folder_name)
    cell_masks_subdir = "Cell_merged_conservative"
    nuclei_masks_subdir = "Nuclei"
    analysis_output_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis"

    if not os.path.isdir(base_input_dir):
        print(f"Error: Base input directory not found: {base_input_dir}")
    else:
        main(base_input_dir, cell_masks_subdir, nuclei_masks_subdir, analysis_output_parent_dir)


=== Senescence Analysis (Rule-Based) for /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20 ===

--- Finding and Pairing Mask Files ---
Found 23 cell files, 23 nuclei files
Total matching pairs: 23


Processing image pairs:   0%|          | 0/23 [00:00<?, ?it/s]


Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001
Found 326 cells, 347 nuclei.


Processing image pairs:   4%|▍         | 1/23 [00:01<00:26,  1.21s/it]


Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002
Found 273 cells, 290 nuclei.


...

    Saved enhanced overlay for 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016 to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/mask_overlays_enhanced/1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016_enhanced_mask_overlay.png

  Processing sample for enhanced overlay: 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017


Generating Enhanced Mask Overlays:  96%|█████████▌| 22/23 [00:36<00:01,  1.55s/it]

    Saved enhanced overlay for 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017 to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/mask_overlays_enhanced/1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017_enhanced_mask_overlay.png

  Processing sample for enhanced overlay: 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018


Generating Enhanced Mask Overlays: 100%|██████████| 23/23 [00:37<00:00,  1.63s/it]

    Saved enhanced overlay for 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018 to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/mask_overlays_enhanced/1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018_enhanced_mask_overlay.png

Enhanced mask overlay visualization complete.

Rule-Based Analysis and Mask Visualization complete!


In [14]:
import os
import numpy as np
import pandas as pd
import cv2 # OpenCV is imported but not explicitly used in the snippet; kept for consistency
import re
from scipy import ndimage
from skimage import io, measure, segmentation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans # Kept for potential exploratory use
import umap
from tqdm import tqdm

# --- Configuration & Parameters ---
# Features to be used for UMAP generation
FEATURES_FOR_UMAP = [
    'cell_area', 'cell_perimeter', 'cell_eccentricity', 'cell_circularity',
    'cell_aspect_ratio', 'avg_nucleus_area', 'max_nucleus_area',
    'avg_nucleus_eccentricity', 'nucleus_area_std', 'nucleus_displacement_avg',
    'nucleus_to_cell_area_ratio',
    'nuclear_enlargement_factor', 'cell_enlargement_factor'
]

# Features to log-transform before UMAP and Senescence Score calculation
AREA_FEATURES_TO_LOG = ['cell_area', 'avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'cell_perimeter']

# Weights for calculating the senescence score
SENESCENCE_SCORE_WEIGHTS = {
    'cell_area': 1.5,
    'cell_perimeter': 0.5,
    'cell_eccentricity': 0.5,
    'cell_circularity': -1.0,
    'cell_aspect_ratio': 0.5,
    'avg_nucleus_area': 1.0,
    'avg_nucleus_eccentricity': 0.3,
    'nucleus_to_cell_area_ratio': -1.0,
    'nuclear_enlargement_factor': 1.0,
    'cell_enlargement_factor': 1.5,
    'nucleus_displacement_avg': 0.2,
}

# Rule-based gating definition
RULE_BASED_GATES = [
    {   'name': 'Polynucleated',
        'conditions': [('nuclei_count', '>', 1)],
        'output_label': 'Rule_Sen_Poly' },
    {   'name': 'Very_Large_Cell',
        'conditions': [('cell_area', '>', 4000)],
        'output_label': 'Rule_Sen_VeryLarge' },
    {   'name': 'Low_Circularity',
        'conditions': [('cell_circularity', '<', 0.2)],
        'output_label': 'Rule_Sen_LowCirc' },
    {   'name': 'Low_NucToCellRatio',
        'conditions': [('nucleus_to_cell_area_ratio', '<', 0.1)],
        'output_label': 'Rule_Sen_LowNucRatio' },
    {   'name': 'High_Score_Not_Otherwise_Caught',
        'conditions': [('senescence_score_normalized', '>', 0.85)],
        'output_label': 'Rule_Sen_HighScore' }
]
RULE_BASED_DEFAULT_LABEL = 'Rule_NonSenescent'

POLYNUCLEATED_SENESCENT_RULE_LABEL = 'Rule_Sen_Poly'

# --- NEW: Flow direction and alignment analysis ---
ASSUMED_FLOW_DIRECTION_CARTESIAN_DEGREES = 0  # Angle of flow in standard Cartesian (0 for X-axis right, 90 for Y-axis up)
FLOW_ALIGNMENT_THRESHOLD_DEGREES = 20         # Tolerance in degrees for classifying alignment (e.g., +/- 20 deg)

# --- Color Definitions for Mask Visualization ---
COLOR_NON_SENESCENT = [0, 0, 255]  # Blue
COLOR_SENESCENT_POLYNUCLEATED = [255, 165, 0]  # Orange
COLOR_SENESCENT_OTHER_RULES = [255, 0, 0]  # Red
COLOR_CELL_BOUNDARY = [255, 255, 255]  # White
COLOR_NUCLEI_OUTLINE = [255, 255, 0]  # Yellow
COLOR_UNKNOWN = [128, 128, 128] # Grey

MASK_VISUALIZATION_SUBDIR = "mask_overlays_enhanced"


def extract_sample_id(filename):
    base_name = os.path.splitext(filename)[0]
    if base_name.startswith('denoised_'):
        base_name = base_name[len('denoised_'):]
    pattern = re.compile(r'(.+?_seq\\d+)')
    match = pattern.search(base_name)
    if match: return match.group(1)
    parts = base_name.split('_')
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0: return '_'.join(parts[:i+1])
    print(f"Warning: Fallback ID extraction for '{filename}'.")
    return '_'.join(base_name.split('_')[:7])

def find_mask_files(cell_dir, nuclei_dir):
    print("\n--- Finding and Pairing Mask Files ---")
    cell_files = [f for f in os.listdir(cell_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    nuclei_files = [f for f in os.listdir(nuclei_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    print(f"Found {len(cell_files)} cell files, {len(nuclei_files)} nuclei files")
    nuclei_lookup = {extract_sample_id(nf): nf for nf in nuclei_files if extract_sample_id(nf)}
    file_pairs = []
    for cf in cell_files:
        sid = extract_sample_id(cf)
        if sid and sid in nuclei_lookup:
            file_pairs.append({'cell_file': os.path.join(cell_dir, cf),
                               'nuclei_file': os.path.join(nuclei_dir, nuclei_lookup[sid]),
                               'sample_id': sid})
        elif sid: print(f"Warning: No matching nuclei file for cell ID: {sid} (from {cf})")
        else: print(f"Warning: Could not extract ID for cell file: {cf}")
    print(f"Total matching pairs: {len(file_pairs)}")
    if not file_pairs and (cell_files or nuclei_files): print("CRITICAL: No pairs matched.")
    return file_pairs

def load_mask_image(filepath):
    try:
        img = io.imread(filepath)
        if img.ndim > 2:
            img = img[..., 0] # Take the first channel if multi-channel (e.g. RGB saved mask)

        # If already a labeled integer mask (max value > 1)
        if img.dtype.kind in 'iu' and np.max(img) > 1: # Integer (signed/unsigned)
            return img.astype(np.uint16)

        # If binary (boolean, or uint8 with max 0 or 1) or float, convert to labeled
        if img.dtype == bool or \
           (img.dtype == np.uint8 and np.max(img) <= 1) or \
           img.dtype.kind == 'f':
            # For float, threshold at 0.5; for others, >0
            binary_img = (img > (0.5 if img.dtype.kind == 'f' else 0)).astype(np.uint8)
            labeled_img, _ = ndimage.label(binary_img)
            return labeled_img.astype(np.uint16)

        # If uint8 but looks labeled (max > 1)
        if img.dtype == np.uint8 and np.max(img) > 1:
             return img.astype(np.uint16)

        # Fallback for other unexpected types: attempt a median threshold and label
        print(f"Warning: Unhandled image type {img.dtype} for {filepath}. Attempting basic labeling.")
        labeled_img, _ = ndimage.label((img > np.median(img)).astype(np.uint8))
        return labeled_img.astype(np.uint16)

    except Exception as e: print(f"Error loading image {filepath}: {e}"); return None

def accurately_track_nuclei_in_cells(cell_mask, nuclei_mask):
    labeled_cells, num_cells = cell_mask, np.max(cell_mask)
    labeled_nuclei, num_nuclei = nuclei_mask, np.max(nuclei_mask)
    print(f"Found {num_cells} cells, {num_nuclei} nuclei.")
    if num_cells == 0 or num_nuclei == 0: return {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    cell_props = measure.regionprops(labeled_cells)
    nuclei_props = measure.regionprops(labeled_nuclei)
    results = {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    for cp in cell_props:
        results['cell_data'].append({
            'cell_id': cp.label, 'area': cp.area, 'perimeter': cp.perimeter,
            'eccentricity': getattr(cp, 'eccentricity', 0),
            'orientation': getattr(cp, 'orientation', 0), # Keep orientation in radians here
            'major_axis_length': getattr(cp, 'major_axis_length', 0),
            'minor_axis_length': getattr(cp, 'minor_axis_length', 0),
            'centroid_y': cp.centroid[0], 'centroid_x': cp.centroid[1],
            'nuclei_count': 0, 'nuclei_ids_in_cell': []
        })
        results['cell_nuclei_mapping'][cp.label] = []

    for np_region in nuclei_props:
        nuc_id, nuc_area = np_region.label, np_region.area
        nuc_mask_single = (labeled_nuclei == nuc_id)
        best_cell_id, max_overlap = None, 0
        min_r, min_c, max_r, max_c = np_region.bbox
        # Consider only cell labels within the bounding box of the nucleus for efficiency
        candidate_cell_labels_in_bbox = np.unique(labeled_cells[min_r:max_r, min_c:max_c])

        for cell_cand_lbl in candidate_cell_labels_in_bbox:
            if cell_cand_lbl == 0: continue # Skip background
            # Further check: ensure the candidate cell region actually overlaps with the nucleus bounding box area.
            # This helps if a large cell's label is in unique_labels but its actual pixels are far from nucleus.
            # For simplicity, this check is omitted but could be added with ndimage.find_objects.

            overlap = np.sum(np.logical_and(labeled_cells == cell_cand_lbl, nuc_mask_single))
            if nuc_area > 0: # Avoid division by zero for empty nucleus mask
                current_overlap_ratio = overlap / nuc_area
                if current_overlap_ratio > max_overlap:
                    max_overlap, best_cell_id = current_overlap_ratio, cell_cand_lbl

        if best_cell_id and max_overlap > 0.5: # Nucleus assigned if >50% overlap
            results['nuclei_data'].append({
                'nucleus_id': nuc_id, 'cell_id': best_cell_id, 'area': nuc_area,
                'eccentricity': getattr(np_region, 'eccentricity', 0),
                'centroid_y': np_region.centroid[0], 'centroid_x': np_region.centroid[1],
                'overlap_ratio': max_overlap
            })
            for cd_item in results['cell_data']:
                if cd_item['cell_id'] == best_cell_id:
                    cd_item['nuclei_count']+=1
                    cd_item['nuclei_ids_in_cell'].append(nuc_id)
                    break
            results['cell_nuclei_mapping'][best_cell_id].append(nuc_id)
    return results

def extract_features_for_cell(cd_item, cn_map, nuc_data_list):
    feats = {'cell_area': cd_item['area'], 'cell_perimeter': cd_item['perimeter'],
             'cell_eccentricity': cd_item.get('eccentricity',0)}
    feats['cell_circularity'] = (4*np.pi*feats['cell_area']/(feats['cell_perimeter']**2)) if feats['cell_perimeter']>0 else 0
    maj, minor = cd_item.get('major_axis_length'), cd_item.get('minor_axis_length')
    feats['cell_aspect_ratio'] = (maj/minor) if maj and minor and minor>0 else 1.0

    orig_cell_lbl = cd_item['cell_id']
    nuc_orig_lbls = cn_map.get(orig_cell_lbl, [])
    feats['nuclei_count'] = len(nuc_orig_lbls)

    for k_nuc in ['avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'avg_nucleus_eccentricity',
                  'nucleus_area_std', 'nucleus_displacement_avg', 'nucleus_to_cell_area_ratio']: feats[k_nuc]=0
    if feats['nuclei_count'] > 0:
        nuc_details_incell = [nd for nd in nuc_data_list if nd['nucleus_id'] in nuc_orig_lbls and nd['cell_id']==orig_cell_lbl]
        if nuc_details_incell:
            areas = [n['area'] for n in nuc_details_incell]; eccs = [n.get('eccentricity',0) for n in nuc_details_incell]
            feats.update({
                'avg_nucleus_area': np.mean(areas) if areas else 0, 'total_nuclear_area': sum(areas),
                'max_nucleus_area': max(areas) if areas else 0, 'avg_nucleus_eccentricity': np.mean(eccs) if eccs else 0,
                'nucleus_area_std': np.std(areas) if len(areas)>1 else 0,
                'nucleus_to_cell_area_ratio': (sum(areas)/feats['cell_area']) if feats['cell_area']>0 else 0
            })
            disps = [np.sqrt((n['centroid_x']-cd_item['centroid_x'])**2 + (n['centroid_y']-cd_item['centroid_y'])**2) for n in nuc_details_incell]
            feats['nucleus_displacement_avg'] = np.mean(disps) if disps else 0

    norm_nuc_A, norm_cell_A = 500, 2000 # Placeholder normal areas
    feats['nuclear_enlargement_factor'] = feats.get('avg_nucleus_area',0)/norm_nuc_A if norm_nuc_A>0 and feats.get('avg_nucleus_area',0)>0 else 1.0
    feats['cell_enlargement_factor'] = feats['cell_area']/norm_cell_A if norm_cell_A>0 and feats['cell_area']>0 else 1.0

    # --- NEW: Cell Orientation and Flow Alignment ---
    # cd_item['orientation'] is from regionprops: angle between Y-axis (0th axis, rows) and major axis,
    # in radians, ranging from -pi/2 to pi/2 counter-clockwise.
    # Image Y-axis typically points downwards. Image X-axis points right.

    raw_orientation_rad = cd_item.get('orientation', 0)
    feats['cell_orientation_y_axis_deg'] = np.degrees(raw_orientation_rad)

    # Convert to standard Cartesian angle for the major axis line: [0, 180) degrees.
    # 0 degrees = horizontal (along X-axis), 90 degrees = vertical (along Y-axis).
    # If raw_orientation_rad = 0 (cell vertical in image), std_angle = 90 deg.
    # If raw_orientation_rad = pi/2 (cell horizontal in image), std_angle = 0 deg.
    # If raw_orientation_rad = -pi/2 (cell horizontal in image), std_angle = 0 deg (after normalization to [0,180)).
    # The formula is: (pi/2 - raw_orientation_rad), then normalize angle to be within [0, pi).
    cell_major_axis_cartesian_rad = (np.pi/2 - raw_orientation_rad)
    # Normalize to [0, pi) range (or 0-180 degrees)
    cell_major_axis_cartesian_rad %= np.pi
    if cell_major_axis_cartesian_rad < 0: # Ensure positive if modulo resulted negative with negative input
        cell_major_axis_cartesian_rad += np.pi

    feats['cell_major_axis_cartesian_deg'] = np.degrees(cell_major_axis_cartesian_rad)

    # Calculate angular misalignment with the defined flow direction
    # Normalize flow direction also to [0, 180)
    flow_dir_norm_deg = ASSUMED_FLOW_DIRECTION_CARTESIAN_DEGREES % 180

    angle_diff_deg = abs(feats['cell_major_axis_cartesian_deg'] - flow_dir_norm_deg)
    # Get the acute angle difference (e.g., diff between 10 deg and 170 deg is 20 deg, not 160)
    angular_misalignment_deg = min(angle_diff_deg, 180 - angle_diff_deg)
    feats['angular_misalignment_to_flow_deg'] = angular_misalignment_deg

    # Categorize alignment
    if angular_misalignment_deg < FLOW_ALIGNMENT_THRESHOLD_DEGREES:
        feats['flow_orientation_category'] = "Aligned with flow"
    # Check for perpendicular alignment (misalignment is close to 90 degrees)
    elif abs(angular_misalignment_deg - 90) < FLOW_ALIGNMENT_THRESHOLD_DEGREES:
        feats['flow_orientation_category'] = "Perpendicular to flow"
    else:
        feats['flow_orientation_category'] = "Oblique to flow"

    return feats

def calculate_senescence_score(df, weights, log_features):
    print("\nCalculating senescence score...")
    present_feats = [f for f in weights.keys() if f in df.columns]
    if not present_feats:
        print("  Warning: No features for senescence score calculation are present. Score will be NaN.")
        df['senescence_score'], df['senescence_score_normalized'] = np.nan, np.nan
        return df

    score_subset = df[present_feats].copy()
    for col in log_features: # Apply log transform to specified features for scoring
        if col in score_subset.columns:
            score_subset[col] = np.log1p(score_subset[col])
            print(f"  Log-transformed for scoring: {col}")

    scaler = StandardScaler()
    num_cols = score_subset.select_dtypes(include=np.number).columns
    std_df = pd.DataFrame(index=score_subset.index) # Initialize empty or with index
    if not num_cols.empty:
        std_vals = scaler.fit_transform(score_subset[num_cols])
        std_df = pd.DataFrame(std_vals, columns=num_cols, index=score_subset.index)
    else:
        print("  Warning: No numeric columns in scoring subset for standardization.")


    df['senescence_score'] = 0.0 # Initialize score column
    for f, w in weights.items():
        if f in std_df.columns:
             df['senescence_score'] += std_df[f] * w
        # else: # Feature not in std_df (e.g. was not numeric or not present initially)
             # print(f"  Warning: Feature '{f}' for scoring not found in standardized set. Skipping.")

    min_s, max_s = df['senescence_score'].min(), df['senescence_score'].max()
    if pd.isna(min_s) or pd.isna(max_s) or min_s == max_s :
        df['senescence_score_normalized'] = 0.5 # Default if score is NaN or uniform
        print("  Warning: Senescence score is NaN or uniform. Normalized score set to 0.5.")
    else:
        df['senescence_score_normalized'] = (df['senescence_score']-min_s)/(max_s-min_s)
    print("Senescence score calculation complete.")
    return df

def apply_rule_based_classification(df, rules, default_label):
    print("\n--- Applying Rule-Based Classification ---")
    df['rule_based_classification_granular'] = default_label
    all_rule_feats = set(cond[0] for r in rules for cond in r['conditions'])

    missing_features = [feat for feat in all_rule_feats if feat not in df.columns]
    if missing_features:
        print(f"  ERROR: Critical features for rule-based gating are missing: {missing_features}. Aborting rule application.")
        df['cell_type'] = 'Unknown_Rules_Missing_Features'
        return df

    for rule in rules:
        print(f"  Applying rule: {rule['name']}")
        # Only apply rule to cells still carrying the default label
        eligible_mask = (df['rule_based_classification_granular'] == default_label)
        if not eligible_mask.any(): # No cells eligible for this rule if all already classified by prior rules
            print(f"    No cells eligible for rule '{rule['name']}' (all previously classified).")
            continue

        current_rule_mask = pd.Series([True]*len(df), index=df.index) # Start with all true for current rule conditions
        for feat, op, val in rule['conditions']:
            if feat not in df.columns: # Should have been caught by missing_features check
                print(f"    Missing feature '{feat}' for rule '{rule['name']}'. Rule cannot be applied.")
                current_rule_mask[:] = False; break

            feat_series = pd.to_numeric(df[feat], errors='coerce')
            nan_mask = feat_series.isnull() # Mark NaNs in feature column

            if op=='>': current_rule_mask &= (feat_series > val)
            elif op=='<': current_rule_mask &= (feat_series < val)
            elif op=='>=': current_rule_mask &= (feat_series >=val)
            elif op=='<=': current_rule_mask &= (feat_series <=val)
            elif op=='==': current_rule_mask &= (feat_series ==value)
            elif op=='!=': current_rule_mask &= (feat_series !=value)
            else:
                print(f"    Unknown operator '{op}' in rule '{rule['name']}'. Rule skipped.")
                current_rule_mask[:]=False; break
            current_rule_mask[nan_mask] = False # Exclude NaNs from satisfying condition

        if not current_rule_mask.any(): # If no cells satisfy this rule's conditions
            print(f"    No cells met conditions for rule '{rule['name']}'.")
            continue

        # Apply label only to cells that are both eligible (default label) AND satisfy current rule
        cells_to_label_now = eligible_mask & current_rule_mask
        df.loc[cells_to_label_now, 'rule_based_classification_granular'] = rule['output_label']
        print(f"    {cells_to_label_now.sum()} cells labeled as '{rule['output_label']}'.")

    df['cell_type'] = np.where(df['rule_based_classification_granular']==default_label, 'Non-senescent', 'Senescent')
    print(f"\nGranular counts:\n{df['rule_based_classification_granular'].value_counts(dropna=False)}")
    print(f"\nBinary 'cell_type' counts:\n{df['cell_type'].value_counts(dropna=False)}")
    return df

def generate_umap_coordinates(df, umap_feats_config, log_feats_config, n_neigh=15, min_d=0.1, rand_state=42):
    print("\n--- Generating UMAP Coordinates ---")
    actual_umap_feats = [f for f in umap_feats_config if f in df.columns]
    if not actual_umap_feats:
        print("  Error: No features for UMAP are present. Skipping UMAP generation.")
        df['umap_x'], df['umap_y'] = np.nan, np.nan
        return df

    umap_df = df[actual_umap_feats].copy()
    for col in log_feats_config: # Apply log transform for UMAP
        if col in umap_df.columns:
            umap_df[col] = np.log1p(umap_df[col])
            print(f"  Log-transformed for UMAP: {col}")

    if umap_df.isnull().values.any():
        print(f"  Found {umap_df.isnull().values.sum()} NaN values in UMAP features. Filling with column means.")
        umap_df = umap_df.fillna(umap_df.mean())

    if umap_df.isnull().values.any(): # Check again if NaNs persist (e.g. if a whole column was NaN)
        print("  Error: NaNs still present after attempting to fill. UMAP cannot proceed.")
        df['umap_x'],df['umap_y']=np.nan,np.nan
        return df

    scaled_feats = StandardScaler().fit_transform(umap_df)
    actual_n_neigh = min(n_neigh, scaled_feats.shape[0]-1)
    if actual_n_neigh < 2: # UMAP requires n_neighbors >= 2
        print(f"  Warning: Not enough samples ({scaled_feats.shape[0]}) for UMAP with n_neighbors={n_neigh}. Skipping UMAP.")
        df['umap_x'],df['umap_y']=np.nan,np.nan
        return df

    reducer = umap.UMAP(n_neighbors=actual_n_neigh, min_dist=min_d, random_state=rand_state, n_components=2)
    embedding = reducer.fit_transform(scaled_feats)
    df['umap_x'], df['umap_y'] = embedding[:,0], embedding[:,1]
    print("UMAP coordinates generated.")
    return df

def visualize_results(df_viz, out_dir):
    print("\n--- Creating Visualizations (Rule-Based) ---")
    if df_viz.empty or 'umap_x' not in df_viz or df_viz['umap_x'].isnull().all():
        print("Skipping UMAP plots as UMAP data is missing or empty.")
        return

    # Binary UMAP plot (Senescent/Non-senescent)
    plt.figure(figsize=(10,8));
    if 'cell_type' in df_viz and df_viz['cell_type'].notna().any():
        sns.scatterplot(x='umap_x', y='umap_y', hue='cell_type',
                        palette={'Senescent':'red',
                                 'Non-senescent':'blue',
                                 'Unknown_Rules_Missing_Features':'grey'}, # Added for robustness
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.legend(title='Cell Type (Rule-Based)')
    else:
        plt.scatter(df_viz['umap_x'], df_viz['umap_y'], s=30, alpha=0.5) # Fallback if no cell_type
    plt.title('UMAP by Rule-Based Classification (Binary)'); plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join(out_dir, 'umap_rule_based_binary.png'), dpi=300, bbox_inches='tight'); plt.close()
    print("Binary UMAP saved.")

    # Granular UMAP plot (detailed rule labels)
    if 'rule_based_classification_granular' in df_viz and df_viz['rule_based_classification_granular'].notna().any():
        plt.figure(figsize=(12,10));
        unique_granular_labels = sorted(df_viz['rule_based_classification_granular'].unique())

        # Define a color palette for granular labels
        palette_granular = {}
        # Get distinct "Rule_Sen_" prefixes for varied colors
        sen_rule_outputs = sorted(list(set(r['output_label'] for r in RULE_BASED_GATES if r['output_label'].startswith("Rule_Sen_"))))

        # Generate enough distinct colors for senescent categories + non-senescent + unknown
        num_sen_cats = len(sen_rule_outputs)
        # Using a qualitative palette that offers more distinct colors
        base_colors = sns.color_palette("Paired", n_colors=max(1, num_sen_cats))

        color_idx = 0
        for label in unique_granular_labels:
            if label == RULE_BASED_DEFAULT_LABEL:
                palette_granular[label] = 'blue' # Consistent color for non-senescent
            elif label.startswith("Rule_Sen_"):
                try: # Try to map to a consistent color based on order in RULE_BASED_GATES
                    rule_order_idx = sen_rule_outputs.index(label)
                    palette_granular[label] = base_colors[rule_order_idx % len(base_colors)]
                except ValueError: # Fallback if label not in predefined sen_rule_outputs (should not happen)
                     palette_granular[label] = base_colors[color_idx % len(base_colors)]; color_idx+=1
            else: # For any other labels like 'Unknown_Rules_Missing_Features'
                palette_granular[label] = 'grey'

        sns.scatterplot(x='umap_x', y='umap_y', hue='rule_based_classification_granular',
                        palette=palette_granular, hue_order=unique_granular_labels,
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.title('UMAP by Granular Rule Classification'); plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.grid(True, alpha=0.3)
        plt.legend(title='Rule Applied', bbox_to_anchor=(1.05,1), loc='upper left', borderaxespad=0.); plt.tight_layout(rect=[0,0,0.85,1]) # Adjust for legend
        plt.savefig(os.path.join(out_dir, 'umap_rule_based_granular.png'), dpi=300); plt.close()
        print("Granular UMAP saved.")

def save_results(df_proc, out_dir):
    if df_proc.empty: print("No data to save."); return
    if not os.path.exists(out_dir): os.makedirs(out_dir)

    output_csv_path = os.path.join(out_dir, 'cell_classification_rule_based_full.csv')
    df_proc.to_csv(output_csv_path, index=False)
    print(f"Full rule-based results saved to: {output_csv_path}")

    if 'sample_id' in df_proc and 'cell_type' in df_proc:
        summary = df_proc.groupby('sample_id')['cell_type'].value_counts(normalize=False).unstack(fill_value=0)
        # Ensure all expected columns are present in summary
        for col in ['Senescent','Non-senescent','Unknown_Rules_Missing_Features']: # Match potential cell_type values
            if col not in summary.columns: summary[col]=0

        summary['total_cells'] = summary.sum(axis=1) # Recalculate total based on actual columns

        if 'Senescent' in summary.columns and (summary['total_cells'] > 0).any():
            summary['percent_senescent'] = (summary['Senescent']/summary['total_cells']*100)
        else:
            summary['percent_senescent'] = 0 # Handle case where no senescent cells or no cells at all

        summary.to_csv(os.path.join(out_dir, 'sample_summary_rule_based.csv'))
        print(f"Rule-based sample summary saved.")

        if 'percent_senescent' in summary.columns:
            plt.figure(figsize=(12,6))
            sns.barplot(x=summary.index, y='percent_senescent', data=summary.reset_index(), color='red')
            plt.title('Senescent Cells by Sample (Rule-Based Classification)')
            plt.xlabel('Sample ID')
            plt.ylabel('Senescent Cells (%)')
            plt.xticks(rotation=90, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(out_dir, 'senescent_percentage_rule_based.png'), dpi=300); plt.close()
            print("Senescent percentage plot saved.")

def visualize_enhanced_classification_on_masks(df_results, cell_mask_dir_path, nuclei_mask_dir_path, output_dir_masks_path):
    print(f"\nGenerating enhanced classification overlays in: {output_dir_masks_path}")
    if not os.path.exists(output_dir_masks_path):
        os.makedirs(output_dir_masks_path)

    required_cols = ['sample_id', 'cell_id_unique', 'original_cell_label', 'rule_based_classification_granular']
    if not all(col in df_results.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_results.columns]
        print(f"  Error: DataFrame missing required columns for mask viz: {missing}. Skipping visualization.")
        return

    # Create a lookup dictionary from cell_id_unique to its granular classification
    classification_lookup = pd.Series(
        df_results.rule_based_classification_granular.values,
        index=df_results.cell_id_unique
    ).to_dict()

    unique_samples = df_results['sample_id'].unique()

    # Get available mask files to avoid errors if some are missing
    available_cell_masks = {extract_sample_id(f): f for f in os.listdir(cell_mask_dir_path) if f.endswith(('.tif', '.tiff'))}
    available_nuclei_masks = {extract_sample_id(f): f for f in os.listdir(nuclei_mask_dir_path) if f.endswith(('.tif', '.tiff'))}


    for sample_id_val in tqdm(unique_samples, desc="Generating Enhanced Mask Overlays"):
        cell_mask_fname = available_cell_masks.get(sample_id_val)
        nuclei_mask_fname = available_nuclei_masks.get(sample_id_val)

        if not cell_mask_fname:
            print(f"  Warning: Cell mask file not found for sample ID: {sample_id_val}. Skipping overlay.")
            continue

        print(f"\n  Processing sample for enhanced overlay: {sample_id_val}")
        labeled_cell_mask_img = load_mask_image(os.path.join(cell_mask_dir_path, cell_mask_fname))
        if labeled_cell_mask_img is None:
            print(f"  Warning: Could not load cell mask for {sample_id_val}. Skipping.")
            continue

        overlay_img = np.zeros((labeled_cell_mask_img.shape[0], labeled_cell_mask_img.shape[1], 3), dtype=np.uint8)

        # Iterate through each cell region in the mask
        for cell_region_props in measure.regionprops(labeled_cell_mask_img):
            original_label_from_mask = cell_region_props.label
            # Construct the unique ID as it was created in the main DataFrame
            lookup_id = f"{sample_id_val}_{original_label_from_mask}"

            granular_status_label = classification_lookup.get(lookup_id, 'Unknown_Label_In_CSV') # Default if ID not in results

            fill_color = COLOR_UNKNOWN # Default fill
            if granular_status_label == POLYNUCLEATED_SENESCENT_RULE_LABEL:
                fill_color = COLOR_SENESCENT_POLYNUCLEATED
            elif granular_status_label.startswith('Rule_Sen_'): # Other senescent rules
                fill_color = COLOR_SENESCENT_OTHER_RULES
            elif granular_status_label == RULE_BASED_DEFAULT_LABEL: # Non-senescent
                fill_color = COLOR_NON_SENESCENT

            overlay_img[labeled_cell_mask_img == original_label_from_mask] = fill_color

        # Draw cell boundaries
        cell_boundaries_img = segmentation.find_boundaries(labeled_cell_mask_img, mode='outer', background=0)
        overlay_img[cell_boundaries_img] = COLOR_CELL_BOUNDARY

        # Draw nuclei outlines if nuclei mask is available
        labeled_nuc_mask_img = None
        if nuclei_mask_fname:
            labeled_nuc_mask_img = load_mask_image(os.path.join(nuclei_mask_dir_path, nuclei_mask_fname))
            if labeled_nuc_mask_img is not None:
                nuc_boundaries_img = segmentation.find_boundaries(labeled_nuc_mask_img, mode='inner', background=0)
                overlay_img[nuc_boundaries_img] = COLOR_NUCLEI_OUTLINE
            else:
                print(f"  Warning: Could not load nuclei mask for {sample_id_val} for overlay.")

        # Plotting with legend
        fig_plot, ax_plot = None, None # Initialize for finally block
        try:
            h_img, w_img = overlay_img.shape[:2]
            # Dynamic figure sizing
            fig_w_calc = max(10, w_img / 100 if w_img > 0 else 10)
            # Adjust height to maintain aspect ratio, with legend space considered
            fig_h_calc = max(8, (h_img / 100 if h_img > 0 else 8) * (fig_w_calc / (w_img/100 if w_img > 0 else 1)))


            fig_plot, ax_plot = plt.subplots(figsize=(fig_w_calc, fig_h_calc), dpi=100) # Adjust dpi for resolution
            ax_plot.imshow(overlay_img)

            # Create legend handles
            legend_handles = [
                mpatches.Patch(color=np.array(COLOR_NON_SENESCENT)/255., label=f"{RULE_BASED_DEFAULT_LABEL} (Non-Senescent)"),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_OTHER_RULES)/255., label='Senescent (Other Rules)'),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_POLYNUCLEATED)/255., label=f'Senescent ({POLYNUCLEATED_SENESCENT_RULE_LABEL})')
            ]

            # Check if any cells in the current image were 'Unknown_Label_In_CSV' to add to legend
            unique_ids_current_sample = [f"{sample_id_val}_{p.label}" for p in measure.regionprops(labeled_cell_mask_img)]
            is_unknown_present_in_image = any(classification_lookup.get(uid, '') == 'Unknown_Label_In_CSV' for uid in unique_ids_current_sample)

            if is_unknown_present_in_image:
                 legend_handles.append(mpatches.Patch(color=np.array(COLOR_UNKNOWN)/255., label='Unknown/Not in CSV'))

            if nuclei_mask_fname and labeled_nuc_mask_img is not None:
                legend_handles.append(mpatches.Patch(color=np.array(COLOR_NUCLEI_OUTLINE)/255., label='Nuclei Outline'))
            legend_handles.append(mpatches.Patch(edgecolor=np.array(COLOR_CELL_BOUNDARY)/255., facecolor='none', label='Cell Boundary', linewidth=1))


            # Position legend outside the plot to the right
            ax_plot.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1.02, 0.5), fontsize='small', title="Legend")
            ax_plot.axis('off') # Turn off axis numbers and ticks
            # Adjust layout to make space for the legend
            plt.tight_layout(rect=[0, 0, 0.83, 1]) # rect=[left, bottom, right, top]

            output_img_filename = os.path.join(output_dir_masks_path, f"{sample_id_val}_enhanced_mask_overlay.png")
            plt.savefig(output_img_filename, dpi=200) # Save with good resolution
            print(f"    Saved enhanced overlay for {sample_id_val} to {output_img_filename}")
        except Exception as e_plt:
            print(f"    Error during plotting/saving mask overlay for {sample_id_val}: {e_plt}")
        finally:
            if fig_plot: # Ensure figure is closed to free memory
                plt.close(fig_plot)

    print("\nEnhanced mask overlay visualization complete.")


def main(base_dir, cell_subdir, nuclei_subdir, output_parent_dir):
    print(f"=== Senescence Analysis (Rule-Based with Flow) for {base_dir} ===")
    cell_mask_dir_path = os.path.join(base_dir, cell_subdir)
    nuclei_mask_dir_path = os.path.join(base_dir, nuclei_subdir)
    dataset_folder_name = os.path.basename(base_dir)
    main_output_dir = os.path.join(output_parent_dir, dataset_folder_name, "Senescence_Results")
    if not os.path.exists(main_output_dir): os.makedirs(main_output_dir); print(f"Created output directory: {main_output_dir}")

    file_pairs_list = find_mask_files(cell_mask_dir_path, nuclei_mask_dir_path)
    if not file_pairs_list: print("No file pairs found. Exiting."); return

    features_list = []
    for pair_info in tqdm(file_pairs_list, desc="Processing image pairs"):
        print(f"\nProcessing sample: {pair_info['sample_id']}")
        try:
            cm_img = load_mask_image(pair_info['cell_file'])
            nm_img = load_mask_image(pair_info['nuclei_file'])
            if cm_img is None or nm_img is None:
                print(f"Skipping {pair_info['sample_id']} due to mask loading error.")
                continue

            track_res = accurately_track_nuclei_in_cells(cm_img, nm_img)
            for cell_d in track_res['cell_data']: # cell_d contains 'orientation' in radians
                cell_feats = extract_features_for_cell(cell_d, track_res['cell_nuclei_mapping'], track_res['nuclei_data'])
                cell_feats['cell_id_unique'] = f"{pair_info['sample_id']}_{cell_d['cell_id']}"
                cell_feats['sample_id'] = pair_info['sample_id']
                cell_feats['original_cell_label'] = cell_d['cell_id'] # Keep original label from mask
                features_list.append(cell_feats)
        except Exception as e_proc:
            print(f"Error processing {pair_info['sample_id']}: {e_proc}")
            import traceback; traceback.print_exc() # Print full traceback for debugging

    if not features_list: print("No cell features were extracted. Please check input files and processing steps."); return
    master_df = pd.DataFrame(features_list)
    print(f"\nTotal cells extracted across all samples: {len(master_df)}")

    master_df = calculate_senescence_score(master_df, SENESCENCE_SCORE_WEIGHTS, AREA_FEATURES_TO_LOG)
    master_df = generate_umap_coordinates(master_df, FEATURES_FOR_UMAP, AREA_FEATURES_TO_LOG) # Use defined constants
    final_classified_df = apply_rule_based_classification(master_df, RULE_BASED_GATES, RULE_BASED_DEFAULT_LABEL)

    save_results(final_classified_df, main_output_dir)
    visualize_results(final_classified_df, main_output_dir)

    # Generate enhanced mask visualizations
    enhanced_masks_output_path = os.path.join(main_output_dir, MASK_VISUALIZATION_SUBDIR)
    visualize_enhanced_classification_on_masks(final_classified_df, cell_mask_dir_path, nuclei_mask_dir_path, enhanced_masks_output_path)

    print("\nRule-Based Analysis and Mask Visualization complete!")

if __name__ == "__main__":
    segmented_data_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented"
    current_dataset_folder_name = "1.4Pa-x20" # Ensure this matches your folder name
    base_input_dir = os.path.join(segmented_data_parent_dir, current_dataset_folder_name)

    cell_masks_subdir = "Cell_merged_conservative" # Subdirectory for cell masks
    nuclei_masks_subdir = "Nuclei"                 # Subdirectory for nuclei masks

    analysis_output_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis"

    if not os.path.isdir(base_input_dir):
        print(f"Error: Base input directory not found: {base_input_dir}")
        print(f"Please ensure the '{current_dataset_folder_name}' folder exists at the specified location within '{segmented_data_parent_dir}'.")
    else:
        main(base_input_dir, cell_masks_subdir, nuclei_masks_subdir, analysis_output_parent_dir)

=== Senescence Analysis (Rule-Based with Flow) for /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20 ===

--- Finding and Pairing Mask Files ---
Found 23 cell files, 23 nuclei files
Total matching pairs: 23


Processing image pairs:   0%|          | 0/23 [00:00<?, ?it/s]


Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq001
Found 326 cells, 347 nuclei.


Processing image pairs:   4%|▍         | 1/23 [00:01<00:26,  1.18s/it]


Processing sample: 1.4Pa_A1_19dec21_20xA_L2RA_FlatA_seq002
Found 273 cells, 290 nuclei.


...

    Saved enhanced overlay for 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016 to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/mask_overlays_enhanced/1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq016_enhanced_mask_overlay.png

  Processing sample for enhanced overlay: 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017


Generating Enhanced Mask Overlays:  96%|█████████▌| 22/23 [00:35<00:01,  1.53s/it]

    Saved enhanced overlay for 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017 to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/mask_overlays_enhanced/1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq017_enhanced_mask_overlay.png

  Processing sample for enhanced overlay: 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018


Generating Enhanced Mask Overlays: 100%|██████████| 23/23 [00:37<00:00,  1.62s/it]

    Saved enhanced overlay for 1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018 to /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/mask_overlays_enhanced/1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq018_enhanced_mask_overlay.png

Enhanced mask overlay visualization complete.

Rule-Based Analysis and Mask Visualization complete!


In [15]:
import os
import numpy as np
import pandas as pd
# import cv2 # Not directly used in this snippet, but often part of image processing workflows
import re
from scipy import ndimage
from skimage import io, measure, segmentation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans # Kept for potential exploratory use but not central
import umap
from tqdm import tqdm

# --- Configuration & Parameters ---
# Features to be used for UMAP generation
FEATURES_FOR_UMAP = [
    'cell_area', 'cell_perimeter', 'cell_eccentricity', 'cell_circularity',
    'cell_aspect_ratio', 'avg_nucleus_area', 'max_nucleus_area',
    'avg_nucleus_eccentricity', 'nucleus_area_std', 'nucleus_displacement_avg',
    'nucleus_to_cell_area_ratio',
    'nuclear_enlargement_factor', 'cell_enlargement_factor'
]

# Features to log-transform before UMAP and Senescence Score calculation
AREA_FEATURES_TO_LOG = ['cell_area', 'avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'cell_perimeter']

# Weights for calculating the senescence score
SENESCENCE_SCORE_WEIGHTS = {
    'cell_area': 1.5,
    'cell_perimeter': 0.5,
    'cell_eccentricity': 0.5,
    'cell_circularity': -1.0,
    'cell_aspect_ratio': 0.5,
    'avg_nucleus_area': 1.0,
    'avg_nucleus_eccentricity': 0.3,
    'nucleus_to_cell_area_ratio': -1.0,
    'nuclear_enlargement_factor': 1.0,
    'cell_enlargement_factor': 1.5,
    'nucleus_displacement_avg': 0.2,
}

# Rule-based gating definition
RULE_BASED_GATES = [
    {   'name': 'Polynucleated',
        'conditions': [('nuclei_count', '>', 1)],
        'output_label': 'Rule_Sen_Poly' },
    {   'name': 'Very_Large_Cell',
        'conditions': [('cell_area', '>', 4000)], # Example value
        'output_label': 'Rule_Sen_VeryLarge' },
    {   'name': 'Low_Circularity',
        'conditions': [('cell_circularity', '<', 0.2)], # Example value
        'output_label': 'Rule_Sen_LowCirc' },
    {   'name': 'Low_NucToCellRatio',
        'conditions': [('nucleus_to_cell_area_ratio', '<', 0.1)], # Example value
        'output_label': 'Rule_Sen_LowNucRatio' },
    {   'name': 'High_Score_Not_Otherwise_Caught',
        'conditions': [('senescence_score_normalized', '>', 0.85)], # Example value
        'output_label': 'Rule_Sen_HighScore' }
]
RULE_BASED_DEFAULT_LABEL = 'Rule_NonSenescent'

POLYNUCLEATED_SENESCENT_RULE_LABEL = 'Rule_Sen_Poly' # Must match an output_label from RULE_BASED_GATES

# Flow direction and alignment analysis
ASSUMED_FLOW_DIRECTION_CARTESIAN_DEGREES = 0  # Angle of flow in standard Cartesian (0 for X-axis right, 90 for Y-axis up)
FLOW_ALIGNMENT_THRESHOLD_DEGREES = 20         # Tolerance in degrees for classifying alignment (e.g., +/- 20 deg)

# Color Definitions for Mask Visualization
COLOR_NON_SENESCENT = [0, 0, 255]  # Blue
COLOR_SENESCENT_POLYNUCLEATED = [255, 165, 0]  # Orange
COLOR_SENESCENT_OTHER_RULES = [255, 0, 0]  # Red
COLOR_CELL_BOUNDARY = [255, 255, 255]  # White
COLOR_NUCLEI_OUTLINE = [255, 255, 0]  # Yellow
COLOR_UNKNOWN = [128, 128, 128] # Grey

MASK_VISUALIZATION_SUBDIR = "mask_overlays_enhanced"


def extract_sample_id(filename):
    base_name = os.path.splitext(filename)[0]
    if base_name.startswith('denoised_'):
        base_name = base_name[len('denoised_'):]
    pattern = re.compile(r'(.+?_seq\\d+)')
    match = pattern.search(base_name)
    if match: return match.group(1)
    parts = base_name.split('_')
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0: return '_'.join(parts[:i+1])
    print(f"Warning: Fallback ID extraction for '{filename}'.")
    return '_'.join(base_name.split('_')[:7]) # Fallback

def find_mask_files(cell_dir, nuclei_dir):
    print("\n--- Finding and Pairing Mask Files ---")
    cell_files = [f for f in os.listdir(cell_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    nuclei_files = [f for f in os.listdir(nuclei_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    print(f"Found {len(cell_files)} cell files, {len(nuclei_files)} nuclei files")
    nuclei_lookup = {extract_sample_id(nf): nf for nf in nuclei_files if extract_sample_id(nf)}
    file_pairs = []
    for cf in cell_files:
        sid = extract_sample_id(cf)
        if sid and sid in nuclei_lookup:
            file_pairs.append({'cell_file': os.path.join(cell_dir, cf),
                               'nuclei_file': os.path.join(nuclei_dir, nuclei_lookup[sid]),
                               'sample_id': sid})
        elif sid: print(f"Warning: No matching nuclei file for cell sample ID: {sid} (from file {cf})")
        else: print(f"Warning: Could not extract sample ID for cell file: {cf}")
    print(f"Total matching cell-nuclei file pairs found: {len(file_pairs)}")
    if not file_pairs and (cell_files or nuclei_files): print("CRITICAL WARNING: No file pairs were matched.")
    return file_pairs

def load_mask_image(filepath):
    try:
        img = io.imread(filepath)
        if img.ndim > 2:
            img = img[..., 0] # Take the first channel if multi-channel

        if img.dtype.kind in 'iu' and np.max(img) > 1: # Already labeled integer mask
            return img.astype(np.uint16)

        if img.dtype == bool or \
           (img.dtype == np.uint8 and np.max(img) <= 1) or \
           img.dtype.kind == 'f': # Binary or float mask, needs labeling
            binary_img = (img > (0.5 if img.dtype.kind == 'f' else 0)).astype(np.uint8)
            labeled_img, _ = ndimage.label(binary_img)
            return labeled_img.astype(np.uint16)

        if img.dtype == np.uint8 and np.max(img) > 1: # uint8 that looks labeled
             return img.astype(np.uint16)

        print(f"Warning: Unhandled image type {img.dtype} for {filepath}. Attempting basic labeling.")
        labeled_img, _ = ndimage.label((img > np.median(img)).astype(np.uint8)) # Fallback
        return labeled_img.astype(np.uint16)

    except Exception as e: print(f"Error loading image {filepath}: {e}"); return None

def accurately_track_nuclei_in_cells(cell_mask, nuclei_mask):
    labeled_cells, num_cells = cell_mask, np.max(cell_mask)
    labeled_nuclei, num_nuclei = nuclei_mask, np.max(nuclei_mask)
    print(f"Found {num_cells} potential cell objects and {num_nuclei} potential nuclei objects.")
    if num_cells == 0 or num_nuclei == 0: return {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    cell_props = measure.regionprops(labeled_cells)
    nuclei_props = measure.regionprops(labeled_nuclei)
    results = {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    for cp in cell_props:
        results['cell_data'].append({
            'cell_id': cp.label, 'area': cp.area, 'perimeter': cp.perimeter,
            'eccentricity': getattr(cp, 'eccentricity', 0),
            'orientation': getattr(cp, 'orientation', 0), # Keep in radians for precision
            'major_axis_length': getattr(cp, 'major_axis_length', 0),
            'minor_axis_length': getattr(cp, 'minor_axis_length', 0),
            'centroid_y': cp.centroid[0], 'centroid_x': cp.centroid[1],
            'nuclei_count': 0, 'nuclei_ids_in_cell': []
        })
        results['cell_nuclei_mapping'][cp.label] = []

    for np_region in nuclei_props:
        nuc_id, nuc_area = np_region.label, np_region.area
        nuc_mask_single = (labeled_nuclei == nuc_id)
        best_cell_id, max_overlap_ratio_found = None, 0.0 # Renamed for clarity
        min_r, min_c, max_r, max_c = np_region.bbox
        candidate_cell_labels_in_bbox = np.unique(labeled_cells[min_r:max_r, min_c:max_c])

        for cell_cand_lbl in candidate_cell_labels_in_bbox:
            if cell_cand_lbl == 0: continue # Skip background label

            overlap_pixels = np.sum(np.logical_and(labeled_cells == cell_cand_lbl, nuc_mask_single))
            if nuc_area > 0: # Avoid division by zero
                current_overlap_ratio = overlap_pixels / nuc_area
                if current_overlap_ratio > max_overlap_ratio_found:
                    max_overlap_ratio_found = current_overlap_ratio
                    best_cell_id = cell_cand_lbl

        if best_cell_id is not None and max_overlap_ratio_found > 0.5: # Assign if >50% of nucleus overlaps
            results['nuclei_data'].append({
                'nucleus_id': nuc_id, 'cell_id': best_cell_id, 'area': nuc_area,
                'eccentricity': getattr(np_region, 'eccentricity', 0),
                'centroid_y': np_region.centroid[0], 'centroid_x': np_region.centroid[1],
                'overlap_ratio': max_overlap_ratio_found
            })
            for cd_item in results['cell_data']:
                if cd_item['cell_id'] == best_cell_id:
                    cd_item['nuclei_count']+=1
                    cd_item['nuclei_ids_in_cell'].append(nuc_id)
                    break
            results['cell_nuclei_mapping'][best_cell_id].append(nuc_id)
    return results

def extract_features_for_cell(cd_item, cn_map, nuc_data_list):
    feats = {'cell_area': cd_item['area'], 'cell_perimeter': cd_item['perimeter'],
             'cell_eccentricity': cd_item.get('eccentricity',0)}
    feats['cell_circularity'] = (4*np.pi*feats['cell_area']/(feats['cell_perimeter']**2)) if feats['cell_perimeter']>0 else 0
    maj, minor = cd_item.get('major_axis_length'), cd_item.get('minor_axis_length')
    feats['cell_aspect_ratio'] = (maj/minor) if maj and minor and minor>0 else 1.0

    orig_cell_lbl = cd_item['cell_id']
    nuc_orig_lbls = cn_map.get(orig_cell_lbl, [])
    feats['nuclei_count'] = len(nuc_orig_lbls)

    for k_nuc in ['avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'avg_nucleus_eccentricity',
                  'nucleus_area_std', 'nucleus_displacement_avg', 'nucleus_to_cell_area_ratio']: feats[k_nuc]=0.0
    if feats['nuclei_count'] > 0:
        nuc_details_incell = [nd for nd in nuc_data_list if nd['nucleus_id'] in nuc_orig_lbls and nd['cell_id']==orig_cell_lbl]
        if nuc_details_incell:
            areas = [n['area'] for n in nuc_details_incell]; eccs = [n.get('eccentricity',0) for n in nuc_details_incell]
            feats.update({
                'avg_nucleus_area': np.mean(areas) if areas else 0.0,
                'total_nuclear_area': sum(areas),
                'max_nucleus_area': max(areas) if areas else 0.0,
                'avg_nucleus_eccentricity': np.mean(eccs) if eccs else 0.0,
                'nucleus_area_std': np.std(areas) if len(areas)>1 else 0.0,
                'nucleus_to_cell_area_ratio': (sum(areas)/feats['cell_area']) if feats['cell_area']>0 else 0.0
            })
            disps = [np.sqrt((n['centroid_x']-cd_item['centroid_x'])**2 + (n['centroid_y']-cd_item['centroid_y'])**2) for n in nuc_details_incell]
            feats['nucleus_displacement_avg'] = np.mean(disps) if disps else 0.0

    norm_nuc_A, norm_cell_A = 500, 2000 # Placeholder normal areas
    feats['nuclear_enlargement_factor'] = feats.get('avg_nucleus_area',0.0)/norm_nuc_A if norm_nuc_A>0 and feats.get('avg_nucleus_area',0.0)>0 else 1.0
    feats['cell_enlargement_factor'] = feats['cell_area']/norm_cell_A if norm_cell_A>0 and feats['cell_area']>0 else 1.0

    # Cell Orientation and Flow Alignment
    raw_orientation_rad = cd_item.get('orientation', 0.0)
    feats['cell_orientation_y_axis_deg'] = np.degrees(raw_orientation_rad)

    cell_major_axis_cartesian_rad = (np.pi/2 - raw_orientation_rad)
    cell_major_axis_cartesian_rad %= np.pi
    if cell_major_axis_cartesian_rad < 0: # Ensure positive in [0, pi)
        cell_major_axis_cartesian_rad += np.pi

    feats['cell_major_axis_cartesian_deg'] = np.degrees(cell_major_axis_cartesian_rad)

    flow_dir_norm_deg = ASSUMED_FLOW_DIRECTION_CARTESIAN_DEGREES % 180 # Normalize flow to [0, 180)

    angle_diff_deg = abs(feats['cell_major_axis_cartesian_deg'] - flow_dir_norm_deg)
    angular_misalignment_deg = min(angle_diff_deg, 180.0 - angle_diff_deg) # Acute angle
    feats['angular_misalignment_to_flow_deg'] = angular_misalignment_deg

    if angular_misalignment_deg < FLOW_ALIGNMENT_THRESHOLD_DEGREES:
        feats['flow_orientation_category'] = "Aligned with flow"
    elif abs(angular_misalignment_deg - 90.0) < FLOW_ALIGNMENT_THRESHOLD_DEGREES: # Check perpendicular
        feats['flow_orientation_category'] = "Perpendicular to flow"
    else:
        feats['flow_orientation_category'] = "Oblique to flow"

    return feats

def calculate_senescence_score(df, weights, log_features):
    print("\nCalculating senescence score...")
    present_feats = [f for f in weights.keys() if f in df.columns]
    if not present_feats:
        print("  Warning: No features for senescence score calculation are present. Score will be NaN.")
        df['senescence_score'] = np.nan
        df['senescence_score_normalized'] = np.nan
        return df

    score_subset = df[present_feats].copy()
    for col in log_features:
        if col in score_subset.columns:
            score_subset[col] = np.log1p(score_subset[col])
            print(f"  Log-transformed for scoring: {col}")

    scaler = StandardScaler()
    num_cols = score_subset.select_dtypes(include=np.number).columns
    std_df = pd.DataFrame(index=score_subset.index) # Initialize
    if not num_cols.empty:
        std_vals = scaler.fit_transform(score_subset[num_cols])
        std_df = pd.DataFrame(std_vals, columns=num_cols, index=score_subset.index)
    else:
        print("  Warning: No numeric columns in scoring subset for standardization.")

    df['senescence_score'] = 0.0 # Initialize
    for f, w in weights.items():
        if f in std_df.columns: # Only use features that were successfully standardized
             df['senescence_score'] += std_df[f] * w

    min_s, max_s = df['senescence_score'].min(), df['senescence_score'].max()
    if pd.isna(min_s) or pd.isna(max_s) or min_s == max_s : # Handle NaN or uniform scores
        df['senescence_score_normalized'] = 0.5
        print("  Warning: Senescence score is NaN or uniform across cells. Normalized score set to 0.5.")
    else:
        df['senescence_score_normalized'] = (df['senescence_score']-min_s)/(max_s-min_s)
    print("Senescence score calculation complete.")
    return df

def apply_rule_based_classification(df, rules, default_label):
    print("\n--- Applying Rule-Based Classification ---")
    df['rule_based_classification_granular'] = default_label # Initialize
    all_rule_feats = set(cond[0] for r in rules for cond in r['conditions'])

    missing_features = [feat for feat in all_rule_feats if feat not in df.columns]
    if missing_features:
        print(f"  ERROR: Critical features for rule-based gating are missing: {missing_features}. Aborting rule application.")
        df['cell_type'] = 'Unknown_Rules_Missing_Features' # Assign a specific unknown type
        return df

    for rule in rules:
        print(f"  Applying rule: {rule['name']}")
        eligible_mask = (df['rule_based_classification_granular'] == default_label)
        if not eligible_mask.any():
            print(f"    No cells eligible for rule '{rule['name']}' (all previously classified by other rules).")
            continue

        current_rule_mask = pd.Series([True]*len(df), index=df.index) # Assume all cells satisfy initially
        for feat, op, val_condition in rule['conditions']: # Renamed val to val_condition for clarity
            if feat not in df.columns: # Should be caught above, but as a safeguard
                print(f"    Critical Missing feature '{feat}' for rule '{rule['name']}'. Rule cannot be applied.")
                current_rule_mask[:] = False; break

            feat_series = pd.to_numeric(df[feat], errors='coerce') # Convert to numeric, coercing errors
            nan_mask = feat_series.isnull() # Identify NaNs after conversion

            if op=='>': current_rule_mask &= (feat_series > val_condition)
            elif op=='<': current_rule_mask &= (feat_series < val_condition)
            elif op=='>=': current_rule_mask &= (feat_series >= val_condition)
            elif op=='<=': current_rule_mask &= (feat_series <= val_condition)
            elif op=='==': current_rule_mask &= (feat_series == val_condition)
            elif op=='!=': current_rule_mask &= (feat_series != val_condition)
            else:
                print(f"    Unknown operator '{op}' in rule '{rule['name']}'. Rule condition skipped.")
                current_rule_mask[:] = False; break # Invalidate this rule if an op is unknown
            current_rule_mask[nan_mask] = False # Cells with NaN for this feature do not satisfy the condition

        if not current_rule_mask.any(): # If no cells satisfy all conditions of this rule
            print(f"    No cells met all conditions for rule '{rule['name']}'.")
            continue

        # Apply label only to cells that are eligible (still default_label) AND satisfy current rule
        cells_to_label_now = eligible_mask & current_rule_mask
        df.loc[cells_to_label_now, 'rule_based_classification_granular'] = rule['output_label']
        print(f"    {cells_to_label_now.sum()} cells labeled as '{rule['output_label']}'.")

    # Final binary classification based on granular labels
    df['cell_type'] = np.where(df['rule_based_classification_granular'] == default_label,
                               'Non-senescent', 'Senescent')
    print(f"\nFinal rule-based classification counts (granular):\n{df['rule_based_classification_granular'].value_counts(dropna=False)}")
    print(f"\nFinal rule-based classification counts (binary 'cell_type'):\n{df['cell_type'].value_counts(dropna=False)}")
    return df

def generate_umap_coordinates(df, umap_feats_config, log_feats_config, n_neigh=15, min_d=0.1, rand_state=42):
    print("\n--- Generating UMAP Coordinates ---")
    actual_umap_feats = [f for f in umap_feats_config if f in df.columns]
    if not actual_umap_feats:
        print("  Error: No features specified for UMAP are present in the DataFrame. Skipping UMAP generation.")
        df['umap_x'] = np.nan
        df['umap_y'] = np.nan
        return df

    umap_df = df[actual_umap_feats].copy()
    for col in log_feats_config: # Apply log transform to specified UMAP features
        if col in umap_df.columns:
            umap_df[col] = np.log1p(umap_df[col])
            print(f"  Log-transformed for UMAP: {col}")

    if umap_df.isnull().values.any(): # Handle NaNs by filling with mean
        print(f"  Found {umap_df.isnull().values.sum()} NaN values in UMAP features. Filling with column means.")
        umap_df = umap_df.fillna(umap_df.mean())

    if umap_df.isnull().values.any(): # If NaNs persist (e.g., whole column was NaN)
        print("  Error: NaNs still present in UMAP features after attempting to fill. UMAP cannot proceed.")
        df['umap_x'], df['umap_y'] = np.nan, np.nan
        return df

    # Standardize features for UMAP
    scaler = StandardScaler()
    scaled_feats = scaler.fit_transform(umap_df)

    # Ensure n_neighbors is less than the number of samples
    actual_n_neigh = min(n_neigh, scaled_feats.shape[0]-1)
    if actual_n_neigh < 2: # UMAP requires n_neighbors >= 2
        print(f"  Warning: Not enough samples ({scaled_feats.shape[0]}) for UMAP with n_neighbors={n_neigh}. Skipping UMAP.")
        df['umap_x'], df['umap_y'] = np.nan, np.nan
        return df

    # UMAP reduction
    reducer = umap.UMAP(n_neighbors=actual_n_neigh, min_dist=min_d, random_state=rand_state, n_components=2)
    embedding = reducer.fit_transform(scaled_feats)
    df['umap_x'] = embedding[:,0]
    df['umap_y'] = embedding[:,1]
    print("UMAP coordinates generated.")
    return df

def visualize_results(df_viz, out_dir):
    print("\n--- Creating Visualizations (Rule-Based) ---")
    if df_viz.empty or 'umap_x' not in df_viz or df_viz['umap_x'].isnull().all():
        print("Skipping UMAP plots as UMAP data is missing or empty.")
        return

    # Binary UMAP plot
    plt.figure(figsize=(10,8));
    if 'cell_type' in df_viz and df_viz['cell_type'].notna().any():
        # Define a specific palette for consistency
        binary_palette = {'Senescent':'red',
                          'Non-senescent':'blue',
                          'Unknown_Rules_Missing_Features':'grey'} # Handle this specific unknown type

        # Ensure hue_order matches palette keys for correct color assignment if not all types are present
        hue_order_binary = [cat for cat in binary_palette.keys() if cat in df_viz['cell_type'].unique()]

        sns.scatterplot(x='umap_x', y='umap_y', hue='cell_type',
                        palette=binary_palette, hue_order=hue_order_binary,
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.legend(title='Cell Type (Rule-Based)')
    else:
        plt.scatter(df_viz['umap_x'], df_viz['umap_y'], s=30, alpha=0.5) # Fallback if no cell_type
    plt.title('UMAP by Rule-Based Classification (Binary)'); plt.xlabel('UMAP Dimension 1'); plt.ylabel('UMAP Dimension 2'); plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(os.path.join(out_dir, 'umap_rule_based_binary.png'), dpi=300, bbox_inches='tight'); plt.close()
    print("Binary UMAP visualization saved.")

    # Granular UMAP plot
    if 'rule_based_classification_granular' in df_viz and df_viz['rule_based_classification_granular'].notna().any():
        plt.figure(figsize=(12,10)); # Slightly larger for potentially more legend items
        unique_granular_labels = sorted(df_viz['rule_based_classification_granular'].unique())

        palette_granular = {}
        # Consistent colors for specific important labels
        palette_granular[RULE_BASED_DEFAULT_LABEL] = 'blue'
        if 'Unknown_Rules_Missing_Features' in unique_granular_labels:
            palette_granular['Unknown_Rules_Missing_Features'] = 'grey'

        # Assign colors to senescent rules
        sen_rule_outputs = sorted(list(set(r['output_label'] for r in RULE_BASED_GATES if r['output_label'].startswith("Rule_Sen_"))))
        num_sen_cats = len(sen_rule_outputs)
        # Use a color palette that provides distinct colors
        sen_base_colors = sns.color_palette("husl", n_colors=max(1, num_sen_cats))

        for i, sen_label in enumerate(sen_rule_outputs):
            if sen_label in unique_granular_labels: # Only assign if present in data
                palette_granular[sen_label] = sen_base_colors[i % len(sen_base_colors)]

        # Assign remaining colors if any other labels (e.g. if new rules are added and not covered)
        other_color_idx = 0
        other_base_colors = sns.color_palette("pastel")
        for label in unique_granular_labels:
            if label not in palette_granular:
                palette_granular[label] = other_base_colors[other_color_idx % len(other_base_colors)]
                other_color_idx += 1

        sns.scatterplot(x='umap_x', y='umap_y', hue='rule_based_classification_granular',
                        palette=palette_granular, hue_order=unique_granular_labels, # Ensure consistent order
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.title('UMAP by Granular Rule-Based Classification'); plt.xlabel('UMAP Dimension 1'); plt.ylabel('UMAP Dimension 2'); plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Rule Applied', bbox_to_anchor=(1.05,1), loc='upper left', borderaxespad=0.); plt.tight_layout(rect=[0,0,0.85,1]) # Adjust for external legend
        plt.savefig(os.path.join(out_dir, 'umap_rule_based_granular.png'), dpi=300); plt.close()
        print("Granular UMAP visualization saved.")

def save_results(df_proc, out_dir):
    if df_proc.empty: print("No data to save."); return
    if not os.path.exists(out_dir): os.makedirs(out_dir)

    output_csv_path = os.path.join(out_dir, 'cell_classification_rule_based_full.csv')
    df_proc.to_csv(output_csv_path, index=False)
    print(f"Full rule-based results saved to: {output_csv_path}")

    if 'sample_id' in df_proc and 'cell_type' in df_proc:
        # Create summary by sample
        summary = df_proc.groupby('sample_id')['cell_type'].value_counts(normalize=False).unstack(fill_value=0)

        # Ensure all expected columns are present for calculations, add if missing
        expected_summary_cols = ['Senescent','Non-senescent','Unknown_Rules_Missing_Features']
        for col in expected_summary_cols:
            if col not in summary.columns: summary[col] = 0

        summary['total_cells'] = summary.sum(axis=1) # Recalculate total based on actual columns present

        # Calculate percent senescent robustly
        if 'Senescent' in summary.columns and (summary['total_cells'] > 0).any():
            summary['percent_senescent'] = (summary['Senescent'] / summary['total_cells'] * 100)
        else:
            summary['percent_senescent'] = 0.0 # Default to 0 if no senescent or no total cells

        summary.to_csv(os.path.join(out_dir, 'sample_summary_rule_based.csv'))
        print(f"Rule-based sample summary saved to: {os.path.join(out_dir, 'sample_summary_rule_based.csv')}")

        # Plot senescent percentage by sample
        if 'percent_senescent' in summary.columns:
            plt.figure(figsize=(12,7)) # Adjusted size for better readability of x-ticks
            sns.barplot(x=summary.index, y='percent_senescent', data=summary.reset_index(), color='red')
            plt.title('Percentage of Senescent Cells by Sample (Rule-Based)')
            plt.xlabel('Sample ID')
            plt.ylabel('Senescent Cells (%)')
            plt.xticks(rotation=75, ha='right', fontsize=8) # Rotate more, smaller font
            plt.tight_layout() # Adjust layout to prevent labels from being cut off
            plt.savefig(os.path.join(out_dir, 'senescent_percentage_rule_based.png'), dpi=300); plt.close()
            print("Senescent percentage bar plot saved.")

def visualize_enhanced_classification_on_masks(df_results, cell_mask_dir_path, nuclei_mask_dir_path, output_dir_masks_path):
    print(f"\nGenerating enhanced classification overlays in: {output_dir_masks_path}")
    if not os.path.exists(output_dir_masks_path):
        os.makedirs(output_dir_masks_path)

    required_cols = ['sample_id', 'cell_id_unique', 'original_cell_label', 'rule_based_classification_granular']
    if not all(col in df_results.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_results.columns]
        print(f"  Error: DataFrame missing required columns for mask viz: {missing}. Skipping visualization.")
        return

    classification_lookup = pd.Series(
        df_results.rule_based_classification_granular.values,
        index=df_results.cell_id_unique
    ).to_dict()

    unique_samples = df_results['sample_id'].unique()

    available_cell_masks = {extract_sample_id(f): f for f in os.listdir(cell_mask_dir_path) if f.endswith(('.tif', '.tiff'))}
    available_nuclei_masks = {extract_sample_id(f): f for f in os.listdir(nuclei_mask_dir_path) if f.endswith(('.tif', '.tiff'))}

    for sample_id_val in tqdm(unique_samples, desc="Generating Enhanced Mask Overlays"):
        cell_mask_fname = available_cell_masks.get(sample_id_val)
        nuclei_mask_fname = available_nuclei_masks.get(sample_id_val)

        if not cell_mask_fname:
            print(f"  Warning: Cell mask file not found for sample ID: {sample_id_val}. Skipping overlay.")
            continue

        # print(f"\n  Processing sample for enhanced overlay: {sample_id_val}") # Reduced verbosity inside loop
        labeled_cell_mask_img = load_mask_image(os.path.join(cell_mask_dir_path, cell_mask_fname))
        if labeled_cell_mask_img is None:
            print(f"  Warning: Could not load cell mask for {sample_id_val}. Skipping.")
            continue

        overlay_img = np.zeros((labeled_cell_mask_img.shape[0], labeled_cell_mask_img.shape[1], 3), dtype=np.uint8)

        for cell_region_props in measure.regionprops(labeled_cell_mask_img):
            original_label_from_mask = cell_region_props.label
            lookup_id = f"{sample_id_val}_{original_label_from_mask}"

            granular_status_label = classification_lookup.get(lookup_id, 'Unknown_Label_In_CSV')

            fill_color = COLOR_UNKNOWN
            if granular_status_label == POLYNUCLEATED_SENESCENT_RULE_LABEL:
                fill_color = COLOR_SENESCENT_POLYNUCLEATED
            elif granular_status_label.startswith('Rule_Sen_'):
                fill_color = COLOR_SENESCENT_OTHER_RULES
            elif granular_status_label == RULE_BASED_DEFAULT_LABEL:
                fill_color = COLOR_NON_SENESCENT

            overlay_img[labeled_cell_mask_img == original_label_from_mask] = fill_color

        cell_boundaries_img = segmentation.find_boundaries(labeled_cell_mask_img, mode='outer', background=0)
        overlay_img[cell_boundaries_img] = COLOR_CELL_BOUNDARY

        labeled_nuc_mask_img = None
        if nuclei_mask_fname:
            labeled_nuc_mask_img = load_mask_image(os.path.join(nuclei_mask_dir_path, nuclei_mask_fname))
            if labeled_nuc_mask_img is not None:
                nuc_boundaries_img = segmentation.find_boundaries(labeled_nuc_mask_img, mode='inner', background=0)
                overlay_img[nuc_boundaries_img] = COLOR_NUCLEI_OUTLINE
            # else: # Reduced verbosity
                # print(f"  Warning: Could not load nuclei mask for {sample_id_val} for overlay.")

        fig_plot, ax_plot = None, None
        try:
            h_img, w_img = overlay_img.shape[:2]
            fig_w_calc = max(10, w_img / 100 if w_img > 0 else 10)
            fig_h_calc = max(8, (h_img / 100 if h_img > 0 else 8) * (fig_w_calc / (w_img/100 if w_img > 0 else 1)))

            fig_plot, ax_plot = plt.subplots(figsize=(fig_w_calc, fig_h_calc), dpi=100)
            ax_plot.imshow(overlay_img)

            legend_handles = [
                mpatches.Patch(color=np.array(COLOR_NON_SENESCENT)/255., label=f"{RULE_BASED_DEFAULT_LABEL} (Non-Senescent)"),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_OTHER_RULES)/255., label='Senescent (Other Rules)'),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_POLYNUCLEATED)/255., label=f'Senescent ({POLYNUCLEATED_SENESCENT_RULE_LABEL})')
            ]

            unique_ids_current_sample = [f"{sample_id_val}_{p.label}" for p in measure.regionprops(labeled_cell_mask_img)]
            is_unknown_present_in_image = any(classification_lookup.get(uid, '') == 'Unknown_Label_In_CSV' for uid in unique_ids_current_sample)

            if is_unknown_present_in_image:
                 legend_handles.append(mpatches.Patch(color=np.array(COLOR_UNKNOWN)/255., label='Unknown/Not in CSV'))

            if nuclei_mask_fname and labeled_nuc_mask_img is not None:
                legend_handles.append(mpatches.Patch(color=np.array(COLOR_NUCLEI_OUTLINE)/255., label='Nuclei Outline'))
            legend_handles.append(mpatches.Patch(edgecolor=np.array(COLOR_CELL_BOUNDARY)/255., facecolor='none', label='Cell Boundary', linewidth=1))

            ax_plot.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1.02, 0.5), fontsize='small', title="Legend")
            ax_plot.axis('off')
            plt.tight_layout(rect=[0, 0, 0.83, 1])

            output_img_filename = os.path.join(output_dir_masks_path, f"{sample_id_val}_enhanced_mask_overlay.png")
            plt.savefig(output_img_filename, dpi=200)
            # print(f"    Saved enhanced overlay for {sample_id_val} to {output_img_filename}") # Reduced verbosity
        except Exception as e_plt:
            print(f"    Error during plotting/saving mask overlay for {sample_id_val}: {e_plt}")
        finally:
            if fig_plot:
                plt.close(fig_plot)

    print("\nEnhanced mask overlay visualization complete.")

# --- NEW VISUALIZATION FUNCTION ---
def visualize_senescence_vs_flow_orientation(df_results, output_directory):
    """
    Visualizes the distribution of flow orientation categories for senescent vs. non-senescent cells.
    """
    print("\n--- Visualizing Senescence Status vs. Flow Orientation ---")

    required_cols_flow_viz = ['cell_type', 'flow_orientation_category']
    if not all(col in df_results.columns for col in required_cols_flow_viz):
        missing_cols = [col for col in required_cols_flow_viz if col not in df_results.columns]
        print(f"  Error: DataFrame is missing required columns for this visualization: {missing_cols}. Skipping.")
        return

    # Create a cross-tabulation of cell_type vs. flow_orientation_category
    cross_tab = pd.crosstab(df_results['cell_type'], df_results['flow_orientation_category'])

    # Normalize to get percentages within each cell_type (row-wise normalization)
    cross_tab_percent = cross_tab.apply(lambda x: (x / x.sum() * 100) if x.sum() > 0 else 0, axis=1)

    if cross_tab_percent.empty:
        print("  Warning: Crosstab for senescence vs. flow orientation is empty. Skipping plot.")
        return

    # Define the order of categories for plotting consistency
    orientation_order = ['Aligned with flow', 'Oblique to flow', 'Perpendicular to flow']
    # Ensure all categories are present in columns, add missing ones with 0
    for cat in orientation_order:
        if cat not in cross_tab_percent.columns:
            cross_tab_percent[cat] = 0.0
    cross_tab_percent = cross_tab_percent[orientation_order] # Reorder columns

    # Plotting
    plt.figure(figsize=(10, 7))
    cross_tab_percent.plot(kind='bar', width=0.8) # Adjust width as needed

    plt.title('Flow Orientation by Senescence Status', fontsize=15)
    plt.xlabel('Cell Type', fontsize=12)
    plt.ylabel('Percentage of Cells (%)', fontsize=12)
    plt.xticks(rotation=0, ha='center', fontsize=10)
    plt.yticks(fontsize=10)
    plt.legend(title='Flow Orientation Category', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout to make space for legend

    plot_filename = os.path.join(output_directory, 'senescence_vs_flow_orientation.png')
    try:
        plt.savefig(plot_filename, dpi=300)
        print(f"Senescence vs. flow orientation plot saved to: {plot_filename}")
    except Exception as e:
        print(f"  Error saving senescence vs. flow orientation plot: {e}")
    plt.close()


def main(base_dir, cell_subdir, nuclei_subdir, output_parent_dir):
    print(f"=== Senescence Analysis (Rule-Based with Flow) for {base_dir} ===")
    cell_mask_dir_path = os.path.join(base_dir, cell_subdir)
    nuclei_mask_dir_path = os.path.join(base_dir, nuclei_subdir)
    dataset_folder_name = os.path.basename(base_dir)
    main_output_dir = os.path.join(output_parent_dir, dataset_folder_name, "Senescence_Results")
    if not os.path.exists(main_output_dir):
        os.makedirs(main_output_dir)
        print(f"Created output directory: {main_output_dir}")

    file_pairs_list = find_mask_files(cell_mask_dir_path, nuclei_mask_dir_path)
    if not file_pairs_list:
        print("No file pairs found. Exiting.")
        return

    features_list = []
    for pair_info in tqdm(file_pairs_list, desc="Processing image pairs"):
        # print(f"\nProcessing sample: {pair_info['sample_id']}") # Reduced verbosity
        try:
            cm_img = load_mask_image(pair_info['cell_file'])
            nm_img = load_mask_image(pair_info['nuclei_file'])
            if cm_img is None or nm_img is None:
                print(f"Skipping {pair_info['sample_id']} due to mask loading error.")
                continue

            track_res = accurately_track_nuclei_in_cells(cm_img, nm_img)
            for cell_d in track_res['cell_data']:
                cell_feats = extract_features_for_cell(cell_d, track_res['cell_nuclei_mapping'], track_res['nuclei_data'])
                cell_feats['cell_id_unique'] = f"{pair_info['sample_id']}_{cell_d['cell_id']}"
                cell_feats['sample_id'] = pair_info['sample_id']
                cell_feats['original_cell_label'] = cell_d['cell_id']
                features_list.append(cell_feats)
        except Exception as e_proc:
            print(f"Error processing {pair_info['sample_id']}: {e_proc}")
            import traceback; traceback.print_exc()

    if not features_list:
        print("No cell features were extracted. Please check input files and processing steps.")
        return

    master_df = pd.DataFrame(features_list)
    print(f"\nTotal cells extracted across all samples: {len(master_df)}")

    # Ensure master_df is not empty before proceeding
    if master_df.empty:
        print("Master DataFrame is empty after feature extraction. Cannot proceed with analysis.")
        return

    master_df = calculate_senescence_score(master_df, SENESCENCE_SCORE_WEIGHTS, AREA_FEATURES_TO_LOG)
    master_df = generate_umap_coordinates(master_df, FEATURES_FOR_UMAP, AREA_FEATURES_TO_LOG)
    final_classified_df = apply_rule_based_classification(master_df, RULE_BASED_GATES, RULE_BASED_DEFAULT_LABEL)

    # Ensure final_classified_df is not empty before saving and visualization
    if final_classified_df.empty:
        print("DataFrame is empty after classification. Cannot save results or visualize.")
        return

    save_results(final_classified_df, main_output_dir)
    visualize_results(final_classified_df, main_output_dir)

    enhanced_masks_output_path = os.path.join(main_output_dir, MASK_VISUALIZATION_SUBDIR)
    visualize_enhanced_classification_on_masks(final_classified_df, cell_mask_dir_path, nuclei_mask_dir_path, enhanced_masks_output_path)

    # --- CALL NEW VISUALIZATION FUNCTION ---
    visualize_senescence_vs_flow_orientation(final_classified_df, main_output_dir)


    print("\nRule-Based Analysis, Mask Visualization, and Flow Orientation Plotting complete!")

if __name__ == "__main__":
    # Define base directory for the new dataset
    # Assuming "Static-x20" is at the same level as "flow3-x20"
    # /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-x20

    # Path to the parent directory containing different datasets like "flow3-x20", "Static-x20"
    segmented_data_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented" # Modify as needed
    # Name of the current dataset's folder
    current_dataset_folder_name = "1.4Pa-x20" # Modify as needed

    # Construct the base_dir for the current dataset
    base_input_dir = os.path.join(segmented_data_parent_dir, current_dataset_folder_name)

    # Subdirectory names (assumed to be the same)
    cell_masks_subdir = "Cell_merged_conservative" # Modify as needed
    nuclei_masks_subdir = "Nuclei"                 # Modify as needed

    # Parent directory for all analysis output
    analysis_output_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis" # Modify as needed

    # Check if the base input directory exists
    if not os.path.isdir(base_input_dir):
        print(f"Error: Base input directory not found: {base_input_dir}")
        print(f"Please ensure the '{current_dataset_folder_name}' folder exists at the specified location within '{segmented_data_parent_dir}'.")
    else:
        main(base_input_dir, cell_masks_subdir, nuclei_masks_subdir, analysis_output_parent_dir)

=== Senescence Analysis (Rule-Based with Flow) for /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20 ===

--- Finding and Pairing Mask Files ---
Found 23 cell files, 23 nuclei files
Total matching cell-nuclei file pairs found: 23


Processing image pairs:   0%|          | 0/23 [00:00<?, ?it/s]

Found 326 potential cell objects and 347 potential nuclei objects.


Processing image pairs:   4%|▍         | 1/23 [00:01<00:27,  1.23s/it]

Found 273 potential cell objects and 290 potential nuclei objects.


...

Processing image pairs: 100%|██████████| 23/23 [00:27<00:00,  1.18s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(



Total cells extracted across all samples: 6663

Calculating senescence score...
  Log-transformed for scoring: cell_area
  Log-transformed for scoring: avg_nucleus_area
  Log-transformed for scoring: cell_perimeter
Senescence score calculation complete.

--- Generating UMAP Coordinates ---
  Log-transformed for UMAP: cell_area
  Log-transformed for UMAP: avg_nucleus_area
  Log-transformed for UMAP: max_nucleus_area
  Log-transformed for UMAP: cell_perimeter
UMAP coordinates generated.

--- Applying Rule-Based Classification ---
  Applying rule: Polynucleated
    417 cells labeled as 'Rule_Sen_Poly'.
  Applying rule: Very_Large_Cell
    1651 cells labeled as 'Rule_Sen_VeryLarge'.
  Applying rule: Low_Circularity
    66 cells labeled as 'Rule_Sen_LowCirc'.
  Applying rule: Low_NucToCellRatio
    67 cells labeled as 'Rule_Sen_LowNucRatio'.
  Applying rule: High_Score_Not_Otherwise_Caught
    0 cells labeled as 'Rule_Sen_HighScore'.

Final rule-based classification counts (granular):
rule

Generating Enhanced Mask Overlays: 100%|██████████| 23/23 [00:37<00:00,  1.64s/it]



Enhanced mask overlay visualization complete.

--- Visualizing Senescence Status vs. Flow Orientation ---
Senescence vs. flow orientation plot saved to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/senescence_vs_flow_orientation.png

Rule-Based Analysis, Mask Visualization, and Flow Orientation Plotting complete!


<Figure size 1000x700 with 0 Axes>

In [16]:
import os
import numpy as np
import pandas as pd
# import cv2 # Not directly used in this snippet, but often part of image processing workflows
import re
from scipy import ndimage
from skimage import io, measure, segmentation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans # Kept for potential exploratory use but not central
import umap
from tqdm import tqdm

# --- Configuration & Parameters ---
# Features to be used for UMAP generation
FEATURES_FOR_UMAP = [
    'cell_area', 'cell_perimeter', 'cell_eccentricity', 'cell_circularity',
    'cell_aspect_ratio', 'avg_nucleus_area', 'max_nucleus_area',
    'avg_nucleus_eccentricity', 'nucleus_area_std', 'nucleus_displacement_avg',
    'nucleus_to_cell_area_ratio',
    'nuclear_enlargement_factor', 'cell_enlargement_factor'
]

# Features to log-transform before UMAP and Senescence Score calculation
AREA_FEATURES_TO_LOG = ['cell_area', 'avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'cell_perimeter']

# Weights for calculating the senescence score
SENESCENCE_SCORE_WEIGHTS = {
    'cell_area': 1.5,
    'cell_perimeter': 0.5,
    'cell_eccentricity': 0.5,
    'cell_circularity': -1.0,
    'cell_aspect_ratio': 0.5,
    'avg_nucleus_area': 1.0,
    'avg_nucleus_eccentricity': 0.3,
    'nucleus_to_cell_area_ratio': -1.0,
    'nuclear_enlargement_factor': 1.0,
    'cell_enlargement_factor': 1.5,
    'nucleus_displacement_avg': 0.2,
}

# Rule-based gating definition
RULE_BASED_GATES = [
    {   'name': 'Polynucleated',
        'conditions': [('nuclei_count', '>', 1)],
        'output_label': 'Rule_Sen_Poly' },
    {   'name': 'Very_Large_Cell',
        'conditions': [('cell_area', '>', 6000)], # Example value
        'output_label': 'Rule_Sen_VeryLarge' },
    {   'name': 'Low_Circularity',
        'conditions': [('cell_circularity', '<', 0.2)], # Example value
        'output_label': 'Rule_Sen_LowCirc' },
    {   'name': 'Low_NucToCellRatio',
        'conditions': [('nucleus_to_cell_area_ratio', '<', 0.1)], # Example value
        'output_label': 'Rule_Sen_LowNucRatio' },
    {   'name': 'High_Score_Not_Otherwise_Caught',
        'conditions': [('senescence_score_normalized', '>', 0.85)], # Example value
        'output_label': 'Rule_Sen_HighScore' }
]
RULE_BASED_DEFAULT_LABEL = 'Rule_NonSenescent'

POLYNUCLEATED_SENESCENT_RULE_LABEL = 'Rule_Sen_Poly' # Must match an output_label from RULE_BASED_GATES

# Flow direction and alignment analysis
ASSUMED_FLOW_DIRECTION_CARTESIAN_DEGREES = 0  # Angle of flow in standard Cartesian (0 for X-axis right, 90 for Y-axis up)
FLOW_ALIGNMENT_THRESHOLD_DEGREES = 20         # Tolerance in degrees for classifying alignment (e.g., +/- 20 deg)

# Color Definitions for Mask Visualization
COLOR_NON_SENESCENT = [0, 0, 255]  # Blue
COLOR_SENESCENT_POLYNUCLEATED = [255, 165, 0]  # Orange
COLOR_SENESCENT_OTHER_RULES = [255, 0, 0]  # Red
COLOR_CELL_BOUNDARY = [255, 255, 255]  # White
COLOR_NUCLEI_OUTLINE = [255, 255, 0]  # Yellow
COLOR_UNKNOWN = [128, 128, 128] # Grey

MASK_VISUALIZATION_SUBDIR = "mask_overlays_enhanced"


def extract_sample_id(filename):
    base_name = os.path.splitext(filename)[0]
    if base_name.startswith('denoised_'):
        base_name = base_name[len('denoised_'):]
    pattern = re.compile(r'(.+?_seq\\d+)')
    match = pattern.search(base_name)
    if match: return match.group(1)
    parts = base_name.split('_')
    for i, part in enumerate(parts):
        if part.startswith('seq') and i > 0: return '_'.join(parts[:i+1])
    print(f"Warning: Fallback ID extraction for '{filename}'.")
    return '_'.join(base_name.split('_')[:7]) # Fallback

def find_mask_files(cell_dir, nuclei_dir):
    print("\n--- Finding and Pairing Mask Files ---")
    cell_files = [f for f in os.listdir(cell_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    nuclei_files = [f for f in os.listdir(nuclei_dir) if f.endswith(('.tif', '.tiff')) and not f.startswith('.')]
    print(f"Found {len(cell_files)} cell files, {len(nuclei_files)} nuclei files")
    nuclei_lookup = {extract_sample_id(nf): nf for nf in nuclei_files if extract_sample_id(nf)}
    file_pairs = []
    for cf in cell_files:
        sid = extract_sample_id(cf)
        if sid and sid in nuclei_lookup:
            file_pairs.append({'cell_file': os.path.join(cell_dir, cf),
                               'nuclei_file': os.path.join(nuclei_dir, nuclei_lookup[sid]),
                               'sample_id': sid})
        elif sid: print(f"Warning: No matching nuclei file for cell sample ID: {sid} (from file {cf})")
        else: print(f"Warning: Could not extract sample ID for cell file: {cf}")
    print(f"Total matching cell-nuclei file pairs found: {len(file_pairs)}")
    if not file_pairs and (cell_files or nuclei_files): print("CRITICAL WARNING: No file pairs were matched.")
    return file_pairs

def load_mask_image(filepath):
    try:
        img = io.imread(filepath)
        if img.ndim > 2:
            img = img[..., 0] # Take the first channel if multi-channel

        if img.dtype.kind in 'iu' and np.max(img) > 1: # Already labeled integer mask
            return img.astype(np.uint16)

        if img.dtype == bool or \
           (img.dtype == np.uint8 and np.max(img) <= 1) or \
           img.dtype.kind == 'f': # Binary or float mask, needs labeling
            binary_img = (img > (0.5 if img.dtype.kind == 'f' else 0)).astype(np.uint8)
            labeled_img, _ = ndimage.label(binary_img)
            return labeled_img.astype(np.uint16)

        if img.dtype == np.uint8 and np.max(img) > 1: # uint8 that looks labeled
             return img.astype(np.uint16)

        print(f"Warning: Unhandled image type {img.dtype} for {filepath}. Attempting basic labeling.")
        labeled_img, _ = ndimage.label((img > np.median(img)).astype(np.uint8)) # Fallback
        return labeled_img.astype(np.uint16)

    except Exception as e: print(f"Error loading image {filepath}: {e}"); return None

def accurately_track_nuclei_in_cells(cell_mask, nuclei_mask):
    labeled_cells, num_cells = cell_mask, np.max(cell_mask)
    labeled_nuclei, num_nuclei = nuclei_mask, np.max(nuclei_mask)
    print(f"Found {num_cells} potential cell objects and {num_nuclei} potential nuclei objects.")
    if num_cells == 0 or num_nuclei == 0: return {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    cell_props = measure.regionprops(labeled_cells)
    nuclei_props = measure.regionprops(labeled_nuclei)
    results = {'cell_data': [], 'nuclei_data': [], 'cell_nuclei_mapping': {}}

    for cp in cell_props:
        results['cell_data'].append({
            'cell_id': cp.label, 'area': cp.area, 'perimeter': cp.perimeter,
            'eccentricity': getattr(cp, 'eccentricity', 0),
            'orientation': getattr(cp, 'orientation', 0), # Keep in radians for precision
            'major_axis_length': getattr(cp, 'major_axis_length', 0),
            'minor_axis_length': getattr(cp, 'minor_axis_length', 0),
            'centroid_y': cp.centroid[0], 'centroid_x': cp.centroid[1],
            'nuclei_count': 0, 'nuclei_ids_in_cell': []
        })
        results['cell_nuclei_mapping'][cp.label] = []

    for np_region in nuclei_props:
        nuc_id, nuc_area = np_region.label, np_region.area
        nuc_mask_single = (labeled_nuclei == nuc_id)
        best_cell_id, max_overlap_ratio_found = None, 0.0 # Renamed for clarity
        min_r, min_c, max_r, max_c = np_region.bbox
        candidate_cell_labels_in_bbox = np.unique(labeled_cells[min_r:max_r, min_c:max_c])

        for cell_cand_lbl in candidate_cell_labels_in_bbox:
            if cell_cand_lbl == 0: continue # Skip background label

            overlap_pixels = np.sum(np.logical_and(labeled_cells == cell_cand_lbl, nuc_mask_single))
            if nuc_area > 0: # Avoid division by zero
                current_overlap_ratio = overlap_pixels / nuc_area
                if current_overlap_ratio > max_overlap_ratio_found:
                    max_overlap_ratio_found = current_overlap_ratio
                    best_cell_id = cell_cand_lbl

        if best_cell_id is not None and max_overlap_ratio_found > 0.5: # Assign if >50% of nucleus overlaps
            results['nuclei_data'].append({
                'nucleus_id': nuc_id, 'cell_id': best_cell_id, 'area': nuc_area,
                'eccentricity': getattr(np_region, 'eccentricity', 0),
                'centroid_y': np_region.centroid[0], 'centroid_x': np_region.centroid[1],
                'overlap_ratio': max_overlap_ratio_found
            })
            for cd_item in results['cell_data']:
                if cd_item['cell_id'] == best_cell_id:
                    cd_item['nuclei_count']+=1
                    cd_item['nuclei_ids_in_cell'].append(nuc_id)
                    break
            results['cell_nuclei_mapping'][best_cell_id].append(nuc_id)
    return results

def extract_features_for_cell(cd_item, cn_map, nuc_data_list):
    feats = {'cell_area': cd_item['area'], 'cell_perimeter': cd_item['perimeter'],
             'cell_eccentricity': cd_item.get('eccentricity',0)}
    feats['cell_circularity'] = (4*np.pi*feats['cell_area']/(feats['cell_perimeter']**2)) if feats['cell_perimeter']>0 else 0
    maj, minor = cd_item.get('major_axis_length'), cd_item.get('minor_axis_length')
    feats['cell_aspect_ratio'] = (maj/minor) if maj and minor and minor>0 else 1.0

    orig_cell_lbl = cd_item['cell_id']
    nuc_orig_lbls = cn_map.get(orig_cell_lbl, [])
    feats['nuclei_count'] = len(nuc_orig_lbls)

    for k_nuc in ['avg_nucleus_area', 'total_nuclear_area', 'max_nucleus_area', 'avg_nucleus_eccentricity',
                  'nucleus_area_std', 'nucleus_displacement_avg', 'nucleus_to_cell_area_ratio']: feats[k_nuc]=0.0
    if feats['nuclei_count'] > 0:
        nuc_details_incell = [nd for nd in nuc_data_list if nd['nucleus_id'] in nuc_orig_lbls and nd['cell_id']==orig_cell_lbl]
        if nuc_details_incell:
            areas = [n['area'] for n in nuc_details_incell]; eccs = [n.get('eccentricity',0) for n in nuc_details_incell]
            feats.update({
                'avg_nucleus_area': np.mean(areas) if areas else 0.0,
                'total_nuclear_area': sum(areas),
                'max_nucleus_area': max(areas) if areas else 0.0,
                'avg_nucleus_eccentricity': np.mean(eccs) if eccs else 0.0,
                'nucleus_area_std': np.std(areas) if len(areas)>1 else 0.0,
                'nucleus_to_cell_area_ratio': (sum(areas)/feats['cell_area']) if feats['cell_area']>0 else 0.0
            })
            disps = [np.sqrt((n['centroid_x']-cd_item['centroid_x'])**2 + (n['centroid_y']-cd_item['centroid_y'])**2) for n in nuc_details_incell]
            feats['nucleus_displacement_avg'] = np.mean(disps) if disps else 0.0

    norm_nuc_A, norm_cell_A = 500, 2000 # Placeholder normal areas
    feats['nuclear_enlargement_factor'] = feats.get('avg_nucleus_area',0.0)/norm_nuc_A if norm_nuc_A>0 and feats.get('avg_nucleus_area',0.0)>0 else 1.0
    feats['cell_enlargement_factor'] = feats['cell_area']/norm_cell_A if norm_cell_A>0 and feats['cell_area']>0 else 1.0

    # Cell Orientation and Flow Alignment
    raw_orientation_rad = cd_item.get('orientation', 0.0)
    feats['cell_orientation_y_axis_deg'] = np.degrees(raw_orientation_rad)

    cell_major_axis_cartesian_rad = (np.pi/2 - raw_orientation_rad)
    cell_major_axis_cartesian_rad %= np.pi
    if cell_major_axis_cartesian_rad < 0: # Ensure positive in [0, pi)
        cell_major_axis_cartesian_rad += np.pi

    feats['cell_major_axis_cartesian_deg'] = np.degrees(cell_major_axis_cartesian_rad)

    flow_dir_norm_deg = ASSUMED_FLOW_DIRECTION_CARTESIAN_DEGREES % 180 # Normalize flow to [0, 180)

    angle_diff_deg = abs(feats['cell_major_axis_cartesian_deg'] - flow_dir_norm_deg)
    angular_misalignment_deg = min(angle_diff_deg, 180.0 - angle_diff_deg) # Acute angle
    feats['angular_misalignment_to_flow_deg'] = angular_misalignment_deg

    if angular_misalignment_deg < FLOW_ALIGNMENT_THRESHOLD_DEGREES:
        feats['flow_orientation_category'] = "Aligned with flow"
    elif abs(angular_misalignment_deg - 90.0) < FLOW_ALIGNMENT_THRESHOLD_DEGREES: # Check perpendicular
        feats['flow_orientation_category'] = "Perpendicular to flow"
    else:
        feats['flow_orientation_category'] = "Oblique to flow"

    return feats

def calculate_senescence_score(df, weights, log_features):
    print("\nCalculating senescence score...")
    present_feats = [f for f in weights.keys() if f in df.columns]
    if not present_feats:
        print("  Warning: No features for senescence score calculation are present. Score will be NaN.")
        df['senescence_score'] = np.nan
        df['senescence_score_normalized'] = np.nan
        return df

    score_subset = df[present_feats].copy()
    for col in log_features:
        if col in score_subset.columns:
            score_subset[col] = np.log1p(score_subset[col])
            print(f"  Log-transformed for scoring: {col}")

    scaler = StandardScaler()
    num_cols = score_subset.select_dtypes(include=np.number).columns
    std_df = pd.DataFrame(index=score_subset.index) # Initialize
    if not num_cols.empty:
        std_vals = scaler.fit_transform(score_subset[num_cols])
        std_df = pd.DataFrame(std_vals, columns=num_cols, index=score_subset.index)
    else:
        print("  Warning: No numeric columns in scoring subset for standardization.")

    df['senescence_score'] = 0.0 # Initialize
    for f, w in weights.items():
        if f in std_df.columns: # Only use features that were successfully standardized
             df['senescence_score'] += std_df[f] * w

    min_s, max_s = df['senescence_score'].min(), df['senescence_score'].max()
    if pd.isna(min_s) or pd.isna(max_s) or min_s == max_s : # Handle NaN or uniform scores
        df['senescence_score_normalized'] = 0.5
        print("  Warning: Senescence score is NaN or uniform across cells. Normalized score set to 0.5.")
    else:
        df['senescence_score_normalized'] = (df['senescence_score']-min_s)/(max_s-min_s)
    print("Senescence score calculation complete.")
    return df

def apply_rule_based_classification(df, rules, default_label):
    print("\n--- Applying Rule-Based Classification ---")
    df['rule_based_classification_granular'] = default_label # Initialize
    all_rule_feats = set(cond[0] for r in rules for cond in r['conditions'])

    missing_features = [feat for feat in all_rule_feats if feat not in df.columns]
    if missing_features:
        print(f"  ERROR: Critical features for rule-based gating are missing: {missing_features}. Aborting rule application.")
        df['cell_type'] = 'Unknown_Rules_Missing_Features' # Assign a specific unknown type
        return df

    for rule in rules:
        print(f"  Applying rule: {rule['name']}")
        eligible_mask = (df['rule_based_classification_granular'] == default_label)
        if not eligible_mask.any():
            print(f"    No cells eligible for rule '{rule['name']}' (all previously classified by other rules).")
            continue

        current_rule_mask = pd.Series([True]*len(df), index=df.index) # Assume all cells satisfy initially
        for feat, op, val_condition in rule['conditions']: # Renamed val to val_condition for clarity
            if feat not in df.columns: # Should be caught above, but as a safeguard
                print(f"    Critical Missing feature '{feat}' for rule '{rule['name']}'. Rule cannot be applied.")
                current_rule_mask[:] = False; break

            feat_series = pd.to_numeric(df[feat], errors='coerce') # Convert to numeric, coercing errors
            nan_mask = feat_series.isnull() # Identify NaNs after conversion

            if op=='>': current_rule_mask &= (feat_series > val_condition)
            elif op=='<': current_rule_mask &= (feat_series < val_condition)
            elif op=='>=': current_rule_mask &= (feat_series >= val_condition)
            elif op=='<=': current_rule_mask &= (feat_series <= val_condition)
            elif op=='==': current_rule_mask &= (feat_series == val_condition)
            elif op=='!=': current_rule_mask &= (feat_series != val_condition)
            else:
                print(f"    Unknown operator '{op}' in rule '{rule['name']}'. Rule condition skipped.")
                current_rule_mask[:] = False; break # Invalidate this rule if an op is unknown
            current_rule_mask[nan_mask] = False # Cells with NaN for this feature do not satisfy the condition

        if not current_rule_mask.any(): # If no cells satisfy all conditions of this rule
            print(f"    No cells met all conditions for rule '{rule['name']}'.")
            continue

        # Apply label only to cells that are eligible (still default_label) AND satisfy current rule
        cells_to_label_now = eligible_mask & current_rule_mask
        df.loc[cells_to_label_now, 'rule_based_classification_granular'] = rule['output_label']
        print(f"    {cells_to_label_now.sum()} cells labeled as '{rule['output_label']}'.")

    # Final binary classification based on granular labels
    df['cell_type'] = np.where(df['rule_based_classification_granular'] == default_label,
                               'Non-senescent', 'Senescent')
    print(f"\nFinal rule-based classification counts (granular):\n{df['rule_based_classification_granular'].value_counts(dropna=False)}")
    print(f"\nFinal rule-based classification counts (binary 'cell_type'):\n{df['cell_type'].value_counts(dropna=False)}")
    return df

def generate_umap_coordinates(df, umap_feats_config, log_feats_config, n_neigh=15, min_d=0.1, rand_state=42):
    print("\n--- Generating UMAP Coordinates ---")
    actual_umap_feats = [f for f in umap_feats_config if f in df.columns]
    if not actual_umap_feats:
        print("  Error: No features specified for UMAP are present in the DataFrame. Skipping UMAP generation.")
        df['umap_x'] = np.nan
        df['umap_y'] = np.nan
        return df

    umap_df = df[actual_umap_feats].copy()
    for col in log_feats_config: # Apply log transform to specified UMAP features
        if col in umap_df.columns:
            umap_df[col] = np.log1p(umap_df[col])
            print(f"  Log-transformed for UMAP: {col}")

    if umap_df.isnull().values.any(): # Handle NaNs by filling with mean
        print(f"  Found {umap_df.isnull().values.sum()} NaN values in UMAP features. Filling with column means.")
        umap_df = umap_df.fillna(umap_df.mean())

    if umap_df.isnull().values.any(): # If NaNs persist (e.g., whole column was NaN)
        print("  Error: NaNs still present in UMAP features after attempting to fill. UMAP cannot proceed.")
        df['umap_x'], df['umap_y'] = np.nan, np.nan
        return df

    # Standardize features for UMAP
    scaler = StandardScaler()
    scaled_feats = scaler.fit_transform(umap_df)

    # Ensure n_neighbors is less than the number of samples
    actual_n_neigh = min(n_neigh, scaled_feats.shape[0]-1)
    if actual_n_neigh < 2: # UMAP requires n_neighbors >= 2
        print(f"  Warning: Not enough samples ({scaled_feats.shape[0]}) for UMAP with n_neighbors={n_neigh}. Skipping UMAP.")
        df['umap_x'], df['umap_y'] = np.nan, np.nan
        return df

    # UMAP reduction
    reducer = umap.UMAP(n_neighbors=actual_n_neigh, min_dist=min_d, random_state=rand_state, n_components=2)
    embedding = reducer.fit_transform(scaled_feats)
    df['umap_x'] = embedding[:,0]
    df['umap_y'] = embedding[:,1]
    print("UMAP coordinates generated.")
    return df

def visualize_results(df_viz, out_dir):
    print("\n--- Creating Visualizations (Rule-Based) ---")
    if df_viz.empty or 'umap_x' not in df_viz or df_viz['umap_x'].isnull().all():
        print("Skipping UMAP plots as UMAP data is missing or empty.")
        return

    # Binary UMAP plot
    plt.figure(figsize=(10,8));
    if 'cell_type' in df_viz and df_viz['cell_type'].notna().any():
        # Define a specific palette for consistency
        binary_palette = {'Senescent':'red',
                          'Non-senescent':'blue',
                          'Unknown_Rules_Missing_Features':'grey'} # Handle this specific unknown type

        # Ensure hue_order matches palette keys for correct color assignment if not all types are present
        hue_order_binary = [cat for cat in binary_palette.keys() if cat in df_viz['cell_type'].unique()]

        sns.scatterplot(x='umap_x', y='umap_y', hue='cell_type',
                        palette=binary_palette, hue_order=hue_order_binary,
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.legend(title='Cell Type (Rule-Based)')
    else:
        plt.scatter(df_viz['umap_x'], df_viz['umap_y'], s=30, alpha=0.5) # Fallback if no cell_type
    plt.title('UMAP by Rule-Based Classification (Binary)'); plt.xlabel('UMAP Dimension 1'); plt.ylabel('UMAP Dimension 2'); plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(os.path.join(out_dir, 'umap_rule_based_binary.png'), dpi=300, bbox_inches='tight'); plt.close()
    print("Binary UMAP visualization saved.")

    # Granular UMAP plot
    if 'rule_based_classification_granular' in df_viz and df_viz['rule_based_classification_granular'].notna().any():
        plt.figure(figsize=(12,10)); # Slightly larger for potentially more legend items
        unique_granular_labels = sorted(df_viz['rule_based_classification_granular'].unique())

        palette_granular = {}
        # Consistent colors for specific important labels
        palette_granular[RULE_BASED_DEFAULT_LABEL] = 'blue'
        if 'Unknown_Rules_Missing_Features' in unique_granular_labels:
            palette_granular['Unknown_Rules_Missing_Features'] = 'grey'

        # Assign colors to senescent rules
        sen_rule_outputs = sorted(list(set(r['output_label'] for r in RULE_BASED_GATES if r['output_label'].startswith("Rule_Sen_"))))
        num_sen_cats = len(sen_rule_outputs)
        # Use a color palette that provides distinct colors
        sen_base_colors = sns.color_palette("husl", n_colors=max(1, num_sen_cats))

        for i, sen_label in enumerate(sen_rule_outputs):
            if sen_label in unique_granular_labels: # Only assign if present in data
                palette_granular[sen_label] = sen_base_colors[i % len(sen_base_colors)]

        # Assign remaining colors if any other labels (e.g. if new rules are added and not covered)
        other_color_idx = 0
        other_base_colors = sns.color_palette("pastel")
        for label in unique_granular_labels:
            if label not in palette_granular:
                palette_granular[label] = other_base_colors[other_color_idx % len(other_base_colors)]
                other_color_idx += 1

        sns.scatterplot(x='umap_x', y='umap_y', hue='rule_based_classification_granular',
                        palette=palette_granular, hue_order=unique_granular_labels, # Ensure consistent order
                        data=df_viz, s=30, alpha=0.7, edgecolor='k', linewidth=0.5)
        plt.title('UMAP by Granular Rule-Based Classification'); plt.xlabel('UMAP Dimension 1'); plt.ylabel('UMAP Dimension 2'); plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(title='Rule Applied', bbox_to_anchor=(1.05,1), loc='upper left', borderaxespad=0.); plt.tight_layout(rect=[0,0,0.85,1]) # Adjust for external legend
        plt.savefig(os.path.join(out_dir, 'umap_rule_based_granular.png'), dpi=300); plt.close()
        print("Granular UMAP visualization saved.")

def save_results(df_proc, out_dir):
    if df_proc.empty: print("No data to save."); return
    if not os.path.exists(out_dir): os.makedirs(out_dir)

    output_csv_path = os.path.join(out_dir, 'cell_classification_rule_based_full.csv')
    df_proc.to_csv(output_csv_path, index=False)
    print(f"Full rule-based results saved to: {output_csv_path}")

    if 'sample_id' in df_proc and 'cell_type' in df_proc:
        # Create summary by sample
        summary = df_proc.groupby('sample_id')['cell_type'].value_counts(normalize=False).unstack(fill_value=0)

        # Ensure all expected columns are present for calculations, add if missing
        expected_summary_cols = ['Senescent','Non-senescent','Unknown_Rules_Missing_Features']
        for col in expected_summary_cols:
            if col not in summary.columns: summary[col] = 0

        summary['total_cells'] = summary.sum(axis=1) # Recalculate total based on actual columns present

        # Calculate percent senescent robustly
        if 'Senescent' in summary.columns and (summary['total_cells'] > 0).any():
            summary['percent_senescent'] = (summary['Senescent'] / summary['total_cells'] * 100)
        else:
            summary['percent_senescent'] = 0.0 # Default to 0 if no senescent or no total cells

        summary.to_csv(os.path.join(out_dir, 'sample_summary_rule_based.csv'))
        print(f"Rule-based sample summary saved to: {os.path.join(out_dir, 'sample_summary_rule_based.csv')}")

        # Plot senescent percentage by sample
        if 'percent_senescent' in summary.columns:
            plt.figure(figsize=(12,7)) # Adjusted size for better readability of x-ticks
            sns.barplot(x=summary.index, y='percent_senescent', data=summary.reset_index(), color='red')
            plt.title('Percentage of Senescent Cells by Sample (Rule-Based)')
            plt.xlabel('Sample ID')
            plt.ylabel('Senescent Cells (%)')
            plt.xticks(rotation=75, ha='right', fontsize=8) # Rotate more, smaller font
            plt.tight_layout() # Adjust layout to prevent labels from being cut off
            plt.savefig(os.path.join(out_dir, 'senescent_percentage_rule_based.png'), dpi=300); plt.close()
            print("Senescent percentage bar plot saved.")

def visualize_enhanced_classification_on_masks(df_results, cell_mask_dir_path, nuclei_mask_dir_path, output_dir_masks_path):
    print(f"\nGenerating enhanced classification overlays in: {output_dir_masks_path}")
    if not os.path.exists(output_dir_masks_path):
        os.makedirs(output_dir_masks_path)

    required_cols = ['sample_id', 'cell_id_unique', 'original_cell_label', 'rule_based_classification_granular']
    if not all(col in df_results.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df_results.columns]
        print(f"  Error: DataFrame missing required columns for mask viz: {missing}. Skipping visualization.")
        return

    classification_lookup = pd.Series(
        df_results.rule_based_classification_granular.values,
        index=df_results.cell_id_unique
    ).to_dict()

    unique_samples = df_results['sample_id'].unique()

    available_cell_masks = {extract_sample_id(f): f for f in os.listdir(cell_mask_dir_path) if f.endswith(('.tif', '.tiff'))}
    available_nuclei_masks = {extract_sample_id(f): f for f in os.listdir(nuclei_mask_dir_path) if f.endswith(('.tif', '.tiff'))}

    for sample_id_val in tqdm(unique_samples, desc="Generating Enhanced Mask Overlays"):
        cell_mask_fname = available_cell_masks.get(sample_id_val)
        nuclei_mask_fname = available_nuclei_masks.get(sample_id_val)

        if not cell_mask_fname:
            print(f"  Warning: Cell mask file not found for sample ID: {sample_id_val}. Skipping overlay.")
            continue

        labeled_cell_mask_img = load_mask_image(os.path.join(cell_mask_dir_path, cell_mask_fname))
        if labeled_cell_mask_img is None:
            print(f"  Warning: Could not load cell mask for {sample_id_val}. Skipping.")
            continue

        overlay_img = np.zeros((labeled_cell_mask_img.shape[0], labeled_cell_mask_img.shape[1], 3), dtype=np.uint8)

        for cell_region_props in measure.regionprops(labeled_cell_mask_img):
            original_label_from_mask = cell_region_props.label
            lookup_id = f"{sample_id_val}_{original_label_from_mask}"

            granular_status_label = classification_lookup.get(lookup_id, 'Unknown_Label_In_CSV')

            fill_color = COLOR_UNKNOWN
            if granular_status_label == POLYNUCLEATED_SENESCENT_RULE_LABEL:
                fill_color = COLOR_SENESCENT_POLYNUCLEATED
            elif granular_status_label.startswith('Rule_Sen_'):
                fill_color = COLOR_SENESCENT_OTHER_RULES
            elif granular_status_label == RULE_BASED_DEFAULT_LABEL:
                fill_color = COLOR_NON_SENESCENT

            overlay_img[labeled_cell_mask_img == original_label_from_mask] = fill_color

        cell_boundaries_img = segmentation.find_boundaries(labeled_cell_mask_img, mode='outer', background=0)
        overlay_img[cell_boundaries_img] = COLOR_CELL_BOUNDARY

        labeled_nuc_mask_img = None
        if nuclei_mask_fname:
            labeled_nuc_mask_img = load_mask_image(os.path.join(nuclei_mask_dir_path, nuclei_mask_fname))
            if labeled_nuc_mask_img is not None:
                nuc_boundaries_img = segmentation.find_boundaries(labeled_nuc_mask_img, mode='inner', background=0)
                overlay_img[nuc_boundaries_img] = COLOR_NUCLEI_OUTLINE

        fig_plot, ax_plot = None, None
        try:
            h_img, w_img = overlay_img.shape[:2]
            fig_w_calc = max(10, w_img / 100 if w_img > 0 else 10)
            fig_h_calc = max(8, (h_img / 100 if h_img > 0 else 8) * (fig_w_calc / (w_img/100 if w_img > 0 else 1)))

            fig_plot, ax_plot = plt.subplots(figsize=(fig_w_calc, fig_h_calc), dpi=100)
            ax_plot.imshow(overlay_img)

            legend_handles = [
                mpatches.Patch(color=np.array(COLOR_NON_SENESCENT)/255., label=f"{RULE_BASED_DEFAULT_LABEL} (Non-Senescent)"),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_OTHER_RULES)/255., label='Senescent (Other Rules)'),
                mpatches.Patch(color=np.array(COLOR_SENESCENT_POLYNUCLEATED)/255., label=f'Senescent ({POLYNUCLEATED_SENESCENT_RULE_LABEL})')
            ]

            unique_ids_current_sample = [f"{sample_id_val}_{p.label}" for p in measure.regionprops(labeled_cell_mask_img)]
            is_unknown_present_in_image = any(classification_lookup.get(uid, '') == 'Unknown_Label_In_CSV' for uid in unique_ids_current_sample)

            if is_unknown_present_in_image:
                 legend_handles.append(mpatches.Patch(color=np.array(COLOR_UNKNOWN)/255., label='Unknown/Not in CSV'))

            if nuclei_mask_fname and labeled_nuc_mask_img is not None:
                legend_handles.append(mpatches.Patch(color=np.array(COLOR_NUCLEI_OUTLINE)/255., label='Nuclei Outline'))
            legend_handles.append(mpatches.Patch(edgecolor=np.array(COLOR_CELL_BOUNDARY)/255., facecolor='none', label='Cell Boundary', linewidth=1))

            ax_plot.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1.02, 0.5), fontsize='small', title="Legend")
            ax_plot.axis('off')
            plt.tight_layout(rect=[0, 0, 0.83, 1])

            output_img_filename = os.path.join(output_dir_masks_path, f"{sample_id_val}_enhanced_mask_overlay.png")
            plt.savefig(output_img_filename, dpi=200)
        except Exception as e_plt:
            print(f"    Error during plotting/saving mask overlay for {sample_id_val}: {e_plt}")
        finally:
            if fig_plot:
                plt.close(fig_plot)

    print("\nEnhanced mask overlay visualization complete.")

def visualize_senescence_vs_flow_orientation_categorical(df_results, output_directory):
    """
    Visualizes the distribution of flow orientation categories for senescent vs. non-senescent cells.
    (This is the function added in the previous response for categorical plotting)
    """
    print("\n--- Visualizing Senescence Status vs. Categorical Flow Orientation ---")

    required_cols_flow_viz = ['cell_type', 'flow_orientation_category']
    if not all(col in df_results.columns for col in required_cols_flow_viz):
        missing_cols = [col for col in required_cols_flow_viz if col not in df_results.columns]
        print(f"  Error: DataFrame is missing required columns for categorical flow visualization: {missing_cols}. Skipping.")
        return

    cross_tab = pd.crosstab(df_results['cell_type'], df_results['flow_orientation_category'])
    cross_tab_percent = cross_tab.apply(lambda x: (x / x.sum() * 100) if x.sum() > 0 else 0, axis=1)

    if cross_tab_percent.empty:
        print("  Warning: Crosstab for senescence vs. categorical flow orientation is empty. Skipping plot.")
        return

    orientation_order = ['Aligned with flow', 'Oblique to flow', 'Perpendicular to flow']
    for cat in orientation_order:
        if cat not in cross_tab_percent.columns:
            cross_tab_percent[cat] = 0.0
    cross_tab_percent = cross_tab_percent[orientation_order]

    plt.figure(figsize=(10, 7))
    cross_tab_percent.plot(kind='bar', width=0.8, ax=plt.gca()) # Pass ax to plot on current figure

    plt.title('Flow Orientation Category by Senescence Status', fontsize=15)
    plt.xlabel('Cell Type', fontsize=12)
    plt.ylabel('Percentage of Cells (%)', fontsize=12)
    plt.xticks(rotation=0, ha='center', fontsize=10)
    plt.yticks(fontsize=10)
    plt.legend(title='Flow Orientation', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout(rect=[0, 0, 0.85, 1])

    plot_filename = os.path.join(output_directory, 'senescence_vs_flow_orientation_categorical.png')
    try:
        plt.savefig(plot_filename, dpi=300)
        print(f"Categorical senescence vs. flow orientation plot saved to: {plot_filename}")
    except Exception as e:
        print(f"  Error saving categorical senescence vs. flow orientation plot: {e}")
    plt.close()

# --- NEW: Function to visualize numerical orientation distributions ---
def visualize_numerical_orientation_distributions(df_results, output_directory):
    """
    Visualizes the distribution of numerical flow orientation values
    for senescent vs. non-senescent cells using histograms and box plots.
    """
    print("\n--- Visualizing Numerical Flow Orientation Distributions by Senescence Status ---")

    required_cols_num_viz = ['cell_type', 'angular_misalignment_to_flow_deg']
    if not all(col in df_results.columns for col in required_cols_num_viz):
        missing_cols = [col for col in required_cols_num_viz if col not in df_results.columns]
        print(f"  Error: DataFrame is missing required columns for numerical orientation visualization: {missing_cols}. Skipping.")
        return

    if df_results['angular_misalignment_to_flow_deg'].isnull().all():
        print("  Warning: 'angular_misalignment_to_flow_deg' column is all NaN. Skipping numerical orientation plots.")
        return

    # Define cell types for iteration
    cell_types = df_results['cell_type'].unique()

    # Palette for consistency
    palette = {'Senescent': 'red', 'Non-senescent': 'blue'}
    # Filter out any unexpected cell_type values for plotting if necessary
    plot_cell_types = [ct for ct in cell_types if ct in palette]


    # 1. Overlaid Histograms / Density Plots
    plt.figure(figsize=(10, 6))
    for cell_type_val in plot_cell_types:
        subset = df_results[df_results['cell_type'] == cell_type_val]
        sns.histplot(subset['angular_misalignment_to_flow_deg'],
                     label=cell_type_val,
                     kde=True, # Adds a Kernel Density Estimate curve
                     stat="density", # Shows density instead of counts, good for comparison
                     common_norm=False, # Normalize each histogram independently
                     bins=18, # 0-90 degrees, 5-degree bins
                     color=palette.get(cell_type_val, 'grey'))

    plt.title('Distribution of Angular Misalignment to Flow', fontsize=15)
    plt.xlabel('Angular Misalignment to Flow (degrees)', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.legend(title='Cell Type')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xlim(0, 90) # Angle ranges from 0 to 90
    plt.tight_layout()
    hist_plot_filename = os.path.join(output_directory, 'orientation_misalignment_distribution.png')
    try:
        plt.savefig(hist_plot_filename, dpi=300)
        print(f"Numerical orientation distribution plot saved to: {hist_plot_filename}")
    except Exception as e:
        print(f"  Error saving numerical orientation distribution plot: {e}")
    plt.close()

    # 2. Box Plots
    plt.figure(figsize=(8, 7))
    # Ensure order for boxplot makes sense, e.g., Non-senescent then Senescent
    boxplot_order = [ct for ct in ['Non-senescent', 'Senescent'] if ct in plot_cell_types]

    if boxplot_order: # Proceed only if there are known cell types to plot
        sns.boxplot(x='cell_type', y='angular_misalignment_to_flow_deg',
                    data=df_results[df_results['cell_type'].isin(boxplot_order)],
                    order=boxplot_order,
                    palette=palette,
                    fliersize=3) # Smaller outlier markers

        plt.title('Angular Misalignment to Flow by Cell Type', fontsize=15)
        plt.xlabel('Cell Type', fontsize=12)
        plt.ylabel('Angular Misalignment to Flow (degrees)', fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.ylim(0, 95) # Angle ranges from 0 to 90, give a little space
        plt.tight_layout()
        box_plot_filename = os.path.join(output_directory, 'orientation_misalignment_boxplot.png')
        try:
            plt.savefig(box_plot_filename, dpi=300)
            print(f"Numerical orientation box plot saved to: {box_plot_filename}")
        except Exception as e:
            print(f"  Error saving numerical orientation box plot: {e}")
        plt.close()
    else:
        print("  Warning: No data for specified cell types for box plot. Skipping.")


def main(base_dir, cell_subdir, nuclei_subdir, output_parent_dir):
    print(f"=== Senescence Analysis (Rule-Based with Flow) for {base_dir} ===")
    cell_mask_dir_path = os.path.join(base_dir, cell_subdir)
    nuclei_mask_dir_path = os.path.join(base_dir, nuclei_subdir)
    dataset_folder_name = os.path.basename(base_dir)
    main_output_dir = os.path.join(output_parent_dir, dataset_folder_name, "Senescence_Results")
    if not os.path.exists(main_output_dir):
        os.makedirs(main_output_dir)
        print(f"Created output directory: {main_output_dir}")

    file_pairs_list = find_mask_files(cell_mask_dir_path, nuclei_mask_dir_path)
    if not file_pairs_list:
        print("No file pairs found. Exiting.")
        return

    features_list = []
    for pair_info in tqdm(file_pairs_list, desc="Processing image pairs"):
        try:
            cm_img = load_mask_image(pair_info['cell_file'])
            nm_img = load_mask_image(pair_info['nuclei_file'])
            if cm_img is None or nm_img is None:
                print(f"Skipping {pair_info['sample_id']} due to mask loading error.")
                continue

            track_res = accurately_track_nuclei_in_cells(cm_img, nm_img)
            for cell_d in track_res['cell_data']:
                cell_feats = extract_features_for_cell(cell_d, track_res['cell_nuclei_mapping'], track_res['nuclei_data'])
                cell_feats['cell_id_unique'] = f"{pair_info['sample_id']}_{cell_d['cell_id']}"
                cell_feats['sample_id'] = pair_info['sample_id']
                cell_feats['original_cell_label'] = cell_d['cell_id']
                features_list.append(cell_feats)
        except Exception as e_proc:
            print(f"Error processing {pair_info['sample_id']}: {e_proc}")
            import traceback; traceback.print_exc()

    if not features_list:
        print("No cell features were extracted. Please check input files and processing steps.")
        return

    master_df = pd.DataFrame(features_list)
    print(f"\nTotal cells extracted across all samples: {len(master_df)}")

    if master_df.empty:
        print("Master DataFrame is empty after feature extraction. Cannot proceed with analysis.")
        return

    master_df = calculate_senescence_score(master_df, SENESCENCE_SCORE_WEIGHTS, AREA_FEATURES_TO_LOG)
    master_df = generate_umap_coordinates(master_df, FEATURES_FOR_UMAP, AREA_FEATURES_TO_LOG)
    final_classified_df = apply_rule_based_classification(master_df, RULE_BASED_GATES, RULE_BASED_DEFAULT_LABEL)

    if final_classified_df.empty:
        print("DataFrame is empty after classification. Cannot save results or visualize.")
        return

    save_results(final_classified_df, main_output_dir)
    visualize_results(final_classified_df, main_output_dir) # UMAP plots

    enhanced_masks_output_path = os.path.join(main_output_dir, MASK_VISUALIZATION_SUBDIR)
    visualize_enhanced_classification_on_masks(final_classified_df, cell_mask_dir_path, nuclei_mask_dir_path, enhanced_masks_output_path)

    visualize_senescence_vs_flow_orientation_categorical(final_classified_df, main_output_dir) # Categorical plot

    # --- CALL NEW NUMERICAL ORIENTATION PLOTTING FUNCTION ---
    visualize_numerical_orientation_distributions(final_classified_df, main_output_dir)


    print("\nRule-Based Analysis, Mask Visualization, and All Flow Orientation Plotting complete!")

if __name__ == "__main__":
    segmented_data_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented"
    current_dataset_folder_name = "1.4Pa-x20"
    base_input_dir = os.path.join(segmented_data_parent_dir, current_dataset_folder_name)

    cell_masks_subdir = "Cell_merged_conservative"
    nuclei_masks_subdir = "Nuclei"

    analysis_output_parent_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis"

    if not os.path.isdir(base_input_dir):
        print(f"Error: Base input directory not found: {base_input_dir}")
        print(f"Please ensure the '{current_dataset_folder_name}' folder exists at the specified location within '{segmented_data_parent_dir}'.")
    else:
        main(base_input_dir, cell_masks_subdir, nuclei_masks_subdir, analysis_output_parent_dir)

=== Senescence Analysis (Rule-Based with Flow) for /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/1.4Pa-x20 ===

--- Finding and Pairing Mask Files ---
Found 23 cell files, 23 nuclei files
Total matching cell-nuclei file pairs found: 23


Processing image pairs:   0%|          | 0/23 [00:00<?, ?it/s]

Found 326 potential cell objects and 347 potential nuclei objects.


Processing image pairs:   4%|▍         | 1/23 [00:01<00:25,  1.17s/it]

Found 273 potential cell objects and 290 potential nuclei objects.


...


Total cells extracted across all samples: 6663

Calculating senescence score...
  Log-transformed for scoring: cell_area
  Log-transformed for scoring: avg_nucleus_area
  Log-transformed for scoring: cell_perimeter
Senescence score calculation complete.

--- Generating UMAP Coordinates ---
  Log-transformed for UMAP: cell_area
  Log-transformed for UMAP: avg_nucleus_area
  Log-transformed for UMAP: max_nucleus_area
  Log-transformed for UMAP: cell_perimeter
UMAP coordinates generated.

--- Applying Rule-Based Classification ---
  Applying rule: Polynucleated
    417 cells labeled as 'Rule_Sen_Poly'.
  Applying rule: Very_Large_Cell
    663 cells labeled as 'Rule_Sen_VeryLarge'.
  Applying rule: Low_Circularity
    91 cells labeled as 'Rule_Sen_LowCirc'.
  Applying rule: Low_NucToCellRatio
    279 cells labeled as 'Rule_Sen_LowNucRatio'.
  Applying rule: High_Score_Not_Otherwise_Caught
    83 cells labeled as 'Rule_Sen_HighScore'.

Final rule-based classification counts (granular):
rul

Generating Enhanced Mask Overlays: 100%|██████████| 23/23 [00:37<00:00,  1.62s/it]



Enhanced mask overlay visualization complete.

--- Visualizing Senescence Status vs. Categorical Flow Orientation ---
Categorical senescence vs. flow orientation plot saved to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/senescence_vs_flow_orientation_categorical.png

--- Visualizing Numerical Flow Orientation Distributions by Senescence Status ---
Numerical orientation distribution plot saved to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/orientation_misalignment_distribution.png


<ipython-input-16-38c09b64e0e5>:696: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='cell_type', y='angular_misalignment_to_flow_deg',


Numerical orientation box plot saved to: /content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/1.4Pa-x20/Senescence_Results/orientation_misalignment_boxplot.png

Rule-Based Analysis, Mask Visualization, and All Flow Orientation Plotting complete!
